In [1]:
import sqlite3
import os
import openpyxl
import pandas as pd
import calendar
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, MetaData
from sqlalchemy_schemadisplay import create_schema_graph
import matplotlib.image as mpimg

In [2]:
conn = sqlite3.connect('bank_analytics.db')
cursor = conn.cursor()

In [3]:
def get_period_from_user():
    period_input = input("Введите период данных (в формате DD-MM-YYYY): ")

    try:
        # Изменяем формат на DD-MM-YYYY
        period = datetime.strptime(period_input, "%d-%m-%Y").date()
        return period
    except ValueError:
        print("Ошибка: Некорректный формат даты.")
        exit()

In [4]:
def calculate_days_from_start_of_year(period):
    try:
        month = period.month
        year = period.year

        if 1 <= month <= 12:
            total_days = sum(calendar.monthrange(year, i)[1] for i in range(1, month + 1))
        else:
            print("Ошибка: Некорректный месяц.")
            exit()
        
        return total_days
    except AttributeError:
        print("Ошибка: Некорректный формат периода.")
        exit()

In [5]:
def get_folder_from_user():
    folder = input("Введите путь к папке с данными: ")
    if not os.path.isdir(folder):
        print("Ошибка: Указанный путь не является директорией.")
        exit()
    return folder

In [6]:
def insert_data_into_table(conn, cursor, table_name, data):
    insert_script = f'''
        INSERT INTO {table_name} 
    ("Наименование Банка",
    "За период",
    "Активы Всего",
    "Активы в НацВалюте",
    "Активы в ИнВалюте эквсум",
    "К получению из других банков",
    "Ценные бумаги",
    "Инвестиции",
    "Кредиты всего",
    "Кредиты в НацВалюте",
    "Кредиты в ИнВалюте эквсум",
    "Созданные Резервы по Кредитам",
    "Кредиты до 365 дней",
    "Кредиты от 365 до 730 дней",
    "Кредиты свыше 2 лет",
    "Обязательства всего", 
    "Обязательства в НацВалюте",
    "Обязательства в ИнВалюте эквсум",
    "К оплате ЦБ РУ",
    "К оплате другие банки",
    "Кредиты и лизинговые операции к оплате",
    "Выпущенные ценные бумаги", 
    "Субординированные долговые обязательства",
    "Капитал",
    "Депозиты всего", 
    "Депозиты физлиц ДВ", 
    "Депозиты физлиц Срочный", 
    "Депозиты физлиц Пластик", 
    "Депозиты юрлиц ДВ",
    "Депозиты юрлиц Срочный",
    "Депозиты юрлиц другие",
    "Депозиты в НацВалюте",
    "Депозиты в ИнВалюте эквсум",
    "Депозиты до 365 дней",
    "Депозиты от 365 до 730 дней",
    "Депозиты свыше 2 лет",
    "Доходы всего",
    "Доходы в НацВалюте", 
    "Доходы в ИнВалюте эквсум", 
    "Процентные Доходы всего",
    "Процентные Доходы в НацВалюте",
    "Процентные Доходы в ИнВалюте эквсум",
    "Процентные Доходы по счетам в другихбанках",
    "Процентные Доходы по кредитам", 
    "Процентные Доходы другие",
    "Беспроцентные Доходы всего",
    "Беспроцентные Доходы в НацВалюте",
    "Беспроцентные Доходы в ИнВалюте эквсум",
    "Беспроцентные Доходы комиссионные",
    "Беспроцентные Доходы другие", 
    "В том числе Возврат резервов",
    "В том числе Возврат списанных кредитов",
    "Расходы всего", 
    "Расходы в НацВалюте",
    "Расходы в ИнВалюте эквсум",
    "Процентные Расходы всего",
    "Процентные Расходы в НацВалюте",
    "Процентные Расходы в ИнВалюте эквсум",
    "Процентные Расходы по счетам в другихбанках", 
    "Процентные Расходы по депозитам",
    "Процентные Расходы займам",
    "Беспроцентные Расходы всего",
    "Беспроцентные Расходы в НацВалюте",
    "Беспроцентные Расходы в ИнВалюте эквсум",
    "Операционные Расходы всего",
    "Операционные Расходы зарплата",
    "Чистая прибыль",
    "Чистая прибыль до уплаты налогов",
    "Созданные Резервы",
    "Оценка возможных убытков по кредитам и лизингу",
    "Оценка некредитных убытков",
    "Экономический норматив коэффициент достаточности капитала", 
    "Экономический норматив коэффициент достаточности капитала 1 уровня", 
    "Экономический норматив Левераж",
    "Экономический норматив коэффициент покрытия ликвидности",
    "Экономический норматив норма чистого стабильного финансирования",
    "Количество сотрудников", 
    "Количество филиалов",
    "Количество банковских центров обслуживания",
    "ДБО пользователи Всего", 
    "ДБО пользователи физлица",
    "ДБО пользователи юрлица",
    "Доходные активы",
    "Расходные обязательства",
    "Резервы по стандартным активам",
    "Возврат резеровов")
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
    '''
    cursor.execute(insert_script, data)
    conn.commit()

In [7]:
def insert_data_into_banks(conn, cursor, table_name, data):
    insert_script = f'''
        INSERT INTO {table_name} 
        ("Наименование Банка",
        "За период",
        "Кредиты на активы в %",
        "Депозиты на обязательства в %",
        "Кредиты на депозиты в %",
        "Доля операционных расходов в %",
        "Доля чистой прибыли в %",
        "Доля чистой процентной маржи за период в %",
        "CIR в %",
        "Доходность кредитного портфеля за период в %",
        "Стоимость депозиты за период в %",
        "ROA за период в %",
        "ROE за период в %",
        "Средняя заработная плата сотрудников за месяц в млн.сум",
        "Кредиты на депозиты ссуда к оплате в %",
        "ROA on EBITA в %",
        "Среднемесячная чистая прибыль на сотрудника",
        "ROE on EBITA в %",
        "COR за период в %",
        "Чистая прибыль в тысячах",
        "Чистая прибыль в тысячах по форме 2")
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
    '''
    cursor.execute(insert_script, data)
    conn.commit()

In [8]:
def find_sheet_by_keyword(excel_file, keyword):
    with pd.ExcelFile(excel_file) as xls:
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(excel_file, sheet_name=sheet_name, header=None)
            if keyword.lower() in df.to_string().lower():
                return sheet_name
    return None

In [9]:
period = get_period_from_user()
folder = get_folder_from_user()
days = calculate_days_from_start_of_year(period)
# Прочитайте данные из Excel-файла для каждого поля (для таблицы NBU)
excel_file_nbu = os.path.join(folder, 'nbu.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_nbu, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_nbu, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_nbu, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_nbu, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_nbu, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_nbu, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_nbu1 = period
column_nbu2 = "NBU"
column_nbu3 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[45,'Unnamed: 2'])
column_nbu4 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[45,'Unnamed: 3'])
column_nbu5 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[45,'1-жадвал'])
column_nbu6 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[6,'Unnamed: 2'])
column_nbu7 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[9,'Unnamed: 2'])
column_nbu8 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[16,'Unnamed: 2'])
column_nbu9 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[23,'Unnamed: 2'])
column_nbu10 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[23,'Unnamed: 3'])
column_nbu11 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[23,'1-жадвал'])
column_nbu12 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[24,'Unnamed: 2'])
column_nbu14 = float(pd.read_excel(excel_file_nbu,sheet_name=s17).at[28,'Unnamed: 14'])
column_nbu15 = float(pd.read_excel(excel_file_nbu,sheet_name=s17).at[28,'Unnamed: 16'])
column_nbu13 = column_nbu9 - column_nbu14 - column_nbu15
column_nbu16 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[61,'Unnamed: 2'])
column_nbu17 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[61,'Unnamed: 3'])
column_nbu18 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[61,'1-жадвал'])
column_nbu19 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[50,'Unnamed: 2'])
column_nbu20 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[51,'Unnamed: 2'])
column_nbu21 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[53,'Unnamed: 2'])
column_nbu22 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[54,'Unnamed: 2'])
column_nbu23 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[55,'Unnamed: 2'])
column_nbu24 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[75,'Unnamed: 2'])
column_nbu25 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[18,'Unnamed: 18'])
column_nbu26 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[5,'Unnamed: 18'])
column_nbu27 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[11,'Unnamed: 18'])
column_nbu28 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[14,'Unnamed: 18'])
column_nbu29 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[9,'Unnamed: 18'])
column_nbu30 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[11,'Unnamed: 18'])
column_nbu31 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[14,'Unnamed: 18'])
column_nbu32 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[18,'Unnamed: 19'])
column_nbu33 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[18,'Unnamed: 20'])
column_nbu35 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[18,'Unnamed: 14'])
column_nbu36 = float(pd.read_excel(excel_file_nbu,sheet_name=s18).at[18,'Unnamed: 16'])
column_nbu34 = column_nbu25 - column_nbu35 - column_nbu36
column_nbu40 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[13,'Unnamed: 2'])
column_nbu41 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[13,'Unnamed: 3'])
column_nbu42 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[13,'3-жадвал'])
column_nbu43 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[5,'Unnamed: 2'])
column_nbu44 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[10,'Unnamed: 2'])
column_nbu45 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[12,'Unnamed: 2'])
column_nbu46 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[43,'Unnamed: 2'])
column_nbu47 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[43,'Unnamed: 3'])
column_nbu48 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[43,'3-жадвал'])
column_nbu37 = column_nbu40 + column_nbu46
column_nbu38 = column_nbu41 + column_nbu47
column_nbu39 = column_nbu42 + column_nbu48
column_nbu49 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[35,'Unnamed: 2'])
column_nbu50 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[39,'Unnamed: 2'])
column_nbu51 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[41,'Unnamed: 2'])
column_nbu52 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[42,'Unnamed: 2'])
column_nbu56 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[25,'Unnamed: 2'])
column_nbu57 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[25,'Unnamed: 3'])
column_nbu58 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[25,'3-жадвал'])
column_nbu59 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[19,'Unnamed: 2'])
column_nbu60 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[19,'Unnamed: 2'])
column_nbu61 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[24,'Unnamed: 2'])
column_nbu62 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[51,'Unnamed: 2'])
column_nbu63 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[51,'Unnamed: 3'])
column_nbu64 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[51,'3-жадвал'])
column_nbu65 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[67,'Unnamed: 2'])
column_nbu66 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[56,'Unnamed: 2'])
column_nbu67 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[76,'Unnamed: 2'])
column_nbu53 = column_nbu37 - column_nbu67
column_nbu54 = column_nbu57 + column_nbu63 + column_nbu65
column_nbu55 = column_nbu58 + column_nbu64
column_nbu68 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[69,'Unnamed: 2'])
column_nbu70 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[28,'Unnamed: 2'])
column_nbu71 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[31,'Unnamed: 2'])
column_nbu69 = column_nbu70 + column_nbu71
column_nbu72 = float(pd.read_excel(excel_file_nbu,sheet_name=s22).at[5,'22-жадвал'])
column_nbu73 = float(pd.read_excel(excel_file_nbu,sheet_name=s22).at[7,'22-жадвал'])
column_nbu74 = float(pd.read_excel(excel_file_nbu,sheet_name=s22).at[11,'22-жадвал'])
column_nbu75 = float(pd.read_excel(excel_file_nbu,sheet_name=s22).at[17,'22-жадвал'])
column_nbu76 = float(pd.read_excel(excel_file_nbu,sheet_name=s22).at[20,'22-жадвал'])
column_nbu77 = 0
if s23 is not None:
    column_nbu77 = float(pd.read_excel(excel_file_nbu,sheet_name=s23).at[3,'23-жадвал'])
column_nbu78 = 0
if s23 is not None:
    column_nbu78 = float(pd.read_excel(excel_file_nbu,sheet_name=s23).at[6,'23-жадвал'])
column_nbu79 = 0
if s23 is not None:
    column_nbu79 = float(pd.read_excel(excel_file_nbu,sheet_name=s23).at[9,'23-жадвал'])
column_nbu83 = 0
if s23 is not None:
    column_nbu83 = float(pd.read_excel(excel_file_nbu,sheet_name=s23).at[10,'23-жадвал'])
column_nbu84 = 0
if s23 is not None: 
    column_nbu84 = float(pd.read_excel(excel_file_nbu,sheet_name=s23).at[12,'23-жадвал'])
column_nbu85 = column_nbu83 - column_nbu84
column_nbu89 = column_nbu9 + column_nbu6/2 + column_nbu7 + column_nbu8
column_nbu90 = column_nbu25 + column_nbu19 + column_nbu20/2 + column_nbu21 + column_nbu22 + column_nbu24
column_nbu91 = float(pd.read_excel(excel_file_nbu,sheet_name=s1).at[37,'Unnamed: 2'])
column_nbu92 = float(pd.read_excel(excel_file_nbu,sheet_name=s3).at[41,'Unnamed: 2'])

# Прочитайте данные из Excel-файла для каждого поля (для таблицы AAB)
excel_file_aab = os.path.join(folder, 'aab.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_aab, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_aab, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_aab, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_aab, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_aab, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_aab, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений

column_aab1 = period
column_aab2 = "ASIA ALIANCE"
column_aab3 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[45,'Unnamed: 2'])
column_aab4 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[45,'Unnamed: 3'])
column_aab5 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[45,'1-жадвал'])
column_aab6 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[6,'Unnamed: 2'])
column_aab7 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[9,'Unnamed: 2'])
column_aab8 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[16,'Unnamed: 2'])
column_aab9 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[23,'Unnamed: 2'])
column_aab10 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[23,'Unnamed: 3'])
column_aab11 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[23,'1-жадвал'])
column_aab12 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[24,'Unnamed: 2'])
column_aab14 = float(pd.read_excel(excel_file_aab,sheet_name=s17).at[28,'Unnamed: 14'])
column_aab15 = float(pd.read_excel(excel_file_aab,sheet_name=s17).at[28,'Unnamed: 16'])
column_aab13 = column_aab9 - column_aab14 - column_aab15
column_aab16 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[61,'Unnamed: 2'])
column_aab17 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[61,'Unnamed: 3'])
column_aab18 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[61,'1-жадвал'])
column_aab19 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[50,'Unnamed: 2'])
column_aab20 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[51,'Unnamed: 2'])
column_aab21 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[53,'Unnamed: 2'])
column_aab22 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[54,'Unnamed: 2'])
column_aab23 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[55,'Unnamed: 2'])
column_aab24 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[75,'Unnamed: 2'])
column_aab25 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[18,'Unnamed: 18'])
column_aab26 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[5,'Unnamed: 18'])
column_aab27 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[11,'Unnamed: 18'])
column_aab28 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[14,'Unnamed: 18'])
column_aab29 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[9,'Unnamed: 18'])
column_aab30 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_aab,sheet_name=s18).at[11,'Unnamed: 18'])
column_aab31 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_aab,sheet_name=s18).at[14,'Unnamed: 18'])
column_aab32 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[18,'Unnamed: 19'])
column_aab33 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[18,'Unnamed: 20'])
column_aab35 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[18,'Unnamed: 14'])
column_aab36 = float(pd.read_excel(excel_file_aab,sheet_name=s18).at[18,'Unnamed: 16'])
column_aab34 = column_aab25 - column_aab35 - column_aab36
column_aab40 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[13,'Unnamed: 2'])
column_aab41 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[13,'Unnamed: 3'])
column_aab42 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[13,'3-жадвал'])
column_aab43 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[5,'Unnamed: 2'])
column_aab44 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[10,'Unnamed: 2'])
column_aab45 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[12,'Unnamed: 2'])
column_aab46 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[43,'Unnamed: 2'])
column_aab47 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[43,'Unnamed: 3'])
column_aab48 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[43,'3-жадвал'])
column_aab37 = column_aab40 + column_aab46
column_aab38 = column_aab41 + column_aab47
column_aab39 = column_aab42 + column_aab48
column_aab49 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[35,'Unnamed: 2'])
column_aab50 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[39,'Unnamed: 2'])
column_aab51 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[41,'Unnamed: 2'])
column_aab52 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[42,'Unnamed: 2'])
column_aab56 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[25,'Unnamed: 2'])
column_aab57 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[25,'Unnamed: 3'])
column_aab58 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[25,'3-жадвал'])
column_aab59 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[19,'Unnamed: 2'])
column_aab60 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[19,'Unnamed: 2'])
column_aab61 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[24,'Unnamed: 2'])
column_aab62 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[51,'Unnamed: 2'])
column_aab63 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[51,'Unnamed: 3'])
column_aab64 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[51,'3-жадвал'])
column_aab65 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[67,'Unnamed: 2'])
column_aab66 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[56,'Unnamed: 2'])
column_aab67 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[76,'Unnamed: 2'])
column_aab53 = column_aab37 - column_aab67
column_aab54 = column_aab57 + column_aab63 + column_aab65
column_aab55 = column_aab58 + column_aab64
column_aab68 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[69,'Unnamed: 2'])
column_aab70 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[28,'Unnamed: 2'])
column_aab71 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_aab,sheet_name=s3).at[31,'Unnamed: 2'])
column_aab69 = column_aab70 + column_aab71
column_aab72 = float(pd.read_excel(excel_file_aab,sheet_name=s22).at[5,'22-жадвал'])
column_aab73 = float(pd.read_excel(excel_file_aab,sheet_name=s22).at[7,'22-жадвал'])
column_aab74 = float(pd.read_excel(excel_file_aab,sheet_name=s22).at[11,'22-жадвал'])
column_aab75 = float(pd.read_excel(excel_file_aab,sheet_name=s22).at[17,'22-жадвал'])
column_aab76 = float(pd.read_excel(excel_file_aab,sheet_name=s22).at[20,'22-жадвал'])
column_aab77 = 0
if s23 is not None:
    column_aab77 = float(pd.read_excel(excel_file_aab,sheet_name=s23).at[3,'23-жадвал'])
column_aab78 = 0
if s23 is not None:
    column_aab78 = float(pd.read_excel(excel_file_aab,sheet_name=s23).at[6,'23-жадвал'])
column_aab79 = 0
if s23 is not None:
    column_aab79 = float(pd.read_excel(excel_file_aab,sheet_name=s23).at[9,'23-жадвал'])
column_aab83 = 0
if s23 is not None:
    column_aab83 = float(pd.read_excel(excel_file_aab,sheet_name=s23).at[10,'23-жадвал'])
column_aab84 = 0
if s23 is not None:
    column_aab84 = float(pd.read_excel(excel_file_aab,sheet_name=s23).at[12,'23-жадвал'])
column_aab85 = column_aab83 - column_aab84
column_aab89 = column_aab9 + column_aab6/2 + column_aab7 + column_aab8
column_aab90 = column_aab25 + column_aab19 + column_aab20/2 + column_aab21 + column_aab22 + column_aab24
column_aab91 = float(pd.read_excel(excel_file_aab,sheet_name=s1).at[37,'Unnamed: 2'])
column_aab92 = float(pd.read_excel(excel_file_aab,sheet_name=s3).at[41,'Unnamed: 2'])
    
# Прочитайте данные из Excel-файла для каждого поля (для таблицы ASAKA)
excel_file_asaka = os.path.join(folder, 'asakabank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_asaka, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_asaka, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_asaka, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_asaka, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_asaka, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_asaka, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_asaka1 = period
column_asaka2 = "ASAKA"
column_asaka3 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[45,'Unnamed: 2'])
column_asaka4 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[45,'Unnamed: 3'])
column_asaka5 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[45,'1-жадвал'])
column_asaka6 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[6,'Unnamed: 2'])
column_asaka7 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[9,'Unnamed: 2'])
column_asaka8 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[16,'Unnamed: 2'])
column_asaka9 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[23,'Unnamed: 2'])
column_asaka10 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[23,'Unnamed: 3'])
column_asaka11 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[23,'1-жадвал'])
column_asaka12 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[24,'Unnamed: 2'])
column_asaka14 = float(pd.read_excel(excel_file_asaka,sheet_name=s17).at[28,'Unnamed: 14'])
column_asaka15 = float(pd.read_excel(excel_file_asaka,sheet_name=s17).at[28,'Unnamed: 16'])
column_asaka13 = column_asaka9 - column_asaka14 - column_asaka15
column_asaka16 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[61,'Unnamed: 2'])
column_asaka17 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[61,'Unnamed: 3'])
column_asaka18 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[61,'1-жадвал'])
column_asaka19 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[50,'Unnamed: 2'])
column_asaka20 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[51,'Unnamed: 2'])
column_asaka21 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[53,'Unnamed: 2'])
column_asaka22 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[54,'Unnamed: 2'])
column_asaka23 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[55,'Unnamed: 2'])
column_asaka24 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[75,'Unnamed: 2'])
column_asaka25 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[18,'Unnamed: 18'])
column_asaka26 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[5,'Unnamed: 18'])
column_asaka27 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[11,'Unnamed: 18'])
column_asaka28 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[14,'Unnamed: 18'])
column_asaka29 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[9,'Unnamed: 18'])
column_asaka30 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[11,'Unnamed: 18'])
column_asaka31 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[14,'Unnamed: 18'])
column_asaka32 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[18,'Unnamed: 19'])
column_asaka33 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[18,'Unnamed: 20'])
column_asaka35 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[18,'Unnamed: 14'])
column_asaka36 = float(pd.read_excel(excel_file_asaka,sheet_name=s18).at[18,'Unnamed: 16'])
column_asaka34 = column_asaka25 - column_asaka35 - column_asaka36
column_asaka40 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[13,'Unnamed: 2'])
column_asaka41 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[13,'Unnamed: 3'])
column_asaka42 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[13,'3-жадвал'])
column_asaka43 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[5,'Unnamed: 2'])
column_asaka44 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[10,'Unnamed: 2'])
column_asaka45 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[12,'Unnamed: 2'])
column_asaka46 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[43,'Unnamed: 2'])
column_asaka47 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[43,'Unnamed: 3'])
column_asaka48 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[43,'3-жадвал'])
column_asaka37 = column_asaka40 + column_asaka46
column_asaka38 = column_asaka41 + column_asaka47
column_asaka39 = column_asaka42 + column_asaka48
column_asaka49 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[35,'Unnamed: 2'])
column_asaka50 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[39,'Unnamed: 2'])
column_asaka51 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[41,'Unnamed: 2'])
column_asaka52 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[42,'Unnamed: 2'])
column_asaka56 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[25,'Unnamed: 2'])
column_asaka57 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[25,'Unnamed: 3'])
column_asaka58 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[25,'3-жадвал'])
column_asaka59 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[19,'Unnamed: 2'])
column_asaka60 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[19,'Unnamed: 2'])
column_asaka61 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[24,'Unnamed: 2'])
column_asaka62 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[51,'Unnamed: 2'])
column_asaka63 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[51,'Unnamed: 3'])
column_asaka64 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[51,'3-жадвал'])
column_asaka65 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[67,'Unnamed: 2'])
column_asaka66 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[56,'Unnamed: 2'])
column_asaka67 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[76,'Unnamed: 2'])
column_asaka53 = column_asaka37 - column_asaka67
column_asaka54 = column_asaka57 + column_asaka63 + column_asaka65
column_asaka55 = column_asaka58 + column_asaka64
column_asaka68 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[69,'Unnamed: 2'])
column_asaka70 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[28,'Unnamed: 2'])
column_asaka71 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[31,'Unnamed: 2'])
column_asaka69 = column_asaka70 + column_asaka71
column_asaka72 = float(pd.read_excel(excel_file_asaka,sheet_name=s22).at[5,'22-жадвал'])
column_asaka73 = float(pd.read_excel(excel_file_asaka,sheet_name=s22).at[7,'22-жадвал'])
column_asaka74 = float(pd.read_excel(excel_file_asaka,sheet_name=s22).at[11,'22-жадвал'])
column_asaka75 = float(pd.read_excel(excel_file_asaka,sheet_name=s22).at[17,'22-жадвал'])
column_asaka76 = float(pd.read_excel(excel_file_asaka,sheet_name=s22).at[20,'22-жадвал'])
column_asaka77 = 0
if s23 is not None:
    column_asaka77 = float(pd.read_excel(excel_file_asaka,sheet_name=s23).at[3,'23-жадвал'])
column_asaka78 = 0
if s23 is not None:
    column_asaka78 = float(pd.read_excel(excel_file_asaka,sheet_name=s23).at[6,'23-жадвал'])
column_asaka79 = 0
if s23 is not None:
    column_asaka79 = float(pd.read_excel(excel_file_asaka,sheet_name=s23).at[9,'23-жадвал'])
column_asaka83 = 0
if s23 is not None:
    column_asaka83 = float(pd.read_excel(excel_file_asaka,sheet_name=s23).at[10,'23-жадвал'])
column_asaka84 = 0
if s23 is not None:
    column_asaka84 = float(pd.read_excel(excel_file_asaka,sheet_name=s23).at[12,'23-жадвал'])
column_asaka85 = column_asaka83 - column_asaka84
column_asaka89 = column_asaka9 + column_asaka6/2 + column_asaka7 + column_asaka8
column_asaka90 = column_asaka25 + column_asaka19 + column_asaka20/2 + column_asaka21 + column_asaka22 + column_asaka24
column_asaka91 = float(pd.read_excel(excel_file_asaka,sheet_name=s1).at[37,'Unnamed: 2'])
column_asaka92 = float(pd.read_excel(excel_file_asaka,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы XALQ)
excel_file_xb = os.path.join(folder, 'xb.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_xb, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_xb, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_xb, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_xb, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_xb, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_xb, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_xb1 = period
column_xb2 = "XALQ"
column_xb3 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[45,'Unnamed: 2'])
column_xb4 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[45,'Unnamed: 3'])
column_xb5 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[45,'1-жадвал'])
column_xb6 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[6,'Unnamed: 2'])
column_xb7 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[9,'Unnamed: 2'])
column_xb8 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[16,'Unnamed: 2'])
column_xb9 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[23,'Unnamed: 2'])
column_xb10 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[23,'Unnamed: 3'])
column_xb11 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[23,'1-жадвал'])
column_xb12 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[24,'Unnamed: 2'])
column_xb14 = float(pd.read_excel(excel_file_xb,sheet_name=s17).at[28,'Unnamed: 14'])
column_xb15 = float(pd.read_excel(excel_file_xb,sheet_name=s17).at[28,'Unnamed: 16'])
column_xb13 = column_xb9 - column_xb14 - column_xb15
column_xb16 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[61,'Unnamed: 2'])
column_xb17 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[61,'Unnamed: 3'])
column_xb18 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[61,'1-жадвал'])
column_xb19 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[50,'Unnamed: 2'])
column_xb20 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[51,'Unnamed: 2'])
column_xb21 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[53,'Unnamed: 2'])
column_xb22 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[54,'Unnamed: 2'])
column_xb23 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[55,'Unnamed: 2'])
column_xb24 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[75,'Unnamed: 2'])
column_xb25 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[18,'Unnamed: 18'])
column_xb26 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[5,'Unnamed: 18'])
column_xb27 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[11,'Unnamed: 18'])
column_xb28 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[14,'Unnamed: 18'])
column_xb29 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[9,'Unnamed: 18'])
column_xb30 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_xb,sheet_name=s18).at[11,'Unnamed: 18'])
column_xb31 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_xb,sheet_name=s18).at[14,'Unnamed: 18'])
column_xb32 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[18,'Unnamed: 19'])
column_xb33 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[18,'Unnamed: 20'])
column_xb35 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[18,'Unnamed: 14'])
column_xb36 = float(pd.read_excel(excel_file_xb,sheet_name=s18).at[18,'Unnamed: 16'])
column_xb34 = column_xb25 - column_xb35 - column_xb36
column_xb40 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[13,'Unnamed: 2'])
column_xb41 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[13,'Unnamed: 3'])
column_xb42 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[13,'3-жадвал'])
column_xb43 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[5,'Unnamed: 2'])
column_xb44 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[10,'Unnamed: 2'])
column_xb45 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[12,'Unnamed: 2'])
column_xb46 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[43,'Unnamed: 2'])
column_xb47 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[43,'Unnamed: 3'])
column_xb48 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[43,'3-жадвал'])
column_xb37 = column_xb40 + column_xb46
column_xb38 = column_xb41 + column_xb47
column_xb39 = column_xb42 + column_xb48
column_xb49 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[35,'Unnamed: 2'])
column_xb50 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[39,'Unnamed: 2'])
column_xb51 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[41,'Unnamed: 2'])
column_xb52 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[42,'Unnamed: 2'])
column_xb56 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[25,'Unnamed: 2'])
column_xb57 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[25,'Unnamed: 3'])
column_xb58 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[25,'3-жадвал'])
column_xb59 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[19,'Unnamed: 2'])
column_xb60 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[19,'Unnamed: 2'])
column_xb61 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[24,'Unnamed: 2'])
column_xb62 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[51,'Unnamed: 2'])
column_xb63 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[51,'Unnamed: 3'])
column_xb64 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[51,'3-жадвал'])
column_xb65 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[67,'Unnamed: 2'])
column_xb66 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[56,'Unnamed: 2'])
column_xb67 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[76,'Unnamed: 2'])
column_xb53 = column_xb37 - column_xb67
column_xb54 = column_xb57 + column_xb63 + column_xb65
column_xb55 = column_xb58 + column_xb64
column_xb68 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[69,'Unnamed: 2'])
column_xb70 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[28,'Unnamed: 2'])
column_xb71 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_xb,sheet_name=s3).at[31,'Unnamed: 2'])
column_xb69 = column_xb70 + column_xb71
column_xb72 = float(pd.read_excel(excel_file_xb,sheet_name=s22).at[5,'22-жадвал'])
column_xb73 = float(pd.read_excel(excel_file_xb,sheet_name=s22).at[7,'22-жадвал'])
column_xb74 = float(pd.read_excel(excel_file_xb,sheet_name=s22).at[11,'22-жадвал'])
column_xb75 = float(pd.read_excel(excel_file_xb,sheet_name=s22).at[17,'22-жадвал'])
column_xb76 = float(pd.read_excel(excel_file_xb,sheet_name=s22).at[20,'22-жадвал'])
column_xb77 = 0
if s23 is not None:
    column_xb77 = float(pd.read_excel(excel_file_xb,sheet_name=s23).at[3,'23-жадвал'])
column_xb78 = 0
if s23 is not None:
    column_xb78 = float(pd.read_excel(excel_file_xb,sheet_name=s23).at[6,'23-жадвал'])
column_xb79 = 0
if s23 is not None:
    column_xb79 = float(pd.read_excel(excel_file_xb,sheet_name=s23).at[9,'23-жадвал'])
column_xb83 = 0
if s23 is not None:
    column_xb83 = float(pd.read_excel(excel_file_xb,sheet_name=s23).at[10,'23-жадвал'])
column_xb84 = 0
if s23 is not None:
    column_xb84 = float(pd.read_excel(excel_file_xb,sheet_name=s23).at[12,'23-жадвал'])
column_xb85 = column_xb83 - column_xb84
column_xb89 = column_xb9 + column_xb6/2 + column_xb7 + column_xb8
column_xb90 = column_xb25 + column_xb19 + column_xb20/2 + column_xb21 + column_xb22 + column_xb24
column_xb91 = float(pd.read_excel(excel_file_xb,sheet_name=s1).at[37,'Unnamed: 2'])
column_xb92 = float(pd.read_excel(excel_file_xb,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы POYTAXT)
excel_file_poytaxt = os.path.join(folder, 'poytaxtbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_poytaxt, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_poytaxt, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_poytaxt, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_poytaxt, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_poytaxt, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_poytaxt, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_poytaxt1 = period
column_poytaxt2 = "POYTAXT"
column_poytaxt3 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[45,'Unnamed: 2'])
column_poytaxt4 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[45,'Unnamed: 3'])
column_poytaxt5 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[45,'1-жадвал'])
column_poytaxt6 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[6,'Unnamed: 2'])
column_poytaxt7 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[9,'Unnamed: 2'])
column_poytaxt8 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[16,'Unnamed: 2'])
column_poytaxt9 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[23,'Unnamed: 2'])
column_poytaxt10 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[23,'Unnamed: 3'])
column_poytaxt11 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[23,'1-жадвал'])
column_poytaxt12 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[24,'Unnamed: 2'])
column_poytaxt14 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s17).at[28,'Unnamed: 14'])
column_poytaxt15 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s17).at[28,'Unnamed: 16'])
column_poytaxt13 = column_poytaxt9 - column_poytaxt14 - column_poytaxt15
column_poytaxt16 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[61,'Unnamed: 2'])
column_poytaxt17 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[61,'Unnamed: 3'])
column_poytaxt18 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[61,'1-жадвал'])
column_poytaxt19 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[50,'Unnamed: 2'])
column_poytaxt20 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[51,'Unnamed: 2'])
column_poytaxt21 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[53,'Unnamed: 2'])
column_poytaxt22 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[54,'Unnamed: 2'])
column_poytaxt23 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[55,'Unnamed: 2'])
column_poytaxt24 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[75,'Unnamed: 2'])
column_poytaxt25 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[18,'Unnamed: 18'])
column_poytaxt26 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[5,'Unnamed: 18'])
column_poytaxt27 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[11,'Unnamed: 18'])
column_poytaxt28 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[14,'Unnamed: 18'])
column_poytaxt29 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[9,'Unnamed: 18'])
column_poytaxt30 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[11,'Unnamed: 18'])
column_poytaxt31 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[14,'Unnamed: 18'])
column_poytaxt32 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[18,'Unnamed: 19'])
column_poytaxt33 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[18,'Unnamed: 20'])
column_poytaxt35 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[18,'Unnamed: 14'])
column_poytaxt36 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s18).at[18,'Unnamed: 16'])
column_poytaxt34 = column_poytaxt25 - column_poytaxt35 - column_poytaxt36
column_poytaxt40 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[13,'Unnamed: 2'])
column_poytaxt41 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[13,'Unnamed: 3'])
column_poytaxt42 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[13,'3-жадвал'])
column_poytaxt43 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[5,'Unnamed: 2'])
column_poytaxt44 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[10,'Unnamed: 2'])
column_poytaxt45 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[12,'Unnamed: 2'])
column_poytaxt46 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[43,'Unnamed: 2'])
column_poytaxt47 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[43,'Unnamed: 3'])
column_poytaxt48 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[43,'3-жадвал'])
column_poytaxt37 = column_poytaxt40 + column_poytaxt46
column_poytaxt38 = column_poytaxt41 + column_poytaxt47
column_poytaxt39 = column_poytaxt42 + column_poytaxt48
column_poytaxt49 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[35,'Unnamed: 2'])
column_poytaxt50 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[39,'Unnamed: 2'])
column_poytaxt51 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[41,'Unnamed: 2'])
column_poytaxt52 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[42,'Unnamed: 2'])
column_poytaxt56 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[25,'Unnamed: 2'])
column_poytaxt57 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[25,'Unnamed: 3'])
column_poytaxt58 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[25,'3-жадвал'])
column_poytaxt59 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[19,'Unnamed: 2'])
column_poytaxt60 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[19,'Unnamed: 2'])
column_poytaxt61 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[24,'Unnamed: 2'])
column_poytaxt62 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[51,'Unnamed: 2'])
column_poytaxt63 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[51,'Unnamed: 3'])
column_poytaxt64 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[51,'3-жадвал'])
column_poytaxt65 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[67,'Unnamed: 2'])
column_poytaxt66 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[56,'Unnamed: 2'])
column_poytaxt67 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[76,'Unnamed: 2'])
column_poytaxt53 = column_poytaxt37 - column_poytaxt67
column_poytaxt54 = column_poytaxt57 + column_poytaxt63 + column_poytaxt65
column_poytaxt55 = column_poytaxt58 + column_poytaxt64
column_poytaxt68 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[69,'Unnamed: 2'])
column_poytaxt70 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[28,'Unnamed: 2'])
column_poytaxt71 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[31,'Unnamed: 2'])
column_poytaxt69 = column_poytaxt70 + column_poytaxt71
column_poytaxt72 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s22).at[5,'22-жадвал'])
column_poytaxt73 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s22).at[7,'22-жадвал'])
column_poytaxt74 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s22).at[11,'22-жадвал'])
column_poytaxt75 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s22).at[17,'22-жадвал'])
column_poytaxt76 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s22).at[20,'22-жадвал'])
column_poytaxt77 = 0
if s23 is not None:
    column_poytaxt77 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s23).at[3,'23-жадвал'])
column_poytaxt78 = 0
if s23 is not None:
    column_poytaxt78 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s23).at[6,'23-жадвал'])
column_poytaxt79 = 0
if s23 is not None:
    column_poytaxt79 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s23).at[9,'23-жадвал'])
column_poytaxt83 = 0
if s23 is not None:
    column_poytaxt83 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s23).at[10,'23-жадвал'])
column_poytaxt84 = 0
if s23 is not None:
    column_poytaxt84 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s23).at[12,'23-жадвал'])
column_poytaxt85 = column_poytaxt83 - column_poytaxt84
column_poytaxt89 = column_poytaxt9 + column_poytaxt6/2 + column_poytaxt7 + column_poytaxt8
column_poytaxt90 = column_poytaxt25 + column_poytaxt19 + column_poytaxt20/2 + column_poytaxt21 + column_poytaxt22 + column_poytaxt24
column_poytaxt91 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s1).at[37,'Unnamed: 2'])
column_poytaxt92 = float(pd.read_excel(excel_file_poytaxt,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы SQB)
excel_file_sqb = os.path.join(folder, 'uzpsb.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_sqb, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_sqb, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_sqb, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_sqb, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_sqb, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_sqb, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_sqb1 = period
column_sqb2 = "SQB"
column_sqb3 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[45,'Unnamed: 2'])
column_sqb4 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[45,'Unnamed: 3'])
column_sqb5 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[45,'1-жадвал'])
column_sqb6 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[6,'Unnamed: 2'])
column_sqb7 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[9,'Unnamed: 2'])
column_sqb8 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[16,'Unnamed: 2'])
column_sqb9 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[23,'Unnamed: 2'])
column_sqb10 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[23,'Unnamed: 3'])
column_sqb11 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[23,'1-жадвал'])
column_sqb12 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[24,'Unnamed: 2'])
column_sqb14 = float(pd.read_excel(excel_file_sqb,sheet_name=s17).at[28,'Unnamed: 14'])
column_sqb15 = float(pd.read_excel(excel_file_sqb,sheet_name=s17).at[28,'Unnamed: 16'])
column_sqb13 = column_sqb9 - column_sqb14 - column_sqb15
column_sqb16 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[61,'Unnamed: 2'])
column_sqb17 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[61,'Unnamed: 3'])
column_sqb18 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[61,'1-жадвал'])
column_sqb19 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[50,'Unnamed: 2'])
column_sqb20 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[51,'Unnamed: 2'])
column_sqb21 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[53,'Unnamed: 2'])
column_sqb22 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[54,'Unnamed: 2'])
column_sqb23 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[55,'Unnamed: 2'])
column_sqb24 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[75,'Unnamed: 2'])
column_sqb25 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[18,'Unnamed: 18'])
column_sqb26 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[5,'Unnamed: 18'])
column_sqb27 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[11,'Unnamed: 18'])
column_sqb28 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[14,'Unnamed: 18'])
column_sqb29 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[9,'Unnamed: 18'])
column_sqb30 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[11,'Unnamed: 18'])
column_sqb31 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[14,'Unnamed: 18'])
column_sqb32 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[18,'Unnamed: 19'])
column_sqb33 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[18,'Unnamed: 20'])
column_sqb35 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[18,'Unnamed: 14'])
column_sqb36 = float(pd.read_excel(excel_file_sqb,sheet_name=s18).at[18,'Unnamed: 16'])
column_sqb34 = column_sqb25 - column_sqb35 - column_sqb36
column_sqb40 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[13,'Unnamed: 2'])
column_sqb41 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[13,'Unnamed: 3'])
column_sqb42 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[13,'3-жадвал'])
column_sqb43 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[5,'Unnamed: 2'])
column_sqb44 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[10,'Unnamed: 2'])
column_sqb45 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[12,'Unnamed: 2'])
column_sqb46 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[43,'Unnamed: 2'])
column_sqb47 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[43,'Unnamed: 3'])
column_sqb48 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[43,'3-жадвал'])
column_sqb37 = column_sqb40 + column_sqb46
column_sqb38 = column_sqb41 + column_sqb47
column_sqb39 = column_sqb42 + column_sqb48
column_sqb49 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[35,'Unnamed: 2'])
column_sqb50 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[39,'Unnamed: 2'])
column_sqb51 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[41,'Unnamed: 2'])
column_sqb52 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[42,'Unnamed: 2'])
column_sqb56 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[25,'Unnamed: 2'])
column_sqb57 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[25,'Unnamed: 3'])
column_sqb58 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[25,'3-жадвал'])
column_sqb59 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[19,'Unnamed: 2'])
column_sqb60 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[19,'Unnamed: 2'])
column_sqb61 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[24,'Unnamed: 2'])
column_sqb62 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[51,'Unnamed: 2'])
column_sqb63 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[51,'Unnamed: 3'])
column_sqb64 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[51,'3-жадвал'])
column_sqb65 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[67,'Unnamed: 2'])
column_sqb66 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[56,'Unnamed: 2'])
column_sqb67 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[76,'Unnamed: 2'])
column_sqb53 = column_sqb37 - column_sqb67
column_sqb54 = column_sqb57 + column_sqb63 + column_sqb65
column_sqb55 = column_sqb58 + column_sqb64
column_sqb68 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[69,'Unnamed: 2'])
column_sqb70 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[28,'Unnamed: 2'])
column_sqb71 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[31,'Unnamed: 2'])
column_sqb69 = column_sqb70 + column_sqb71
column_sqb72 = float(pd.read_excel(excel_file_sqb,sheet_name=s22).at[5,'22-жадвал'])
column_sqb73 = float(pd.read_excel(excel_file_sqb,sheet_name=s22).at[7,'22-жадвал'])
column_sqb74 = float(pd.read_excel(excel_file_sqb,sheet_name=s22).at[11,'22-жадвал'])
column_sqb75 = float(pd.read_excel(excel_file_sqb,sheet_name=s22).at[17,'22-жадвал'])
column_sqb76 = float(pd.read_excel(excel_file_sqb,sheet_name=s22).at[20,'22-жадвал'])
column_sqb77 = 0
if s23 is not None:
    column_sqb77 = float(pd.read_excel(excel_file_sqb,sheet_name=s23).at[3,'23-жадвал'])
column_sqb78 = 0
if s23 is not None:
    column_sqb78 = float(pd.read_excel(excel_file_sqb,sheet_name=s23).at[6,'23-жадвал'])
column_sqb79 = 0
if s23 is not None:
    column_sqb79 = float(pd.read_excel(excel_file_sqb,sheet_name=s23).at[9,'23-жадвал'])
column_sqb83 = 0
if s23 is not None:
    column_sqb83 = float(pd.read_excel(excel_file_sqb,sheet_name=s23).at[10,'23-жадвал'])
column_sqb84 = 0
if s23 is not None:
    column_sqb84 = float(pd.read_excel(excel_file_sqb,sheet_name=s23).at[12,'23-жадвал'])
column_sqb85 = column_sqb83 - column_sqb84
column_sqb89 = column_sqb9 + column_sqb6/2 + column_sqb7 + column_sqb8
column_sqb90 = column_sqb25 + column_sqb19 + column_sqb20/2 + column_sqb21 + column_sqb22 + column_sqb24
column_sqb91 = float(pd.read_excel(excel_file_sqb,sheet_name=s1).at[37,'Unnamed: 2'])
column_sqb92 = float(pd.read_excel(excel_file_sqb,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы IPOTEKA)
excel_file_ipoteka = os.path.join(folder, 'ipotekabank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_ipoteka, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_ipoteka, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_ipoteka, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_ipoteka, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_ipoteka, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_ipoteka, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_ipoteka1 = period
column_ipoteka2 = "IPOTEKA"
column_ipoteka3 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[45,'Unnamed: 2'])
column_ipoteka4 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[45,'Unnamed: 3'])
column_ipoteka5 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[45,'1-жадвал'])
column_ipoteka6 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[6,'Unnamed: 2'])
column_ipoteka7 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[9,'Unnamed: 2'])
column_ipoteka8 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[16,'Unnamed: 2'])
column_ipoteka9 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[23,'Unnamed: 2'])
column_ipoteka10 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[23,'Unnamed: 3'])
column_ipoteka11 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[23,'1-жадвал'])
column_ipoteka12 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[24,'Unnamed: 2'])
column_ipoteka14 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s17).at[28,'Unnamed: 14'])
column_ipoteka15 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s17).at[28,'Unnamed: 16'])
column_ipoteka13 = column_ipoteka9 - column_ipoteka14 - column_ipoteka15
column_ipoteka16 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[61,'Unnamed: 2'])
column_ipoteka17 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[61,'Unnamed: 3'])
column_ipoteka18 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[61,'1-жадвал'])
column_ipoteka19 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[50,'Unnamed: 2'])
column_ipoteka20 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[51,'Unnamed: 2'])
column_ipoteka21 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[53,'Unnamed: 2'])
column_ipoteka22 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[54,'Unnamed: 2'])
column_ipoteka23 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[55,'Unnamed: 2'])
column_ipoteka24 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[75,'Unnamed: 2'])
column_ipoteka25 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[18,'Unnamed: 18'])
column_ipoteka26 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[5,'Unnamed: 18'])
column_ipoteka27 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[11,'Unnamed: 18'])
column_ipoteka28 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[14,'Unnamed: 18'])
column_ipoteka29 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[9,'Unnamed: 18'])
column_ipoteka30 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[11,'Unnamed: 18'])
column_ipoteka31 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[14,'Unnamed: 18'])
column_ipoteka32 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[18,'Unnamed: 19'])
column_ipoteka33 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[18,'Unnamed: 20'])
column_ipoteka35 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[18,'Unnamed: 14'])
column_ipoteka36 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s18).at[18,'Unnamed: 16'])
column_ipoteka34 = column_ipoteka25 - column_ipoteka35 - column_ipoteka36
column_ipoteka40 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[13,'Unnamed: 2'])
column_ipoteka41 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[13,'Unnamed: 3'])
column_ipoteka42 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[13,'3-жадвал'])
column_ipoteka43 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[5,'Unnamed: 2'])
column_ipoteka44 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[10,'Unnamed: 2'])
column_ipoteka45 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[12,'Unnamed: 2'])
column_ipoteka46 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[43,'Unnamed: 2'])
column_ipoteka47 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[43,'Unnamed: 3'])
column_ipoteka48 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[43,'3-жадвал'])
column_ipoteka37 = column_ipoteka40 + column_ipoteka46
column_ipoteka38 = column_ipoteka41 + column_ipoteka47
column_ipoteka39 = column_ipoteka42 + column_ipoteka48
column_ipoteka49 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[35,'Unnamed: 2'])
column_ipoteka50 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[39,'Unnamed: 2'])
column_ipoteka51 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[41,'Unnamed: 2'])
column_ipoteka52 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[42,'Unnamed: 2'])
column_ipoteka56 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[25,'Unnamed: 2'])
column_ipoteka57 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[25,'Unnamed: 3'])
column_ipoteka58 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[25,'3-жадвал'])
column_ipoteka59 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[19,'Unnamed: 2'])
column_ipoteka60 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[19,'Unnamed: 2'])
column_ipoteka61 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[24,'Unnamed: 2'])
column_ipoteka62 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[51,'Unnamed: 2'])
column_ipoteka63 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[51,'Unnamed: 3'])
column_ipoteka64 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[51,'3-жадвал'])
column_ipoteka65 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[67,'Unnamed: 2'])
column_ipoteka66 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[56,'Unnamed: 2'])
column_ipoteka67 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[76,'Unnamed: 2'])
column_ipoteka53 = column_ipoteka37 - column_ipoteka67
column_ipoteka54 = column_ipoteka57 + column_ipoteka63 + column_ipoteka65
column_ipoteka55 = column_ipoteka58 + column_ipoteka64
column_ipoteka68 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[69,'Unnamed: 2'])
column_ipoteka70 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[28,'Unnamed: 2'])
column_ipoteka71 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[31,'Unnamed: 2'])
column_ipoteka69 = column_ipoteka70 + column_ipoteka71
column_ipoteka72 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s22).at[5,'22-жадвал'])
column_ipoteka73 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s22).at[7,'22-жадвал'])
column_ipoteka74 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s22).at[11,'22-жадвал'])
column_ipoteka75 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s22).at[17,'22-жадвал'])
column_ipoteka76 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s22).at[20,'22-жадвал'])
column_ipoteka77 = 0
if s23 is not None:
    column_ipoteka77 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s23).at[3,'23-жадвал'])
column_ipoteka78 = 0
if s23 is not None:
    column_ipoteka78 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s23).at[6,'23-жадвал'])
column_ipoteka79 = 0
if s23 is not None:
    column_ipoteka79 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s23).at[9,'23-жадвал'])
column_ipoteka83 = 0
if s23 is not None:
    column_ipoteka83 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s23).at[10,'23-жадвал'])
column_ipoteka84 = 0
if s23 is not None:
    column_ipoteka84 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s23).at[12,'23-жадвал'])
column_ipoteka85 = column_ipoteka83 - column_ipoteka84
column_ipoteka89 = column_ipoteka9 + column_ipoteka6/2 + column_ipoteka7 + column_ipoteka8
column_ipoteka90 = column_ipoteka25 + column_ipoteka19 + column_ipoteka20/2 + column_ipoteka21 + column_ipoteka22 + column_ipoteka24
column_ipoteka91 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s1).at[37,'Unnamed: 2'])
column_ipoteka92 = float(pd.read_excel(excel_file_ipoteka,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы Agrobank)
excel_file_agro = os.path.join(folder, 'agrobank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_agro, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_agro, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_agro, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_agro, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_agro, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_agro, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_agro1 = period
column_agro2 = "AGROBANK"
column_agro3 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[45,'Unnamed: 2'])
column_agro4 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[45,'Unnamed: 3'])
column_agro5 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[45,'1-жадвал'])
column_agro6 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[6,'Unnamed: 2'])
column_agro7 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[9,'Unnamed: 2'])
column_agro8 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[16,'Unnamed: 2'])
column_agro9 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[23,'Unnamed: 2'])
column_agro10 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[23,'Unnamed: 3'])
column_agro11 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[23,'1-жадвал'])
column_agro12 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[24,'Unnamed: 2'])
column_agro14 = float(pd.read_excel(excel_file_agro,sheet_name=s17).at[28,'Unnamed: 14'])
column_agro15 = float(pd.read_excel(excel_file_agro,sheet_name=s17).at[28,'Unnamed: 16'])
column_agro13 = column_agro9 - column_agro14 - column_agro15
column_agro16 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[61,'Unnamed: 2'])
column_agro17 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[61,'Unnamed: 3'])
column_agro18 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[61,'1-жадвал'])
column_agro19 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[50,'Unnamed: 2'])
column_agro20 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[51,'Unnamed: 2'])
column_agro21 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[53,'Unnamed: 2'])
column_agro22 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[54,'Unnamed: 2'])
column_agro23 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[55,'Unnamed: 2'])
column_agro24 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[75,'Unnamed: 2'])
column_agro25 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[18,'Unnamed: 18'])
column_agro26 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[5,'Unnamed: 18'])
column_agro27 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[11,'Unnamed: 18'])
column_agro28 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[14,'Unnamed: 18'])
column_agro29 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[9,'Unnamed: 18'])
column_agro30 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_agro,sheet_name=s18).at[11,'Unnamed: 18'])
column_agro31 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_agro,sheet_name=s18).at[14,'Unnamed: 18'])
column_agro32 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[18,'Unnamed: 19'])
column_agro33 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[18,'Unnamed: 20'])
column_agro35 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[18,'Unnamed: 14'])
column_agro36 = float(pd.read_excel(excel_file_agro,sheet_name=s18).at[18,'Unnamed: 16'])
column_agro34 = column_agro25 - column_agro35 - column_agro36
column_agro40 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[13,'Unnamed: 2'])
column_agro41 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[13,'Unnamed: 3'])
column_agro42 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[13,'3-жадвал'])
column_agro43 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[5,'Unnamed: 2'])
column_agro44 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[10,'Unnamed: 2'])
column_agro45 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[12,'Unnamed: 2'])
column_agro46 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[43,'Unnamed: 2'])
column_agro47 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[43,'Unnamed: 3'])
column_agro48 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[43,'3-жадвал'])
column_agro37 = column_agro40 + column_agro46
column_agro38 = column_agro41 + column_agro47
column_agro39 = column_agro42 + column_agro48
column_agro49 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[35,'Unnamed: 2'])
column_agro50 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[39,'Unnamed: 2'])
column_agro51 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[41,'Unnamed: 2'])
column_agro52 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[42,'Unnamed: 2'])
column_agro56 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[25,'Unnamed: 2'])
column_agro57 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[25,'Unnamed: 3'])
column_agro58 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[25,'3-жадвал'])
column_agro59 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[19,'Unnamed: 2'])
column_agro60 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[19,'Unnamed: 2'])
column_agro61 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[24,'Unnamed: 2'])
column_agro62 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[51,'Unnamed: 2'])
column_agro63 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[51,'Unnamed: 3'])
column_agro64 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[51,'3-жадвал'])
column_agro65 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[67,'Unnamed: 2'])
column_agro66 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[56,'Unnamed: 2'])
column_agro67 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[76,'Unnamed: 2'])
column_agro53 = column_agro37 - column_agro67
column_agro54 = column_agro57 + column_agro63 + column_agro65
column_agro55 = column_agro58 + column_agro64
column_agro68 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[69,'Unnamed: 2'])
column_agro70 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[28,'Unnamed: 2'])
column_agro71 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_agro,sheet_name=s3).at[31,'Unnamed: 2'])
column_agro69 = column_agro70 + column_agro71
column_agro72 = float(pd.read_excel(excel_file_agro,sheet_name=s22).at[5,'22-жадвал'])
column_agro73 = float(pd.read_excel(excel_file_agro,sheet_name=s22).at[7,'22-жадвал'])
column_agro74 = float(pd.read_excel(excel_file_agro,sheet_name=s22).at[11,'22-жадвал'])
column_agro75 = float(pd.read_excel(excel_file_agro,sheet_name=s22).at[17,'22-жадвал'])
column_agro76 = float(pd.read_excel(excel_file_agro,sheet_name=s22).at[20,'22-жадвал'])
column_agro77 = 0
if s23 is not None:
    column_agro77 = float(pd.read_excel(excel_file_agro,sheet_name=s23).at[3,'23-жадвал'])
column_agro78 = 0
if s23 is not None:
    column_agro78 = float(pd.read_excel(excel_file_agro,sheet_name=s23).at[6,'23-жадвал'])
column_agro79 = 0
if s23 is not None:
    column_agro79 = float(pd.read_excel(excel_file_agro,sheet_name=s23).at[9,'23-жадвал'])
column_agro83 = 0
if s23 is not None:
    column_agro83 = float(pd.read_excel(excel_file_agro,sheet_name=s23).at[10,'23-жадвал'])
column_agro84 = 0
if s23 is not None:
    column_agro84 = float(pd.read_excel(excel_file_agro,sheet_name=s23).at[12,'23-жадвал'])
column_agro85 = column_agro83 - column_agro84
column_agro89 = column_agro9 + column_agro6/2 + column_agro7 + column_agro8
column_agro90 = column_agro25 + column_agro19 + column_agro20/2 + column_agro21 + column_agro22 + column_agro24
column_agro91 = float(pd.read_excel(excel_file_agro,sheet_name=s1).at[37,'Unnamed: 2'])
column_agro92 = float(pd.read_excel(excel_file_agro,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы QQB)
excel_file_qqb = os.path.join(folder, 'qqb.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_qqb, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_qqb, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_qqb, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_qqb, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_qqb, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_qqb, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_qqb1 = period
column_qqb2 = "BIZNES RIVOJLANTIRISH BANK"
column_qqb3 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[45,'Unnamed: 2'])
column_qqb4 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[45,'Unnamed: 3'])
column_qqb5 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[45,'1-жадвал'])
column_qqb6 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[6,'Unnamed: 2'])
column_qqb7 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[9,'Unnamed: 2'])
column_qqb8 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[16,'Unnamed: 2'])
column_qqb9 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[23,'Unnamed: 2'])
column_qqb10 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[23,'Unnamed: 3'])
column_qqb11 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[23,'1-жадвал'])
column_qqb12 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[24,'Unnamed: 2'])
column_qqb14 = float(pd.read_excel(excel_file_qqb,sheet_name=s17).at[28,'Unnamed: 14'])
column_qqb15 = float(pd.read_excel(excel_file_qqb,sheet_name=s17).at[28,'Unnamed: 16'])
column_qqb13 = column_qqb9 - column_qqb14 - column_qqb15
column_qqb16 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[61,'Unnamed: 2'])
column_qqb17 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[61,'Unnamed: 3'])
column_qqb18 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[61,'1-жадвал'])
column_qqb19 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[50,'Unnamed: 2'])
column_qqb20 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[51,'Unnamed: 2'])
column_qqb21 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[53,'Unnamed: 2'])
column_qqb22 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[54,'Unnamed: 2'])
column_qqb23 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[55,'Unnamed: 2'])
column_qqb24 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[75,'Unnamed: 2'])
column_qqb25 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[18,'Unnamed: 18'])
column_qqb26 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[5,'Unnamed: 18'])
column_qqb27 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[11,'Unnamed: 18'])
column_qqb28 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[14,'Unnamed: 18'])
column_qqb29 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[9,'Unnamed: 18'])
column_qqb30 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[11,'Unnamed: 18'])
column_qqb31 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[14,'Unnamed: 18'])
column_qqb32 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[18,'Unnamed: 19'])
column_qqb33 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[18,'Unnamed: 20'])
column_qqb35 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[18,'Unnamed: 14'])
column_qqb36 = float(pd.read_excel(excel_file_qqb,sheet_name=s18).at[18,'Unnamed: 16'])
column_qqb34 = column_qqb25 - column_qqb35 - column_qqb36
column_qqb40 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[13,'Unnamed: 2'])
column_qqb41 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[13,'Unnamed: 3'])
column_qqb42 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[13,'3-жадвал'])
column_qqb43 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[5,'Unnamed: 2'])
column_qqb44 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[10,'Unnamed: 2'])
column_qqb45 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[12,'Unnamed: 2'])
column_qqb46 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[43,'Unnamed: 2'])
column_qqb47 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[43,'Unnamed: 3'])
column_qqb48 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[43,'3-жадвал'])
column_qqb37 = column_qqb40 + column_qqb46
column_qqb38 = column_qqb41 + column_qqb47
column_qqb39 = column_qqb42 + column_qqb48
column_qqb49 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[35,'Unnamed: 2'])
column_qqb50 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[39,'Unnamed: 2'])
column_qqb51 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[41,'Unnamed: 2'])
column_qqb52 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[42,'Unnamed: 2'])
column_qqb56 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[25,'Unnamed: 2'])
column_qqb57 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[25,'Unnamed: 3'])
column_qqb58 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[25,'3-жадвал'])
column_qqb59 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[19,'Unnamed: 2'])
column_qqb60 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[19,'Unnamed: 2'])
column_qqb61 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[24,'Unnamed: 2'])
column_qqb62 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[51,'Unnamed: 2'])
column_qqb63 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[51,'Unnamed: 3'])
column_qqb64 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[51,'3-жадвал'])
column_qqb65 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[67,'Unnamed: 2'])
column_qqb66 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[56,'Unnamed: 2'])
column_qqb67 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[76,'Unnamed: 2'])
column_qqb53 = column_qqb37 - column_qqb67
column_qqb54 = column_qqb57 + column_qqb63 + column_qqb65
column_qqb55 = column_qqb58 + column_qqb64
column_qqb68 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[69,'Unnamed: 2'])
column_qqb70 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[28,'Unnamed: 2'])
column_qqb71 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[31,'Unnamed: 2'])
column_qqb69 = column_qqb70 + column_qqb71
column_qqb72 = float(pd.read_excel(excel_file_qqb,sheet_name=s22).at[5,'22-жадвал'])
column_qqb73 = float(pd.read_excel(excel_file_qqb,sheet_name=s22).at[7,'22-жадвал'])
column_qqb74 = float(pd.read_excel(excel_file_qqb,sheet_name=s22).at[11,'22-жадвал'])
column_qqb75 = float(pd.read_excel(excel_file_qqb,sheet_name=s22).at[17,'22-жадвал'])
column_qqb76 = float(pd.read_excel(excel_file_qqb,sheet_name=s22).at[20,'22-жадвал'])
column_qqb77 = 0
if s23 is not None:
    column_qqb77 = float(pd.read_excel(excel_file_qqb,sheet_name=s23).at[3,'23-жадвал'])
column_qqb78 = 0
if s23 is not None:
    column_qqb78 = float(pd.read_excel(excel_file_qqb,sheet_name=s23).at[6,'23-жадвал'])
column_qqb79 = 0
if s23 is not None:
    column_qqb79 = float(pd.read_excel(excel_file_qqb,sheet_name=s23).at[9,'23-жадвал'])
column_qqb83 = 0
if s23 is not None:
    column_qqb83 = float(pd.read_excel(excel_file_qqb,sheet_name=s23).at[10,'23-жадвал'])
column_qqb84 = 0
if s23 is not None:
    column_qqb84 = float(pd.read_excel(excel_file_qqb,sheet_name=s23).at[12,'23-жадвал'])
column_qqb85 = column_qqb83 - column_qqb84
column_qqb89 = column_qqb9 + column_qqb6/2 + column_qqb7 + column_qqb8
column_qqb90 = column_qqb25 + column_qqb19 + column_qqb20/2 + column_qqb21 + column_qqb22 + column_qqb24
column_qqb91 = float(pd.read_excel(excel_file_qqb,sheet_name=s1).at[37,'Unnamed: 2'])
column_qqb92 = float(pd.read_excel(excel_file_qqb,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы Turon)
excel_file_turon = os.path.join(folder, 'turonbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_turon, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_turon, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_turon, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str (s17)
    s18 = find_sheet_by_keyword(excel_file_turon, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_turon, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_turon, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_turon1 = period
column_turon2 = "TURON"
column_turon3 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[45,'Unnamed: 2'])
column_turon4 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[45,'Unnamed: 3'])
column_turon5 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[45,'1-жадвал'])
column_turon6 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[6,'Unnamed: 2'])
column_turon7 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[9,'Unnamed: 2'])
column_turon8 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[16,'Unnamed: 2'])
column_turon9 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[23,'Unnamed: 2'])
column_turon10 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[23,'Unnamed: 3'])
column_turon11 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[23,'1-жадвал'])
column_turon12 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[24,'Unnamed: 2'])
column_turon14 = float(pd.read_excel(excel_file_turon,sheet_name=s17).at[28,'Unnamed: 14'])
column_turon15 = float(pd.read_excel(excel_file_turon,sheet_name=s17).at[28,'Unnamed: 16'])
column_turon13 = column_turon9 - column_turon14 - column_turon15
column_turon16 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[61,'Unnamed: 2'])
column_turon17 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[61,'Unnamed: 3'])
column_turon18 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[61,'1-жадвал'])
column_turon19 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[50,'Unnamed: 2'])
column_turon20 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[51,'Unnamed: 2'])
column_turon21 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[53,'Unnamed: 2'])
column_turon22 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[54,'Unnamed: 2'])
column_turon23 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[55,'Unnamed: 2'])
column_turon24 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[75,'Unnamed: 2'])
column_turon25 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[18,'Unnamed: 18'])
column_turon26 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[5,'Unnamed: 18'])
column_turon27 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[11,'Unnamed: 18'])
column_turon28 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[14,'Unnamed: 18'])
column_turon29 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[9,'Unnamed: 18'])
column_turon30 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_turon,sheet_name=s18).at[11,'Unnamed: 18'])
column_turon31 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_turon,sheet_name=s18).at[14,'Unnamed: 18'])
column_turon32 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[18,'Unnamed: 19'])
column_turon33 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[18,'Unnamed: 20'])
column_turon35 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[18,'Unnamed: 14'])
column_turon36 = float(pd.read_excel(excel_file_turon,sheet_name=s18).at[18,'Unnamed: 16'])
column_turon34 = column_turon25 - column_turon35 - column_turon36
column_turon40 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[13,'Unnamed: 2'])
column_turon41 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[13,'Unnamed: 3'])
column_turon42 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[13,'3-жадвал'])
column_turon43 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[5,'Unnamed: 2'])
column_turon44 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[10,'Unnamed: 2'])
column_turon45 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[12,'Unnamed: 2'])
column_turon46 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[43,'Unnamed: 2'])
column_turon47 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[43,'Unnamed: 3'])
column_turon48 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[43,'3-жадвал'])
column_turon37 = column_turon40 + column_turon46
column_turon38 = column_turon41 + column_turon47
column_turon39 = column_turon42 + column_turon48
column_turon49 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[35,'Unnamed: 2'])
column_turon50 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[39,'Unnamed: 2'])
column_turon51 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[41,'Unnamed: 2'])
column_turon52 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[42,'Unnamed: 2'])
column_turon56 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[25,'Unnamed: 2'])
column_turon57 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[25,'Unnamed: 3'])
column_turon58 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[25,'3-жадвал'])
column_turon59 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[19,'Unnamed: 2'])
column_turon60 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[19,'Unnamed: 2'])
column_turon61 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[24,'Unnamed: 2'])
column_turon62 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[51,'Unnamed: 2'])
column_turon63 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[51,'Unnamed: 3'])
column_turon64 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[51,'3-жадвал'])
column_turon65 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[67,'Unnamed: 2'])
column_turon66 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[56,'Unnamed: 2'])
column_turon67 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[76,'Unnamed: 2'])
column_turon53 = column_turon37 - column_turon67
column_turon54 = column_turon57 + column_turon63 + column_turon65
column_turon55 = column_turon58 + column_turon64
column_turon68 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[69,'Unnamed: 2'])
column_turon70 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[28,'Unnamed: 2'])
column_turon71 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_turon,sheet_name=s3).at[31,'Unnamed: 2'])
column_turon69 = column_turon70 + column_turon71
column_turon72 = float(pd.read_excel(excel_file_turon,sheet_name=s22).at[5,'22-жадвал'])
column_turon73 = float(pd.read_excel(excel_file_turon,sheet_name=s22).at[7,'22-жадвал'])
column_turon74 = float(pd.read_excel(excel_file_turon,sheet_name=s22).at[11,'22-жадвал'])
column_turon75 = float(pd.read_excel(excel_file_turon,sheet_name=s22).at[17,'22-жадвал'])
column_turon76 = float(pd.read_excel(excel_file_turon,sheet_name=s22).at[20,'22-жадвал'])
column_turon77 = 0
if s23 is not None:
    try:
        column_turon77 = float(pd.read_excel(excel_file_turon,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_turon78 = 0
if s23 is not None:
    try:
        column_turon78 = float(pd.read_excel(excel_file_turon,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_turon79 = 0
if s23 is not None:
    try:
        column_turon79 = float(pd.read_excel(excel_file_turon,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_turon83 = 0
if s23 is not None:
    try:
        column_turon83 = float(pd.read_excel(excel_file_turon,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_turon84 = 0
if s23 is not None:
    try:
        column_turon84 = float(pd.read_excel(excel_file_turon,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_turon85 = column_turon83 - column_turon84
column_turon89 = column_turon9 + column_turon6/2 + column_turon7 + column_turon8
column_turon90 = column_turon25 + column_turon19 + column_turon20/2 + column_turon21 + column_turon22 + column_turon24
column_turon91 = float(pd.read_excel(excel_file_turon,sheet_name=s1).at[37,'Unnamed: 2'])
column_turon92 = float(pd.read_excel(excel_file_turon,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы MICROCREDIT)
excel_file_micro = os.path.join(folder, 'mikrokreditbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_micro, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_micro, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_micro, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_micro, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_micro, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_micro, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_micro1 = period
column_micro2 = "MIKROCREDIT"
column_micro3 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[45,'Unnamed: 2'])
column_micro4 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[45,'Unnamed: 3'])
column_micro5 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[45,'1-жадвал'])
column_micro6 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[6,'Unnamed: 2'])
column_micro7 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[9,'Unnamed: 2'])
column_micro8 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[16,'Unnamed: 2'])
column_micro9 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[23,'Unnamed: 2'])
column_micro10 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[23,'Unnamed: 3'])
column_micro11 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[23,'1-жадвал'])
column_micro12 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[24,'Unnamed: 2'])
column_micro14 = float(pd.read_excel(excel_file_micro,sheet_name=s17).at[28,'Unnamed: 14'])
column_micro15 = float(pd.read_excel(excel_file_micro,sheet_name=s17).at[28,'Unnamed: 16'])
column_micro13 = column_micro9 - column_micro14 - column_micro15
column_micro16 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[61,'Unnamed: 2'])
column_micro17 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[61,'Unnamed: 3'])
column_micro18 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[61,'1-жадвал'])
column_micro19 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[50,'Unnamed: 2'])
column_micro20 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[51,'Unnamed: 2'])
column_micro21 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[53,'Unnamed: 2'])
column_micro22 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[54,'Unnamed: 2'])
column_micro23 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[55,'Unnamed: 2'])
column_micro24 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[75,'Unnamed: 2'])
column_micro25 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[18,'Unnamed: 18'])
column_micro26 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[5,'Unnamed: 18'])
column_micro27 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[11,'Unnamed: 18'])
column_micro28 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[14,'Unnamed: 18'])
column_micro29 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[9,'Unnamed: 18'])
column_micro30 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_micro,sheet_name=s18).at[11,'Unnamed: 18'])
column_micro31 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_micro,sheet_name=s18).at[14,'Unnamed: 18'])
column_micro32 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[18,'Unnamed: 19'])
column_micro33 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[18,'Unnamed: 20'])
column_micro35 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[18,'Unnamed: 14'])
column_micro36 = float(pd.read_excel(excel_file_micro,sheet_name=s18).at[18,'Unnamed: 16'])
column_micro34 = column_micro25 - column_micro35 - column_micro36
column_micro40 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[13,'Unnamed: 2'])
column_micro41 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[13,'Unnamed: 3'])
column_micro42 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[13,'3-жадвал'])
column_micro43 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[5,'Unnamed: 2'])
column_micro44 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[10,'Unnamed: 2'])
column_micro45 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[12,'Unnamed: 2'])
column_micro46 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[43,'Unnamed: 2'])
column_micro47 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[43,'Unnamed: 3'])
column_micro48 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[43,'3-жадвал'])
column_micro37 = column_micro40 + column_micro46
column_micro38 = column_micro41 + column_micro47
column_micro39 = column_micro42 + column_micro48
column_micro49 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[35,'Unnamed: 2'])
column_micro50 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[39,'Unnamed: 2'])
column_micro51 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[41,'Unnamed: 2'])
column_micro52 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[42,'Unnamed: 2'])
column_micro56 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[25,'Unnamed: 2'])
column_micro57 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[25,'Unnamed: 3'])
column_micro58 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[25,'3-жадвал'])
column_micro59 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[19,'Unnamed: 2'])
column_micro60 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[19,'Unnamed: 2'])
column_micro61 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[24,'Unnamed: 2'])
column_micro62 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[51,'Unnamed: 2'])
column_micro63 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[51,'Unnamed: 3'])
column_micro64 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[51,'3-жадвал'])
column_micro65 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[67,'Unnamed: 2'])
column_micro66 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[56,'Unnamed: 2'])
column_micro67 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[76,'Unnamed: 2'])
column_micro53 = column_micro37 - column_micro67
column_micro54 = column_micro57 + column_micro63 + column_micro65
column_micro55 = column_micro58 + column_micro64
column_micro68 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[69,'Unnamed: 2'])
column_micro70 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[28,'Unnamed: 2'])
column_micro71 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_micro,sheet_name=s3).at[31,'Unnamed: 2'])
column_micro69 = column_micro70 + column_micro71
column_micro72 = float(pd.read_excel(excel_file_micro,sheet_name=s22).at[5,'22-жадвал'])
column_micro73 = float(pd.read_excel(excel_file_micro,sheet_name=s22).at[7,'22-жадвал'])
column_micro74 = float(pd.read_excel(excel_file_micro,sheet_name=s22).at[11,'22-жадвал'])
column_micro75 = float(pd.read_excel(excel_file_micro,sheet_name=s22).at[17,'22-жадвал'])
column_micro76 = float(pd.read_excel(excel_file_micro,sheet_name=s22).at[20,'22-жадвал'])
column_micro77 = 0
if s23 is not None:
    column_micro77 = float(pd.read_excel(excel_file_micro,sheet_name=s23).at[3,'23-жадвал'])
column_micro78 = 0
if s23 is not None:
    column_micro78 = float(pd.read_excel(excel_file_micro,sheet_name=s23).at[6,'23-жадвал'])
column_micro79 = 0
if s23 is not None:
    column_micro79 = float(pd.read_excel(excel_file_micro,sheet_name=s23).at[9,'23-жадвал'])
column_micro83 = 0
if s23 is not None:
    column_micro83 = float(pd.read_excel(excel_file_micro,sheet_name=s23).at[10,'23-жадвал'])
column_micro84 = 0
if s23 is not None:
    column_micro84 = float(pd.read_excel(excel_file_micro,sheet_name=s23).at[12,'23-жадвал'])
column_micro85 = column_micro83 - column_micro84
column_micro89 = column_micro9 + column_micro6/2 + column_micro7 + column_micro8
column_micro90 = column_micro25 + column_micro19 + column_micro20/2 + column_micro21 + column_micro22 + column_micro24
column_micro91 = float(pd.read_excel(excel_file_micro,sheet_name=s1).at[37,'Unnamed: 2'])
column_micro92 = float(pd.read_excel(excel_file_micro,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы ALOQA)
excel_file_aloqa = os.path.join(folder, 'aloqabank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_aloqa, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_aloqa, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_aloqa, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_aloqa, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_aloqa, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_aloqa, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_aloqa1 = period
column_aloqa2 = "ALOQA"
column_aloqa3 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[45,'Unnamed: 2'])
column_aloqa4 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[45,'Unnamed: 3'])
column_aloqa5 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[45,'1-жадвал'])
column_aloqa6 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[6,'Unnamed: 2'])
column_aloqa7 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[9,'Unnamed: 2'])
column_aloqa8 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[16,'Unnamed: 2'])
column_aloqa9 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[23,'Unnamed: 2'])
column_aloqa10 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[23,'Unnamed: 3'])
column_aloqa11 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[23,'1-жадвал'])
column_aloqa12 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[24,'Unnamed: 2'])
column_aloqa14 = float(pd.read_excel(excel_file_aloqa,sheet_name=s17).at[28,'Unnamed: 14'])
column_aloqa15 = float(pd.read_excel(excel_file_aloqa,sheet_name=s17).at[28,'Unnamed: 16'])
column_aloqa13 = column_aloqa9 - column_aloqa14 - column_aloqa15
column_aloqa16 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[61,'Unnamed: 2'])
column_aloqa17 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[61,'Unnamed: 3'])
column_aloqa18 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[61,'1-жадвал'])
column_aloqa19 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[50,'Unnamed: 2'])
column_aloqa20 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[51,'Unnamed: 2'])
column_aloqa21 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[53,'Unnamed: 2'])
column_aloqa22 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[54,'Unnamed: 2'])
column_aloqa23 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[55,'Unnamed: 2'])
column_aloqa24 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[75,'Unnamed: 2'])
column_aloqa25 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[18,'Unnamed: 18'])
column_aloqa26 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[5,'Unnamed: 18'])
column_aloqa27 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[11,'Unnamed: 18'])
column_aloqa28 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[14,'Unnamed: 18'])
column_aloqa29 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[9,'Unnamed: 18'])
column_aloqa30 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[11,'Unnamed: 18'])
column_aloqa31 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[14,'Unnamed: 18'])
column_aloqa32 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[18,'Unnamed: 19'])
column_aloqa33 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[18,'Unnamed: 20'])
column_aloqa35 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[18,'Unnamed: 14'])
column_aloqa36 = float(pd.read_excel(excel_file_aloqa,sheet_name=s18).at[18,'Unnamed: 16'])
column_aloqa34 = column_aloqa25 - column_aloqa35 - column_aloqa36
column_aloqa40 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[13,'Unnamed: 2'])
column_aloqa41 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[13,'Unnamed: 3'])
column_aloqa42 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[13,'3-жадвал'])
column_aloqa43 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[5,'Unnamed: 2'])
column_aloqa44 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[10,'Unnamed: 2'])
column_aloqa45 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[12,'Unnamed: 2'])
column_aloqa46 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[43,'Unnamed: 2'])
column_aloqa47 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[43,'Unnamed: 3'])
column_aloqa48 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[43,'3-жадвал'])
column_aloqa37 = column_aloqa40 + column_aloqa46
column_aloqa38 = column_aloqa41 + column_aloqa47
column_aloqa39 = column_aloqa42 + column_aloqa48
column_aloqa49 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[35,'Unnamed: 2'])
column_aloqa50 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[39,'Unnamed: 2'])
column_aloqa51 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[41,'Unnamed: 2'])
column_aloqa52 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[42,'Unnamed: 2'])
column_aloqa56 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[25,'Unnamed: 2'])
column_aloqa57 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[25,'Unnamed: 3'])
column_aloqa58 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[25,'3-жадвал'])
column_aloqa59 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[19,'Unnamed: 2'])
column_aloqa60 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[19,'Unnamed: 2'])
column_aloqa61 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[24,'Unnamed: 2'])
column_aloqa62 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[51,'Unnamed: 2'])
column_aloqa63 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[51,'Unnamed: 3'])
column_aloqa64 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[51,'3-жадвал'])
column_aloqa65 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[67,'Unnamed: 2'])
column_aloqa66 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[56,'Unnamed: 2'])
column_aloqa67 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[76,'Unnamed: 2'])
column_aloqa53 = column_aloqa37 - column_aloqa67
column_aloqa54 = column_aloqa57 + column_aloqa63 + column_aloqa65
column_aloqa55 = column_aloqa58 + column_aloqa64
column_aloqa68 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[69,'Unnamed: 2'])
column_aloqa70 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[28,'Unnamed: 2'])
column_aloqa71 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[31,'Unnamed: 2'])
column_aloqa69 = column_aloqa70 + column_aloqa71
column_aloqa72 = float(pd.read_excel(excel_file_aloqa,sheet_name=s22).at[5,'22-жадвал'])
column_aloqa73 = float(pd.read_excel(excel_file_aloqa,sheet_name=s22).at[7,'22-жадвал'])
column_aloqa74 = float(pd.read_excel(excel_file_aloqa,sheet_name=s22).at[11,'22-жадвал'])
column_aloqa75 = float(pd.read_excel(excel_file_aloqa,sheet_name=s22).at[17,'22-жадвал'])
column_aloqa76 = float(pd.read_excel(excel_file_aloqa,sheet_name=s22).at[20,'22-жадвал'])
column_aloqa77 = 0
if s23 is not None:
    column_aloqa77 = float(pd.read_excel(excel_file_aloqa,sheet_name=s23).at[3,'23-жадвал'])
column_aloqa78 = 0
if s23 is not None:
    column_aloqa78 = float(pd.read_excel(excel_file_aloqa,sheet_name=s23).at[6,'23-жадвал'])
column_aloqa79 = 0
if s23 is not None:
    column_aloqa79 = float(pd.read_excel(excel_file_aloqa,sheet_name=s23).at[9,'23-жадвал'])
column_aloqa83 = 0
if s23 is not None:
    column_aloqa83 = float(pd.read_excel(excel_file_aloqa,sheet_name=s23).at[10,'23-жадвал'])
column_aloqa84 = 0
if s23 is not None:
    column_aloqa84 = float(pd.read_excel(excel_file_aloqa,sheet_name=s23).at[12,'23-жадвал'])
column_aloqa85 = column_aloqa83 - column_aloqa84
column_aloqa89 = column_aloqa9 + column_aloqa6/2 + column_aloqa7 + column_aloqa8
column_aloqa90 = column_aloqa25 + column_aloqa19 + column_aloqa20/2 + column_aloqa21 + column_aloqa22 + column_aloqa24
column_aloqa91 = float(pd.read_excel(excel_file_aloqa,sheet_name=s1).at[37,'Unnamed: 2'])
column_aloqa92 = float(pd.read_excel(excel_file_aloqa,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы ALOQA)
excel_file_ipak = os.path.join(folder, 'ipakyulibank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_ipak, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_ipak, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_ipak, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_ipak, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_ipak, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_ipak, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_ipak1 = period
column_ipak2 = "IPAK YULI"
column_ipak3 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[45,'Unnamed: 2'])
column_ipak4 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[45,'Unnamed: 3'])
column_ipak5 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[45,'1-жадвал'])
column_ipak6 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[6,'Unnamed: 2'])
column_ipak7 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[9,'Unnamed: 2'])
column_ipak8 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[16,'Unnamed: 2'])
column_ipak9 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[23,'Unnamed: 2'])
column_ipak10 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[23,'Unnamed: 3'])
column_ipak11 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[23,'1-жадвал'])
column_ipak12 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[24,'Unnamed: 2'])
column_ipak14 = float(pd.read_excel(excel_file_ipak ,sheet_name=s17).at[28,'Unnamed: 14'])
column_ipak15 = float(pd.read_excel(excel_file_ipak ,sheet_name=s17).at[28,'Unnamed: 16'])
column_ipak13 = column_ipak9 - column_ipak14 - column_ipak15
column_ipak16 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[61,'Unnamed: 2'])
column_ipak17 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[61,'Unnamed: 3'])
column_ipak18 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[61,'1-жадвал'])
column_ipak19 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[50,'Unnamed: 2'])
column_ipak20 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[51,'Unnamed: 2'])
column_ipak21 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[53,'Unnamed: 2'])
column_ipak22 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[54,'Unnamed: 2'])
column_ipak23 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[55,'Unnamed: 2'])
column_ipak24 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[75,'Unnamed: 2'])
column_ipak25 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[18,'Unnamed: 18'])
column_ipak26 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[5,'Unnamed: 18'])
column_ipak27 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[11,'Unnamed: 18'])
column_ipak28 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[14,'Unnamed: 18'])
column_ipak29 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[9,'Unnamed: 18'])
column_ipak30 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[11,'Unnamed: 18'])
column_ipak31 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[14,'Unnamed: 18'])
column_ipak32 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[18,'Unnamed: 19'])
column_ipak33 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[18,'Unnamed: 20'])
column_ipak35 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[18,'Unnamed: 14'])
column_ipak36 = float(pd.read_excel(excel_file_ipak ,sheet_name=s18).at[18,'Unnamed: 16'])
column_ipak34 = column_ipak25 - column_ipak35 - column_ipak36
column_ipak40 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[13,'Unnamed: 2'])
column_ipak41 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[13,'Unnamed: 3'])
column_ipak42 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[13,'3-жадвал'])
column_ipak43 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[5,'Unnamed: 2'])
column_ipak44 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[10,'Unnamed: 2'])
column_ipak45 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[12,'Unnamed: 2'])
column_ipak46 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[43,'Unnamed: 2'])
column_ipak47 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[43,'Unnamed: 3'])
column_ipak48 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[43,'3-жадвал'])
column_ipak37 = column_ipak40 + column_ipak46
column_ipak38 = column_ipak41 + column_ipak47
column_ipak39 = column_ipak42 + column_ipak48
column_ipak49 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[35,'Unnamed: 2'])
column_ipak50 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[39,'Unnamed: 2'])
column_ipak51 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[41,'Unnamed: 2'])
column_ipak52 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[42,'Unnamed: 2'])
column_ipak56 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[25,'Unnamed: 2'])
column_ipak57 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[25,'Unnamed: 3'])
column_ipak58 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[25,'3-жадвал'])
column_ipak59 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[19,'Unnamed: 2'])
column_ipak60 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[19,'Unnamed: 2'])
column_ipak61 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[24,'Unnamed: 2'])
column_ipak62 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[51,'Unnamed: 2'])
column_ipak63 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[51,'Unnamed: 3'])
column_ipak64 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[51,'3-жадвал'])
column_ipak65 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[67,'Unnamed: 2'])
column_ipak66 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[56,'Unnamed: 2'])
column_ipak67 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[76,'Unnamed: 2'])
column_ipak53 = column_ipak37 - column_ipak67
column_ipak54 = column_ipak57 + column_ipak63 + column_ipak65
column_ipak55 = column_ipak58 + column_ipak64
column_ipak68 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[69,'Unnamed: 2'])
column_ipak70 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[28,'Unnamed: 2'])
column_ipak71 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[31,'Unnamed: 2'])
column_ipak69 = column_ipak70 + column_ipak71
column_ipak72 = float(pd.read_excel(excel_file_ipak ,sheet_name=s22).at[5,'22-жадвал'])
column_ipak73 = float(pd.read_excel(excel_file_ipak ,sheet_name=s22).at[7,'22-жадвал'])
column_ipak74 = float(pd.read_excel(excel_file_ipak ,sheet_name=s22).at[11,'22-жадвал'])
column_ipak75 = float(pd.read_excel(excel_file_ipak ,sheet_name=s22).at[17,'22-жадвал'])
column_ipak76 = float(pd.read_excel(excel_file_ipak ,sheet_name=s22).at[20,'22-жадвал'])
column_ipak77 = 0
if s23 is not None:
    column_ipak77 = float(pd.read_excel(excel_file_ipak ,sheet_name=s23).at[3,'23-жадвал'])
column_ipak78 = 0
if s23 is not None:
    column_ipak78 = float(pd.read_excel(excel_file_ipak ,sheet_name=s23).at[6,'23-жадвал'])
column_ipak79 = 0
if s23 is not None:
    column_ipak79 = float(pd.read_excel(excel_file_ipak ,sheet_name=s23).at[9,'23-жадвал'])
column_ipak83 = 0
if s23 is not None:
    column_ipak83 = float(pd.read_excel(excel_file_ipak ,sheet_name=s23).at[10,'23-жадвал'])
column_ipak84 = 0
if s23 is not None:
    column_ipak84 = float(pd.read_excel(excel_file_ipak ,sheet_name=s23).at[12,'23-жадвал'])
column_ipak85 = column_ipak83 - column_ipak84
column_ipak89 = column_ipak9 + column_ipak6/2 + column_ipak7 + column_ipak8
column_ipak90 = column_ipak25 + column_ipak19 + column_ipak20/2 + column_ipak21 + column_ipak22 + column_ipak24
column_ipak91 = float(pd.read_excel(excel_file_ipak ,sheet_name=s1).at[37,'Unnamed: 2'])
column_ipak92 = float(pd.read_excel(excel_file_ipak ,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы Kapital)
excel_file_kapital = os.path.join(folder, 'kapitalbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_kapital, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_kapital, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_kapital, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_kapital, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_kapital, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_kapital, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_kapital1 = period
column_kapital2 = "KAPITAL"
column_kapital3 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[45,'Unnamed: 2'])
column_kapital4 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[45,'Unnamed: 3'])
column_kapital5 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[45,'1-жадвал'])
column_kapital6 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[6,'Unnamed: 2'])
column_kapital7 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[9,'Unnamed: 2'])
column_kapital8 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[16,'Unnamed: 2'])
column_kapital9 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[23,'Unnamed: 2'])
column_kapital10 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[23,'Unnamed: 3'])
column_kapital11 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[23,'1-жадвал'])
column_kapital12 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[24,'Unnamed: 2'])
column_kapital14 = float(pd.read_excel(excel_file_kapital ,sheet_name=s17).at[28,'Unnamed: 14'])
column_kapital15 = float(pd.read_excel(excel_file_kapital ,sheet_name=s17).at[28,'Unnamed: 16'])
column_kapital13 = column_kapital9 - column_kapital14 - column_kapital15
column_kapital16 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[61,'Unnamed: 2'])
column_kapital17 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[61,'Unnamed: 3'])
column_kapital18 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[61,'1-жадвал'])
column_kapital19 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[50,'Unnamed: 2'])
column_kapital20 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[51,'Unnamed: 2'])
column_kapital21 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[53,'Unnamed: 2'])
column_kapital22 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[54,'Unnamed: 2'])
column_kapital23 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[55,'Unnamed: 2'])
column_kapital24 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[75,'Unnamed: 2'])
column_kapital25 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[18,'Unnamed: 18'])
column_kapital26 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[5,'Unnamed: 18'])
column_kapital27 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[11,'Unnamed: 18'])
column_kapital28 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[14,'Unnamed: 18'])
column_kapital29 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[9,'Unnamed: 18'])
column_kapital30 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[11,'Unnamed: 18'])
column_kapital31 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[14,'Unnamed: 18'])
column_kapital32 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[18,'Unnamed: 19'])
column_kapital33 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[18,'Unnamed: 20'])
column_kapital35 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[18,'Unnamed: 14'])
column_kapital36 = float(pd.read_excel(excel_file_kapital ,sheet_name=s18).at[18,'Unnamed: 16'])
column_kapital34 = column_kapital25 - column_kapital35 - column_kapital36
column_kapital40 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[13,'Unnamed: 2'])
column_kapital41 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[13,'Unnamed: 3'])
column_kapital42 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[13,'3-жадвал'])
column_kapital43 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[5,'Unnamed: 2'])
column_kapital44 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[10,'Unnamed: 2'])
column_kapital45 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[12,'Unnamed: 2'])
column_kapital46 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[43,'Unnamed: 2'])
column_kapital47 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[43,'Unnamed: 3'])
column_kapital48 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[43,'3-жадвал'])
column_kapital37 = column_kapital40 + column_kapital46
column_kapital38 = column_kapital41 + column_kapital47
column_kapital39 = column_kapital42 + column_kapital48
column_kapital49 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[35,'Unnamed: 2'])
column_kapital50 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[39,'Unnamed: 2'])
column_kapital51 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[41,'Unnamed: 2'])
column_kapital52 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[42,'Unnamed: 2'])
column_kapital56 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[25,'Unnamed: 2'])
column_kapital57 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[25,'Unnamed: 3'])
column_kapital58 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[25,'3-жадвал'])
column_kapital59 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[19,'Unnamed: 2'])
column_kapital60 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[19,'Unnamed: 2'])
column_kapital61 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[24,'Unnamed: 2'])
column_kapital62 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[51,'Unnamed: 2'])
column_kapital63 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[51,'Unnamed: 3'])
column_kapital64 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[51,'3-жадвал'])
column_kapital65 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[67,'Unnamed: 2'])
column_kapital66 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[56,'Unnamed: 2'])
column_kapital67 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[76,'Unnamed: 2'])
column_kapital53 = column_kapital37 - column_kapital67
column_kapital54 = column_kapital57 + column_kapital63 + column_kapital65
column_kapital55 = column_kapital58 + column_kapital64
column_kapital68 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[69,'Unnamed: 2'])
column_kapital70 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[28,'Unnamed: 2'])
column_kapital71 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[31,'Unnamed: 2'])
column_kapital69 = column_kapital70 + column_kapital71
column_kapital72 = float(pd.read_excel(excel_file_kapital ,sheet_name=s22).at[5,'22-жадвал'])
column_kapital73 = float(pd.read_excel(excel_file_kapital ,sheet_name=s22).at[7,'22-жадвал'])
column_kapital74 = float(pd.read_excel(excel_file_kapital ,sheet_name=s22).at[11,'22-жадвал'])
column_kapital75 = float(pd.read_excel(excel_file_kapital ,sheet_name=s22).at[17,'22-жадвал'])
column_kapital76 = float(pd.read_excel(excel_file_kapital ,sheet_name=s22).at[20,'22-жадвал'])
column_kapital77 = 0
if s23 is not None:
    column_kapital77 = float(pd.read_excel(excel_file_kapital ,sheet_name=s23).at[3,'23-жадвал'])
column_kapital78 = 0
if s23 is not None:
    column_kapital78 = float(pd.read_excel(excel_file_kapital ,sheet_name=s23).at[6,'23-жадвал'])
column_kapital79 = 0
if s23 is not None:
    column_kapital79 = float(pd.read_excel(excel_file_kapital ,sheet_name=s23).at[9,'23-жадвал'])
column_kapital83 = 0
if s23 is not None:
    column_kapital83 = float(pd.read_excel(excel_file_kapital ,sheet_name=s23).at[10,'23-жадвал'])
column_kapital84 = 0
if s23 is not None:
    column_kapital84 = float(pd.read_excel(excel_file_kapital ,sheet_name=s23).at[12,'23-жадвал'])
column_kapital85 = column_kapital83 - column_kapital84
column_kapital89 = column_kapital9 + column_kapital6/2 + column_kapital7 + column_kapital8
column_kapital90 = column_kapital25 + column_kapital19 + column_kapital20/2 + column_kapital21 + column_kapital22 + column_kapital24
column_kapital91 = float(pd.read_excel(excel_file_kapital ,sheet_name=s1).at[37,'Unnamed: 2'])
column_kapital92 = float(pd.read_excel(excel_file_kapital ,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы Infin)
excel_file_infin = os.path.join(folder, 'infinbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_infin, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_infin, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_infin, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_infin, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_infin, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_infin, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_infin1 = period
column_infin2 = "INVEST FINANCE"
column_infin3 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[45,'Unnamed: 2'])
column_infin4 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[45,'Unnamed: 3'])
column_infin5 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[45,'1-жадвал'])
column_infin6 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[6,'Unnamed: 2'])
column_infin7 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[9,'Unnamed: 2'])
column_infin8 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[16,'Unnamed: 2'])
column_infin9 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[23,'Unnamed: 2'])
column_infin10 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[23,'Unnamed: 3'])
column_infin11 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[23,'1-жадвал'])
column_infin12 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[24,'Unnamed: 2'])
column_infin14 = float(pd.read_excel(excel_file_infin ,sheet_name=s17).at[28,'Unnamed: 14'])
column_infin15 = float(pd.read_excel(excel_file_infin ,sheet_name=s17).at[28,'Unnamed: 16'])
column_infin13 = column_infin9 - column_infin14 - column_infin15
column_infin16 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[61,'Unnamed: 2'])
column_infin17 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[61,'Unnamed: 3'])
column_infin18 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[61,'1-жадвал'])
column_infin19 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[50,'Unnamed: 2'])
column_infin20 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[51,'Unnamed: 2'])
column_infin21 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[53,'Unnamed: 2'])
column_infin22 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[54,'Unnamed: 2'])
column_infin23 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[55,'Unnamed: 2'])
column_infin24 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[75,'Unnamed: 2'])
column_infin25 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[18,'Unnamed: 18'])
column_infin26 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[5,'Unnamed: 18'])
column_infin27 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[11,'Unnamed: 18'])
column_infin28 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[14,'Unnamed: 18'])
column_infin29 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[9,'Unnamed: 18'])
column_infin30 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[11,'Unnamed: 18'])
column_infin31 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[14,'Unnamed: 18'])
column_infin32 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[18,'Unnamed: 19'])
column_infin33 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[18,'Unnamed: 20'])
column_infin35 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[18,'Unnamed: 14'])
column_infin36 = float(pd.read_excel(excel_file_infin ,sheet_name=s18).at[18,'Unnamed: 16'])
column_infin34 = column_infin25 - column_infin35 - column_infin36
column_infin40 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[13,'Unnamed: 2'])
column_infin41 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[13,'Unnamed: 3'])
column_infin42 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[13,'3-жадвал'])
column_infin43 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[5,'Unnamed: 2'])
column_infin44 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[10,'Unnamed: 2'])
column_infin45 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[12,'Unnamed: 2'])
column_infin46 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[43,'Unnamed: 2'])
column_infin47 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[43,'Unnamed: 3'])
column_infin48 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[43,'3-жадвал'])
column_infin37 = column_infin40 + column_infin46
column_infin38 = column_infin41 + column_infin47
column_infin39 = column_infin42 + column_infin48
column_infin49 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[35,'Unnamed: 2'])
column_infin50 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[39,'Unnamed: 2'])
column_infin51 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[41,'Unnamed: 2'])
column_infin52 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[42,'Unnamed: 2'])
column_infin56 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[25,'Unnamed: 2'])
column_infin57 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[25,'Unnamed: 3'])
column_infin58 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[25,'3-жадвал'])
column_infin59 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[19,'Unnamed: 2'])
column_infin60 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[19,'Unnamed: 2'])
column_infin61 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[24,'Unnamed: 2'])
column_infin62 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[51,'Unnamed: 2'])
column_infin63 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[51,'Unnamed: 3'])
column_infin64 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[51,'3-жадвал'])
column_infin65 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[67,'Unnamed: 2'])
column_infin66 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[56,'Unnamed: 2'])
column_infin67 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[76,'Unnamed: 2'])
column_infin53 = column_infin37 - column_infin67
column_infin54 = column_infin57 + column_infin63 + column_infin65
column_infin55 = column_infin58 + column_infin64
column_infin68 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[69,'Unnamed: 2'])
column_infin70 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[28,'Unnamed: 2'])
column_infin71 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[31,'Unnamed: 2'])
column_infin69 = column_infin70 + column_infin71
column_infin72 = float(pd.read_excel(excel_file_infin ,sheet_name=s22).at[5,'22-жадвал'])
column_infin73 = float(pd.read_excel(excel_file_infin ,sheet_name=s22).at[7,'22-жадвал'])
column_infin74 = float(pd.read_excel(excel_file_infin ,sheet_name=s22).at[11,'22-жадвал'])
column_infin75 = float(pd.read_excel(excel_file_infin ,sheet_name=s22).at[17,'22-жадвал'])
column_infin76 = float(pd.read_excel(excel_file_infin ,sheet_name=s22).at[20,'22-жадвал'])
column_infin77 = 0
if s23 is not None:
    column_infin77 = float(pd.read_excel(excel_file_infin ,sheet_name=s23).at[3,'23-жадвал'])
column_infin78 = 0
if s23 is not None:
    column_infin78 = float(pd.read_excel(excel_file_infin ,sheet_name=s23).at[6,'23-жадвал'])
column_infin79 = 0
if s23 is not None:
    column_infin79 = float(pd.read_excel(excel_file_infin ,sheet_name=s23).at[9,'23-жадвал'])
column_infin83 = 0
if s23 is not None:
    column_infin83 = float(pd.read_excel(excel_file_infin ,sheet_name=s23).at[10,'23-жадвал'])
column_infin84 = 0
if s23 is not None:
    column_infin84 = float(pd.read_excel(excel_file_infin ,sheet_name=s23).at[12,'23-жадвал'])
column_infin85 = column_infin83 - column_infin84
column_infin89 = column_infin9 + column_infin6/2 + column_infin7 + column_infin8
column_infin90 = column_infin25 + column_infin19 + column_infin20/2 + column_infin21 + column_infin22 + column_infin24
column_infin91 = float(pd.read_excel(excel_file_infin ,sheet_name=s1).at[37,'Unnamed: 2'])
column_infin92 = float(pd.read_excel(excel_file_infin ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Universal)
excel_file_universal = os.path.join(folder, 'universalbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_universal, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_universal, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_universal, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_universal, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_universal, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_universal, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_universal1 = period
column_universal2 = "UNIVERSAL"
column_universal3 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[45,'Unnamed: 2'])
column_universal4 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[45,'Unnamed: 3'])
column_universal5 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[45,'1-жадвал'])
column_universal6 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[6,'Unnamed: 2'])
column_universal7 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[9,'Unnamed: 2'])
column_universal8 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[16,'Unnamed: 2'])
column_universal9 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[23,'Unnamed: 2'])
column_universal10 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[23,'Unnamed: 3'])
column_universal11 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[23,'1-жадвал'])
column_universal12 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[24,'Unnamed: 2'])
column_universal14 = float(pd.read_excel(excel_file_universal ,sheet_name=s17).at[28,'Unnamed: 14'])
column_universal15 = float(pd.read_excel(excel_file_universal ,sheet_name=s17).at[28,'Unnamed: 16'])
column_universal13 = column_universal9 - column_universal14 - column_universal15
column_universal16 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[61,'Unnamed: 2'])
column_universal17 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[61,'Unnamed: 3'])
column_universal18 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[61,'1-жадвал'])
column_universal19 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[50,'Unnamed: 2'])
column_universal20 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[51,'Unnamed: 2'])
column_universal21 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[53,'Unnamed: 2'])
column_universal22 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[54,'Unnamed: 2'])
column_universal23 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[55,'Unnamed: 2'])
column_universal24 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[75,'Unnamed: 2'])
column_universal25 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[18,'Unnamed: 18'])
column_universal26 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[5,'Unnamed: 18'])
column_universal27 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[11,'Unnamed: 18'])
column_universal28 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[14,'Unnamed: 18'])
column_universal29 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[9,'Unnamed: 18'])
column_universal30 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[11,'Unnamed: 18'])
column_universal31 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[14,'Unnamed: 18'])
column_universal32 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[18,'Unnamed: 19'])
column_universal33 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[18,'Unnamed: 20'])
column_universal35 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[18,'Unnamed: 14'])
column_universal36 = float(pd.read_excel(excel_file_universal ,sheet_name=s18).at[18,'Unnamed: 16'])
column_universal34 = column_universal25 - column_universal35 - column_universal36
column_universal40 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[13,'Unnamed: 2'])
column_universal41 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[13,'Unnamed: 3'])
column_universal42 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[13,'3-жадвал'])
column_universal43 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[5,'Unnamed: 2'])
column_universal44 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[10,'Unnamed: 2'])
column_universal45 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[12,'Unnamed: 2'])
column_universal46 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[43,'Unnamed: 2'])
column_universal47 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[43,'Unnamed: 3'])
column_universal48 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[43,'3-жадвал'])
column_universal37 = column_universal40 + column_universal46
column_universal38 = column_universal41 + column_universal47
column_universal39 = column_universal42 + column_universal48
column_universal49 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[35,'Unnamed: 2'])
column_universal50 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[39,'Unnamed: 2'])
column_universal51 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[41,'Unnamed: 2'])
column_universal52 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[42,'Unnamed: 2'])
column_universal56 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[25,'Unnamed: 2'])
column_universal57 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[25,'Unnamed: 3'])
column_universal58 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[25,'3-жадвал'])
column_universal59 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[19,'Unnamed: 2'])
column_universal60 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[19,'Unnamed: 2'])
column_universal61 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[24,'Unnamed: 2'])
column_universal62 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[51,'Unnamed: 2'])
column_universal63 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[51,'Unnamed: 3'])
column_universal64 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[51,'3-жадвал'])
column_universal65 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[67,'Unnamed: 2'])
column_universal66 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[56,'Unnamed: 2'])
column_universal67 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[76,'Unnamed: 2'])
column_universal53 = column_universal37 - column_universal67
column_universal54 = column_universal57 + column_universal63 + column_universal65
column_universal55 = column_universal58 + column_universal64
column_universal68 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[69,'Unnamed: 2'])
column_universal70 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[28,'Unnamed: 2'])
column_universal71 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[31,'Unnamed: 2'])
column_universal69 = column_universal70 + column_universal71
column_universal72 = float(pd.read_excel(excel_file_universal ,sheet_name=s22).at[5,'22-жадвал'])
column_universal73 = float(pd.read_excel(excel_file_universal ,sheet_name=s22).at[7,'22-жадвал'])
column_universal74 = float(pd.read_excel(excel_file_universal ,sheet_name=s22).at[11,'22-жадвал'])
column_universal75 = float(pd.read_excel(excel_file_universal ,sheet_name=s22).at[17,'22-жадвал'])
column_universal76 = float(pd.read_excel(excel_file_universal ,sheet_name=s22).at[20,'22-жадвал'])
column_universal77 = 0
if s23 is not None:
    column_universal77 = float(pd.read_excel(excel_file_universal ,sheet_name=s23).at[3,'23-жадвал'])
column_universal78 = 0
if s23 is not None:
    column_universal78 = float(pd.read_excel(excel_file_universal ,sheet_name=s23).at[6,'23-жадвал'])
column_universal79 = 0
if s23 is not None:
    column_universal79 = float(pd.read_excel(excel_file_universal ,sheet_name=s23).at[9,'23-жадвал'])
column_universal83 = 0
if s23 is not None:
    column_universal83 = float(pd.read_excel(excel_file_universal ,sheet_name=s23).at[10,'23-жадвал'])
column_universal84 = 0
if s23 is not None:
    column_universal84 = float(pd.read_excel(excel_file_universal ,sheet_name=s23).at[12,'23-жадвал'])
column_universal85 = column_universal83 - column_universal84
column_universal89 = column_universal9 + column_universal6/2 + column_universal7 + column_universal8
column_universal90 = column_universal25 + column_universal19 + column_universal20/2 + column_universal21 + column_universal22 + column_universal24
column_universal91 = float(pd.read_excel(excel_file_universal ,sheet_name=s1).at[37,'Unnamed: 2'])
column_universal92 = float(pd.read_excel(excel_file_universal ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы AVO)
excel_file_avo = os.path.join(folder, 'uaeb.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_avo, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_avo, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_avo, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_avo, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_avo, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_avo, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_avo1 = period
column_avo2 = "AVO"
column_avo3 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[45,'Unnamed: 2'])
column_avo4 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[45,'Unnamed: 3'])
column_avo5 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[45,'1-жадвал'])
column_avo6 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[6,'Unnamed: 2'])
column_avo7 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[9,'Unnamed: 2'])
column_avo8 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[16,'Unnamed: 2'])
column_avo9 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[23,'Unnamed: 2'])
column_avo10 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[23,'Unnamed: 3'])
column_avo11 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[23,'1-жадвал'])
column_avo12 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[24,'Unnamed: 2'])
column_avo14 = float(pd.read_excel(excel_file_avo ,sheet_name=s17).at[28,'Unnamed: 14'])
column_avo15 = float(pd.read_excel(excel_file_avo ,sheet_name=s17).at[28,'Unnamed: 16'])
column_avo13 = column_avo9 - column_avo14 - column_avo15
column_avo16 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[61,'Unnamed: 2'])
column_avo17 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[61,'Unnamed: 3'])
column_avo18 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[61,'1-жадвал'])
column_avo19 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[50,'Unnamed: 2'])
column_avo20 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[51,'Unnamed: 2'])
column_avo21 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[53,'Unnamed: 2'])
column_avo22 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[54,'Unnamed: 2'])
column_avo23 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[55,'Unnamed: 2'])
column_avo24 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[75,'Unnamed: 2'])
column_avo25 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[18,'Unnamed: 18'])
column_avo26 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[5,'Unnamed: 18'])
column_avo27 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[11,'Unnamed: 18'])
column_avo28 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[14,'Unnamed: 18'])
column_avo29 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[9,'Unnamed: 18'])
column_avo30 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[11,'Unnamed: 18'])
column_avo31 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[14,'Unnamed: 18'])
column_avo32 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[18,'Unnamed: 19'])
column_avo33 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[18,'Unnamed: 20'])
column_avo35 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[18,'Unnamed: 14'])
column_avo36 = float(pd.read_excel(excel_file_avo ,sheet_name=s18).at[18,'Unnamed: 16'])
column_avo34 = column_avo25 - column_avo35 - column_avo36
column_avo40 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[13,'Unnamed: 2'])
column_avo41 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[13,'Unnamed: 3'])
column_avo42 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[13,'3-жадвал'])
column_avo43 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[5,'Unnamed: 2'])
column_avo44 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[10,'Unnamed: 2'])
column_avo45 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[12,'Unnamed: 2'])
column_avo46 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[43,'Unnamed: 2'])
column_avo47 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[43,'Unnamed: 3'])
column_avo48 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[43,'3-жадвал'])
column_avo37 = column_avo40 + column_avo46
column_avo38 = column_avo41 + column_avo47
column_avo39 = column_avo42 + column_avo48
column_avo49 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[35,'Unnamed: 2'])
column_avo50 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[39,'Unnamed: 2'])
column_avo51 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[41,'Unnamed: 2'])
column_avo52 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[42,'Unnamed: 2'])
column_avo56 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[25,'Unnamed: 2'])
column_avo57 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[25,'Unnamed: 3'])
column_avo58 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[25,'3-жадвал'])
column_avo59 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[19,'Unnamed: 2'])
column_avo60 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[19,'Unnamed: 2'])
column_avo61 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[24,'Unnamed: 2'])
column_avo62 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[51,'Unnamed: 2'])
column_avo63 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[51,'Unnamed: 3'])
column_avo64 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[51,'3-жадвал'])
column_avo65 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[67,'Unnamed: 2'])
column_avo66 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[56,'Unnamed: 2'])
column_avo67 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[76,'Unnamed: 2'])
column_avo53 = column_avo37 - column_avo67
column_avo54 = column_avo57 + column_avo63 + column_avo65
column_avo55 = column_avo58 + column_avo64
column_avo68 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[69,'Unnamed: 2'])
column_avo70 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[28,'Unnamed: 2'])
column_avo71 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[31,'Unnamed: 2'])
column_avo69 = column_avo70 + column_avo71
column_avo72 = float(pd.read_excel(excel_file_avo ,sheet_name=s22).at[5,'22-жадвал'])
column_avo73 = float(pd.read_excel(excel_file_avo ,sheet_name=s22).at[7,'22-жадвал'])
column_avo74 = float(pd.read_excel(excel_file_avo ,sheet_name=s22).at[11,'22-жадвал'])
column_avo75 = float(pd.read_excel(excel_file_avo ,sheet_name=s22).at[17,'22-жадвал'])
column_avo76 = float(pd.read_excel(excel_file_avo ,sheet_name=s22).at[20,'22-жадвал'])
column_avo77 = 0
if s23 is not None:
    column_avo77 = float(pd.read_excel(excel_file_avo ,sheet_name=s23).at[3,'23-жадвал'])
column_avo78 = 0
if s23 is not None:
    column_avo78 = float(pd.read_excel(excel_file_avo ,sheet_name=s23).at[6,'23-жадвал'])
column_avo79 = 0
if s23 is not None:
    column_avo79 = float(pd.read_excel(excel_file_avo ,sheet_name=s23).at[9,'23-жадвал'])
column_avo83 = 0
if s23 is not None:
    column_avo83 = float(pd.read_excel(excel_file_avo ,sheet_name=s23).at[10,'23-жадвал'])
column_avo84 = 0
if s23 is not None:
    column_avo84 = float(pd.read_excel(excel_file_avo ,sheet_name=s23).at[12,'23-жадвал'])
column_avo85 = column_avo83 - column_avo84
column_avo89 = column_avo9 + column_avo6/2 + column_avo7 + column_avo8
column_avo90 = column_avo25 + column_avo19 + column_avo20/2 + column_avo21 + column_avo22 + column_avo24
column_avo91 = float(pd.read_excel(excel_file_avo ,sheet_name=s1).at[37,'Unnamed: 2'])
column_avo92 = float(pd.read_excel(excel_file_avo ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Мадад)
excel_file_madad = os.path.join(folder, 'madadinvestbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_madad, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_madad, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_madad, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_madad, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_madad, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_madad, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_madad1 = period
column_madad2 = "MADAD INVEST"
column_madad3 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[45,'Unnamed: 2'])
column_madad4 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[45,'Unnamed: 3'])
column_madad5 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[45,'1-жадвал'])
column_madad6 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[6,'Unnamed: 2'])
column_madad7 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[9,'Unnamed: 2'])
column_madad8 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[16,'Unnamed: 2'])
column_madad9 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[23,'Unnamed: 2'])
column_madad10 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[23,'Unnamed: 3'])
column_madad11 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[23,'1-жадвал'])
column_madad12 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[24,'Unnamed: 2'])
column_madad14 = float(pd.read_excel(excel_file_madad ,sheet_name=s17).at[28,'Unnamed: 14'])
column_madad15 = float(pd.read_excel(excel_file_madad ,sheet_name=s17).at[28,'Unnamed: 16'])
column_madad13 = column_madad9 - column_madad14 - column_madad15
column_madad16 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[61,'Unnamed: 2'])
column_madad17 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[61,'Unnamed: 3'])
column_madad18 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[61,'1-жадвал'])
column_madad19 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[50,'Unnamed: 2'])
column_madad20 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[51,'Unnamed: 2'])
column_madad21 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[53,'Unnamed: 2'])
column_madad22 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[54,'Unnamed: 2'])
column_madad23 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[55,'Unnamed: 2'])
column_madad24 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[75,'Unnamed: 2'])
column_madad25 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[18,'Unnamed: 18'])
column_madad26 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[5,'Unnamed: 18'])
column_madad27 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[11,'Unnamed: 18'])
column_madad28 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[14,'Unnamed: 18'])
column_madad29 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[9,'Unnamed: 18'])
column_madad30 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[11,'Unnamed: 18'])
column_madad31 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[14,'Unnamed: 18'])
column_madad32 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[18,'Unnamed: 19'])
column_madad33 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[18,'Unnamed: 20'])
column_madad35 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[18,'Unnamed: 14'])
column_madad36 = float(pd.read_excel(excel_file_madad ,sheet_name=s18).at[18,'Unnamed: 16'])
column_madad34 = column_madad25 - column_madad35 - column_madad36
column_madad40 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[13,'Unnamed: 2'])
column_madad41 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[13,'Unnamed: 3'])
column_madad42 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[13,'3-жадвал'])
column_madad43 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[5,'Unnamed: 2'])
column_madad44 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[10,'Unnamed: 2'])
column_madad45 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[12,'Unnamed: 2'])
column_madad46 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[43,'Unnamed: 2'])
column_madad47 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[43,'Unnamed: 3'])
column_madad48 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[43,'3-жадвал'])
column_madad37 = column_madad40 + column_madad46
column_madad38 = column_madad41 + column_madad47
column_madad39 = column_madad42 + column_madad48
column_madad49 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[35,'Unnamed: 2'])
column_madad50 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[39,'Unnamed: 2'])
column_madad51 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[41,'Unnamed: 2'])
column_madad52 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[42,'Unnamed: 2'])
column_madad56 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[25,'Unnamed: 2'])
column_madad57 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[25,'Unnamed: 3'])
column_madad58 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[25,'3-жадвал'])
column_madad59 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[19,'Unnamed: 2'])
column_madad60 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[19,'Unnamed: 2'])
column_madad61 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[24,'Unnamed: 2'])
column_madad62 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[51,'Unnamed: 2'])
column_madad63 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[51,'Unnamed: 3'])
column_madad64 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[51,'3-жадвал'])
column_madad65 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[67,'Unnamed: 2'])
column_madad66 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[56,'Unnamed: 2'])
column_madad67 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[76,'Unnamed: 2'])
column_madad53 = column_madad37 - column_madad67
column_madad54 = column_madad57 + column_madad63 + column_madad65
column_madad55 = column_madad58 + column_madad64
column_madad68 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[69,'Unnamed: 2'])
column_madad70 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[28,'Unnamed: 2'])
column_madad71 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[31,'Unnamed: 2'])
column_madad69 = column_madad70 + column_madad71
column_madad72 = float(pd.read_excel(excel_file_madad ,sheet_name=s22).at[5,'22-жадвал'])
column_madad73 = float(pd.read_excel(excel_file_madad ,sheet_name=s22).at[7,'22-жадвал'])
column_madad74 = float(pd.read_excel(excel_file_madad ,sheet_name=s22).at[11,'22-жадвал'])
column_madad75 = float(pd.read_excel(excel_file_madad ,sheet_name=s22).at[17,'22-жадвал'])
column_madad76 = float(pd.read_excel(excel_file_madad ,sheet_name=s22).at[20,'22-жадвал'])
column_madad77 = 0
if s23 is not None:
    column_madad77 = float(pd.read_excel(excel_file_madad ,sheet_name=s23).at[3,'23-жадвал'])
column_madad78 = 0
if s23 is not None:
    column_madad78 = float(pd.read_excel(excel_file_madad ,sheet_name=s23).at[6,'23-жадвал'])
column_madad79 = 0
if s23 is not None:
    column_madad79 = float(pd.read_excel(excel_file_madad ,sheet_name=s23).at[9,'23-жадвал'])
column_madad83 = 0
if s23 is not None:
    column_madad83 = float(pd.read_excel(excel_file_madad ,sheet_name=s23).at[10,'23-жадвал'])
column_madad84 = 0
if s23 is not None:
    column_madad84 = float(pd.read_excel(excel_file_madad ,sheet_name=s23).at[12,'23-жадвал'])
column_madad85 = column_madad83 - column_madad84
column_madad89 = column_madad9 + column_madad6/2 + column_madad7 + column_madad8
column_madad90 = column_madad25 + column_madad19 + column_madad20/2 + column_madad21 + column_madad22 + column_madad24
column_madad91 = float(pd.read_excel(excel_file_madad ,sheet_name=s1).at[37,'Unnamed: 2'])
column_madad92 = float(pd.read_excel(excel_file_madad ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы OFB)
excel_file_ofb = os.path.join(folder, 'ofb.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_ofb, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_ofb, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_ofb, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_ofb, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_ofb, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_ofb, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_ofb1 = period
column_ofb2 = "ORIENT FINANCE"
column_ofb3 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[45,'Unnamed: 2'])
column_ofb4 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[45,'Unnamed: 3'])
column_ofb5 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[45,'1-жадвал'])
column_ofb6 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[6,'Unnamed: 2'])
column_ofb7 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[9,'Unnamed: 2'])
column_ofb8 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[16,'Unnamed: 2'])
column_ofb9 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[23,'Unnamed: 2'])
column_ofb10 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[23,'Unnamed: 3'])
column_ofb11 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[23,'1-жадвал'])
column_ofb12 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[24,'Unnamed: 2'])
column_ofb14 = float(pd.read_excel(excel_file_ofb ,sheet_name=s17).at[28,'Unnamed: 14'])
column_ofb15 = float(pd.read_excel(excel_file_ofb ,sheet_name=s17).at[28,'Unnamed: 16'])
column_ofb13 = column_ofb9 - column_ofb14 - column_ofb15
column_ofb16 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[61,'Unnamed: 2'])
column_ofb17 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[61,'Unnamed: 3'])
column_ofb18 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[61,'1-жадвал'])
column_ofb19 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[50,'Unnamed: 2'])
column_ofb20 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[51,'Unnamed: 2'])
column_ofb21 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[53,'Unnamed: 2'])
column_ofb22 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[54,'Unnamed: 2'])
column_ofb23 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[55,'Unnamed: 2'])
column_ofb24 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[75,'Unnamed: 2'])
column_ofb25 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[18,'Unnamed: 18'])
column_ofb26 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[5,'Unnamed: 18'])
column_ofb27 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[11,'Unnamed: 18'])
column_ofb28 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[14,'Unnamed: 18'])
column_ofb29 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[9,'Unnamed: 18'])
column_ofb30 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[11,'Unnamed: 18'])
column_ofb31 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[14,'Unnamed: 18'])
column_ofb32 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[18,'Unnamed: 19'])
column_ofb33 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[18,'Unnamed: 20'])
column_ofb35 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[18,'Unnamed: 14'])
column_ofb36 = float(pd.read_excel(excel_file_ofb ,sheet_name=s18).at[18,'Unnamed: 16'])
column_ofb34 = column_ofb25 - column_ofb35 - column_ofb36
column_ofb40 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[13,'Unnamed: 2'])
column_ofb41 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[13,'Unnamed: 3'])
column_ofb42 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[13,'3-жадвал'])
column_ofb43 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[5,'Unnamed: 2'])
column_ofb44 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[10,'Unnamed: 2'])
column_ofb45 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[12,'Unnamed: 2'])
column_ofb46 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[43,'Unnamed: 2'])
column_ofb47 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[43,'Unnamed: 3'])
column_ofb48 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[43,'3-жадвал'])
column_ofb37 = column_ofb40 + column_ofb46
column_ofb38 = column_ofb41 + column_ofb47
column_ofb39 = column_ofb42 + column_ofb48
column_ofb49 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[35,'Unnamed: 2'])
column_ofb50 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[39,'Unnamed: 2'])
column_ofb51 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[41,'Unnamed: 2'])
column_ofb52 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[42,'Unnamed: 2'])
column_ofb56 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[25,'Unnamed: 2'])
column_ofb57 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[25,'Unnamed: 3'])
column_ofb58 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[25,'3-жадвал'])
column_ofb59 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[19,'Unnamed: 2'])
column_ofb60 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[19,'Unnamed: 2'])
column_ofb61 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[24,'Unnamed: 2'])
column_ofb62 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[51,'Unnamed: 2'])
column_ofb63 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[51,'Unnamed: 3'])
column_ofb64 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[51,'3-жадвал'])
column_ofb65 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[67,'Unnamed: 2'])
column_ofb66 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[56,'Unnamed: 2'])
column_ofb67 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[76,'Unnamed: 2'])
column_ofb53 = column_ofb37 - column_ofb67
column_ofb54 = column_ofb57 + column_ofb63 + column_ofb65
column_ofb55 = column_ofb58 + column_ofb64
column_ofb68 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[69,'Unnamed: 2'])
column_ofb70 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[28,'Unnamed: 2'])
column_ofb71 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[31,'Unnamed: 2'])
column_ofb69 = column_ofb70 + column_ofb71
column_ofb72 = float(pd.read_excel(excel_file_ofb ,sheet_name=s22).at[5,'22-жадвал'])
column_ofb73 = float(pd.read_excel(excel_file_ofb ,sheet_name=s22).at[7,'22-жадвал'])
column_ofb74 = float(pd.read_excel(excel_file_ofb ,sheet_name=s22).at[11,'22-жадвал'])
column_ofb75 = float(pd.read_excel(excel_file_ofb ,sheet_name=s22).at[17,'22-жадвал'])
column_ofb76 = float(pd.read_excel(excel_file_ofb ,sheet_name=s22).at[20,'22-жадвал'])
column_ofb77 = 0
if s23 is not None:
    try:
        column_ofb77 = float(pd.read_excel(excel_file_ofb ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ofb78 = 0
if s23 is not None:
    try:
        column_ofb78 = float(pd.read_excel(excel_file_ofb ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ofb79 = 0
if s23 is not None:
    try:
        column_ofb79 = float(pd.read_excel(excel_file_ofb ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ofb83 = 0
if s23 is not None:
    try:
        column_ofb83 = float(pd.read_excel(excel_file_ofb ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ofb84 = 0
if s23 is not None:
    try:
        column_ofb84 = float(pd.read_excel(excel_file_ofb ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ofb85 = column_ofb83 - column_ofb84
column_ofb89 = column_ofb9 + column_ofb6/2 + column_ofb7 + column_ofb8
column_ofb90 = column_ofb25 + column_ofb19 + column_ofb20/2 + column_ofb21 + column_ofb22 + column_ofb24
column_ofb91 = float(pd.read_excel(excel_file_ofb ,sheet_name=s1).at[37,'Unnamed: 2'])
column_ofb92 = float(pd.read_excel(excel_file_ofb ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы TRUST)
excel_file_trust = os.path.join(folder, 'trastbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_trust, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_trust, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_trust, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_trust, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_trust, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_trust, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_trust1 = period
column_trust2 = "TRAST"
column_trust3 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[45,'Unnamed: 2'])
column_trust4 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[45,'Unnamed: 3'])
column_trust5 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[45,'1-жадвал'])
column_trust6 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[6,'Unnamed: 2'])
column_trust7 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[9,'Unnamed: 2'])
column_trust8 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[16,'Unnamed: 2'])
column_trust9 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[23,'Unnamed: 2'])
column_trust10 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[23,'Unnamed: 3'])
column_trust11 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[23,'1-жадвал'])
column_trust12 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[24,'Unnamed: 2'])
column_trust14 = float(pd.read_excel(excel_file_trust ,sheet_name=s17).at[28,'Unnamed: 14'])
column_trust15 = float(pd.read_excel(excel_file_trust ,sheet_name=s17).at[28,'Unnamed: 16'])
column_trust13 = column_trust9 - column_trust14 - column_trust15
column_trust16 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[61,'Unnamed: 2'])
column_trust17 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[61,'Unnamed: 3'])
column_trust18 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[61,'1-жадвал'])
column_trust19 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[50,'Unnamed: 2'])
column_trust20 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[51,'Unnamed: 2'])
column_trust21 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[53,'Unnamed: 2'])
column_trust22 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[54,'Unnamed: 2'])
column_trust23 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[55,'Unnamed: 2'])
column_trust24 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[75,'Unnamed: 2'])
column_trust25 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[18,'Unnamed: 18'])
column_trust26 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[5,'Unnamed: 18'])
column_trust27 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[11,'Unnamed: 18'])
column_trust28 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[14,'Unnamed: 18'])
column_trust29 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[9,'Unnamed: 18'])
column_trust30 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[11,'Unnamed: 18'])
column_trust31 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[14,'Unnamed: 18'])
column_trust32 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[18,'Unnamed: 19'])
column_trust33 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[18,'Unnamed: 20'])
column_trust35 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[18,'Unnamed: 14'])
column_trust36 = float(pd.read_excel(excel_file_trust ,sheet_name=s18).at[18,'Unnamed: 16'])
column_trust34 = column_trust25 - column_trust35 - column_trust36
column_trust40 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[13,'Unnamed: 2'])
column_trust41 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[13,'Unnamed: 3'])
column_trust42 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[13,'3-жадвал'])
column_trust43 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[5,'Unnamed: 2'])
column_trust44 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[10,'Unnamed: 2'])
column_trust45 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[12,'Unnamed: 2'])
column_trust46 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[43,'Unnamed: 2'])
column_trust47 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[43,'Unnamed: 3'])
column_trust48 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[43,'3-жадвал'])
column_trust37 = column_trust40 + column_trust46
column_trust38 = column_trust41 + column_trust47
column_trust39 = column_trust42 + column_trust48
column_trust49 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[35,'Unnamed: 2'])
column_trust50 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[39,'Unnamed: 2'])
column_trust51 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[41,'Unnamed: 2'])
column_trust52 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[42,'Unnamed: 2'])
column_trust56 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[25,'Unnamed: 2'])
column_trust57 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[25,'Unnamed: 3'])
column_trust58 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[25,'3-жадвал'])
column_trust59 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[19,'Unnamed: 2'])
column_trust60 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[19,'Unnamed: 2'])
column_trust61 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[24,'Unnamed: 2'])
column_trust62 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[51,'Unnamed: 2'])
column_trust63 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[51,'Unnamed: 3'])
column_trust64 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[51,'3-жадвал'])
column_trust65 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[67,'Unnamed: 2'])
column_trust66 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[56,'Unnamed: 2'])
column_trust67 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[76,'Unnamed: 2'])
column_trust53 = column_trust37 - column_trust67
column_trust54 = column_trust57 + column_trust63 + column_trust65
column_trust55 = column_trust58 + column_trust64
column_trust68 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[69,'Unnamed: 2'])
column_trust70 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[28,'Unnamed: 2'])
column_trust71 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[31,'Unnamed: 2'])
column_trust69 = column_trust70 + column_trust71
column_trust72 = float(pd.read_excel(excel_file_trust ,sheet_name=s22).at[5,'22-жадвал'])
column_trust73 = float(pd.read_excel(excel_file_trust ,sheet_name=s22).at[7,'22-жадвал'])
column_trust74 = float(pd.read_excel(excel_file_trust ,sheet_name=s22).at[11,'22-жадвал'])
column_trust75 = float(pd.read_excel(excel_file_trust ,sheet_name=s22).at[17,'22-жадвал'])
column_trust76 = float(pd.read_excel(excel_file_trust ,sheet_name=s22).at[20,'22-жадвал'])
column_trust77 = 0
if s23 is not None:
    try:
        column_trust77 = float(pd.read_excel(excel_file_trust ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_trust78 = 0
if s23 is not None:
    try:
        column_trust78 = float(pd.read_excel(excel_file_trust ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_trust79 = 0
if s23 is not None:
    try:
        column_trust79 = float(pd.read_excel(excel_file_trust ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_trust83 = 0
if s23 is not None:
    try:
        column_trust83 = float(pd.read_excel(excel_file_trust ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_trust84 = 0
if s23 is not None:
    try:
        column_trust84 = float(pd.read_excel(excel_file_trust ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_trust85 = column_trust83 - column_trust84
column_trust89 = column_trust9 + column_trust6/2 + column_trust7 + column_trust8
column_trust90 = column_trust25 + column_trust19 + column_trust20/2 + column_trust21 + column_trust22 + column_trust24
column_trust91 = float(pd.read_excel(excel_file_trust ,sheet_name=s1).at[37,'Unnamed: 2'])
column_trust92 = float(pd.read_excel(excel_file_trust ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Davr)
excel_file_davr = os.path.join(folder, 'davrbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_davr, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_davr, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_davr, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_davr, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_davr, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_davr, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_davr1 = period
column_davr2 = "DAVR"
column_davr3 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[45,'Unnamed: 2'])
column_davr4 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[45,'Unnamed: 3'])
column_davr5 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[45,'1-жадвал'])
column_davr6 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[6,'Unnamed: 2'])
column_davr7 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[9,'Unnamed: 2'])
column_davr8 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[16,'Unnamed: 2'])
column_davr9 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[23,'Unnamed: 2'])
column_davr10 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[23,'Unnamed: 3'])
column_davr11 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[23,'1-жадвал'])
column_davr12 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[24,'Unnamed: 2'])
column_davr14 = float(pd.read_excel(excel_file_davr ,sheet_name=s17).at[28,'Unnamed: 14'])
column_davr15 = float(pd.read_excel(excel_file_davr ,sheet_name=s17).at[28,'Unnamed: 16'])
column_davr13 = column_davr9 - column_davr14 - column_davr15
column_davr16 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[61,'Unnamed: 2'])
column_davr17 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[61,'Unnamed: 3'])
column_davr18 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[61,'1-жадвал'])
column_davr19 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[50,'Unnamed: 2'])
column_davr20 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[51,'Unnamed: 2'])
column_davr21 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[53,'Unnamed: 2'])
column_davr22 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[54,'Unnamed: 2'])
column_davr23 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[55,'Unnamed: 2'])
column_davr24 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[75,'Unnamed: 2'])
column_davr25 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[18,'Unnamed: 18'])
column_davr26 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[5,'Unnamed: 18'])
column_davr27 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[11,'Unnamed: 18'])
column_davr28 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[14,'Unnamed: 18'])
column_davr29 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[9,'Unnamed: 18'])
column_davr30 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[11,'Unnamed: 18'])
column_davr31 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[14,'Unnamed: 18'])
column_davr32 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[18,'Unnamed: 19'])
column_davr33 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[18,'Unnamed: 20'])
column_davr35 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[18,'Unnamed: 14'])
column_davr36 = float(pd.read_excel(excel_file_davr ,sheet_name=s18).at[18,'Unnamed: 16'])
column_davr34 = column_davr25 - column_davr35 - column_davr36
column_davr40 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[13,'Unnamed: 2'])
column_davr41 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[13,'Unnamed: 3'])
column_davr42 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[13,'3-жадвал'])
column_davr43 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[5,'Unnamed: 2'])
column_davr44 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[10,'Unnamed: 2'])
column_davr45 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[12,'Unnamed: 2'])
column_davr46 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[43,'Unnamed: 2'])
column_davr47 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[43,'Unnamed: 3'])
column_davr48 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[43,'3-жадвал'])
column_davr37 = column_davr40 + column_davr46
column_davr38 = column_davr41 + column_davr47
column_davr39 = column_davr42 + column_davr48
column_davr49 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[35,'Unnamed: 2'])
column_davr50 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[39,'Unnamed: 2'])
column_davr51 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[41,'Unnamed: 2'])
column_davr52 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[42,'Unnamed: 2'])
column_davr56 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[25,'Unnamed: 2'])
column_davr57 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[25,'Unnamed: 3'])
column_davr58 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[25,'3-жадвал'])
column_davr59 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[19,'Unnamed: 2'])
column_davr60 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[19,'Unnamed: 2'])
column_davr61 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[24,'Unnamed: 2'])
column_davr62 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[51,'Unnamed: 2'])
column_davr63 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[51,'Unnamed: 3'])
column_davr64 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[51,'3-жадвал'])
column_davr65 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[67,'Unnamed: 2'])
column_davr66 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[56,'Unnamed: 2'])
column_davr67 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[76,'Unnamed: 2'])
column_davr53 = column_davr37 - column_davr67
column_davr54 = column_davr57 + column_davr63 + column_davr65
column_davr55 = column_davr58 + column_davr64
column_davr68 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[69,'Unnamed: 2'])
column_davr70 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[28,'Unnamed: 2'])
column_davr71 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[31,'Unnamed: 2'])
column_davr69 = column_davr70 + column_davr71
column_davr72 = float(pd.read_excel(excel_file_davr ,sheet_name=s22).at[5,'22-жадвал'])
column_davr73 = float(pd.read_excel(excel_file_davr ,sheet_name=s22).at[7,'22-жадвал'])
column_davr74 = float(pd.read_excel(excel_file_davr ,sheet_name=s22).at[11,'22-жадвал'])
column_davr75 = float(pd.read_excel(excel_file_davr ,sheet_name=s22).at[17,'22-жадвал'])
column_davr76 = float(pd.read_excel(excel_file_davr ,sheet_name=s22).at[20,'22-жадвал'])
column_davr77 = 0
if s23 is not None:
    try:
        column_davr77 = float(pd.read_excel(excel_file_davr ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_davr78 = 0
if s23 is not None:
    try:
        column_davr78 = float(pd.read_excel(excel_file_davr ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_davr79 = 0
if s23 is not None:
    try:
        column_davr79 = float(pd.read_excel(excel_file_davr ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_davr83 = 0
if s23 is not None:
    try:
        column_davr83 = float(pd.read_excel(excel_file_davr ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_davr84 = 0
if s23 is not None:
    try:
        column_davr84 = float(pd.read_excel(excel_file_davr ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_davr85 = column_davr83 - column_davr84
column_davr89 = column_davr9 + column_davr6/2 + column_davr7 + column_davr8
column_davr90 = column_davr25 + column_davr19 + column_davr20/2 + column_davr21 + column_davr22 + column_davr24
column_davr91 = float(pd.read_excel(excel_file_davr ,sheet_name=s1).at[37,'Unnamed: 2'])
column_davr92 = float(pd.read_excel(excel_file_davr ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Ravnaq)
excel_file_ravnaq = os.path.join(folder, 'ravnaqbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_ravnaq, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_ravnaq, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_ravnaq, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_ravnaq, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_ravnaq, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_ravnaq, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_ravnaq1 = period
column_ravnaq2 = "OCTOBANK"
column_ravnaq3 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[45,'Unnamed: 2'])
column_ravnaq4 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[45,'Unnamed: 3'])
column_ravnaq5 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[45,'1-жадвал'])
column_ravnaq6 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[6,'Unnamed: 2'])
column_ravnaq7 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[9,'Unnamed: 2'])
column_ravnaq8 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[16,'Unnamed: 2'])
column_ravnaq9 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[23,'Unnamed: 2'])
column_ravnaq10 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[23,'Unnamed: 3'])
column_ravnaq11 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[23,'1-жадвал'])
column_ravnaq12 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[24,'Unnamed: 2'])
column_ravnaq14 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s17).at[28,'Unnamed: 14'])
column_ravnaq15 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s17).at[28,'Unnamed: 16'])
column_ravnaq13 = column_ravnaq9 - column_ravnaq14 - column_ravnaq15
column_ravnaq16 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[61,'Unnamed: 2'])
column_ravnaq17 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[61,'Unnamed: 3'])
column_ravnaq18 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[61,'1-жадвал'])
column_ravnaq19 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[50,'Unnamed: 2'])
column_ravnaq20 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[51,'Unnamed: 2'])
column_ravnaq21 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[53,'Unnamed: 2'])
column_ravnaq22 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[54,'Unnamed: 2'])
column_ravnaq23 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[55,'Unnamed: 2'])
column_ravnaq24 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[75,'Unnamed: 2'])
column_ravnaq25 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[18,'Unnamed: 18'])
column_ravnaq26 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[5,'Unnamed: 18'])
column_ravnaq27 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[11,'Unnamed: 18'])
column_ravnaq28 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[14,'Unnamed: 18'])
column_ravnaq29 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[9,'Unnamed: 18'])
column_ravnaq30 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[11,'Unnamed: 18'])
column_ravnaq31 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[14,'Unnamed: 18'])
column_ravnaq32 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[18,'Unnamed: 19'])
column_ravnaq33 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[18,'Unnamed: 20'])
column_ravnaq35 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[18,'Unnamed: 14'])
column_ravnaq36 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s18).at[18,'Unnamed: 16'])
column_ravnaq34 = column_ravnaq25 - column_ravnaq35 - column_ravnaq36
column_ravnaq40 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[13,'Unnamed: 2'])
column_ravnaq41 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[13,'Unnamed: 3'])
column_ravnaq42 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[13,'3-жадвал'])
column_ravnaq43 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[5,'Unnamed: 2'])
column_ravnaq44 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[10,'Unnamed: 2'])
column_ravnaq45 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[12,'Unnamed: 2'])
column_ravnaq46 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[43,'Unnamed: 2'])
column_ravnaq47 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[43,'Unnamed: 3'])
column_ravnaq48 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[43,'3-жадвал'])
column_ravnaq37 = column_ravnaq40 + column_ravnaq46
column_ravnaq38 = column_ravnaq41 + column_ravnaq47
column_ravnaq39 = column_ravnaq42 + column_ravnaq48
column_ravnaq49 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[35,'Unnamed: 2'])
column_ravnaq50 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[39,'Unnamed: 2'])
column_ravnaq51 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[41,'Unnamed: 2'])
column_ravnaq52 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[42,'Unnamed: 2'])
column_ravnaq56 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[25,'Unnamed: 2'])
column_ravnaq57 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[25,'Unnamed: 3'])
column_ravnaq58 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[25,'3-жадвал'])
column_ravnaq59 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[19,'Unnamed: 2'])
column_ravnaq60 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[19,'Unnamed: 2'])
column_ravnaq61 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[24,'Unnamed: 2'])
column_ravnaq62 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[51,'Unnamed: 2'])
column_ravnaq63 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[51,'Unnamed: 3'])
column_ravnaq64 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[51,'3-жадвал'])
column_ravnaq65 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[67,'Unnamed: 2'])
column_ravnaq66 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[56,'Unnamed: 2'])
column_ravnaq67 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[76,'Unnamed: 2'])
column_ravnaq53 = column_ravnaq37 - column_ravnaq67
column_ravnaq54 = column_ravnaq57 + column_ravnaq63 + column_ravnaq65
column_ravnaq55 = column_ravnaq58 + column_ravnaq64
column_ravnaq68 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[69,'Unnamed: 2'])
column_ravnaq70 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[28,'Unnamed: 2'])
column_ravnaq71 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[31,'Unnamed: 2'])
column_ravnaq69 = column_ravnaq70 + column_ravnaq71
column_ravnaq72 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s22).at[5,'22-жадвал'])
column_ravnaq73 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s22).at[7,'22-жадвал'])
column_ravnaq74 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s22).at[11,'22-жадвал'])
column_ravnaq75 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s22).at[17,'22-жадвал'])
column_ravnaq76 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s22).at[20,'22-жадвал'])
column_ravnaq77 = 0
if s23 is not None:
    try:
        column_ravnaq77 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ravnaq78 = 0
if s23 is not None:
    try:
        column_ravnaq78 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ravnaq79 = 0
if s23 is not None:
    try:
        column_ravnaq79 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ravnaq83 = 0
if s23 is not None:
    try:
        column_ravnaq83 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ravnaq84 = 0
if s23 is not None:
    try:
        column_ravnaq84 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_ravnaq85 = column_ravnaq83 - column_ravnaq84
column_ravnaq89 = column_ravnaq9 + column_ravnaq6/2 + column_ravnaq7 + column_ravnaq8
column_ravnaq90 = column_ravnaq25 + column_ravnaq19 + column_ravnaq20/2 + column_ravnaq21 + column_ravnaq22 + column_ravnaq24
column_ravnaq91 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s1).at[37,'Unnamed: 2'])
column_ravnaq92 = float(pd.read_excel(excel_file_ravnaq ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Anor)
excel_file_anor = os.path.join(folder, 'anorbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_anor, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_anor, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_anor, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_anor, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_anor, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_anor, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_anor1 = period
column_anor2 = "ANOR"
column_anor3 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[45,'Unnamed: 2'])
column_anor4 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[45,'Unnamed: 3'])
column_anor5 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[45,'1-жадвал'])
column_anor6 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[6,'Unnamed: 2'])
column_anor7 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[9,'Unnamed: 2'])
column_anor8 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[16,'Unnamed: 2'])
column_anor9 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[23,'Unnamed: 2'])
column_anor10 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[23,'Unnamed: 3'])
column_anor11 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[23,'1-жадвал'])
column_anor12 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[24,'Unnamed: 2'])
column_anor14 = float(pd.read_excel(excel_file_anor ,sheet_name=s17).at[28,'Unnamed: 14'])
column_anor15 = float(pd.read_excel(excel_file_anor ,sheet_name=s17).at[28,'Unnamed: 16'])
column_anor13 = column_anor9 - column_anor14 - column_anor15
column_anor16 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[61,'Unnamed: 2'])
column_anor17 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[61,'Unnamed: 3'])
column_anor18 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[61,'1-жадвал'])
column_anor19 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[50,'Unnamed: 2'])
column_anor20 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[51,'Unnamed: 2'])
column_anor21 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[53,'Unnamed: 2'])
column_anor22 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[54,'Unnamed: 2'])
column_anor23 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[55,'Unnamed: 2'])
column_anor24 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[75,'Unnamed: 2'])
column_anor25 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[18,'Unnamed: 18'])
column_anor26 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[5,'Unnamed: 18'])
column_anor27 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[11,'Unnamed: 18'])
column_anor28 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[14,'Unnamed: 18'])
column_anor29 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[9,'Unnamed: 18'])
column_anor30 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[11,'Unnamed: 18'])
column_anor31 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[14,'Unnamed: 18'])
column_anor32 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[18,'Unnamed: 19'])
column_anor33 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[18,'Unnamed: 20'])
column_anor35 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[18,'Unnamed: 14'])
column_anor36 = float(pd.read_excel(excel_file_anor ,sheet_name=s18).at[18,'Unnamed: 16'])
column_anor34 = column_anor25 - column_anor35 - column_anor36
column_anor40 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[13,'Unnamed: 2'])
column_anor41 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[13,'Unnamed: 3'])
column_anor42 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[13,'3-жадвал'])
column_anor43 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[5,'Unnamed: 2'])
column_anor44 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[10,'Unnamed: 2'])
column_anor45 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[12,'Unnamed: 2'])
column_anor46 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[43,'Unnamed: 2'])
column_anor47 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[43,'Unnamed: 3'])
column_anor48 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[43,'3-жадвал'])
column_anor37 = column_anor40 + column_anor46
column_anor38 = column_anor41 + column_anor47
column_anor39 = column_anor42 + column_anor48
column_anor49 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[35,'Unnamed: 2'])
column_anor50 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[39,'Unnamed: 2'])
column_anor51 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[41,'Unnamed: 2'])
column_anor52 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[42,'Unnamed: 2'])
column_anor56 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[25,'Unnamed: 2'])
column_anor57 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[25,'Unnamed: 3'])
column_anor58 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[25,'3-жадвал'])
column_anor59 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[19,'Unnamed: 2'])
column_anor60 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[19,'Unnamed: 2'])
column_anor61 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[24,'Unnamed: 2'])
column_anor62 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[51,'Unnamed: 2'])
column_anor63 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[51,'Unnamed: 3'])
column_anor64 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[51,'3-жадвал'])
column_anor65 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[67,'Unnamed: 2'])
column_anor66 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[56,'Unnamed: 2'])
column_anor67 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[76,'Unnamed: 2'])
column_anor53 = column_anor37 - column_anor67
column_anor54 = column_anor57 + column_anor63 + column_anor65
column_anor55 = column_anor58 + column_anor64
column_anor68 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[69,'Unnamed: 2'])
column_anor70 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[28,'Unnamed: 2'])
column_anor71 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[31,'Unnamed: 2'])
column_anor69 = column_anor70 + column_anor71
column_anor72 = float(pd.read_excel(excel_file_anor ,sheet_name=s22).at[5,'22-жадвал'])
column_anor73 = float(pd.read_excel(excel_file_anor ,sheet_name=s22).at[7,'22-жадвал'])
column_anor74 = float(pd.read_excel(excel_file_anor ,sheet_name=s22).at[11,'22-жадвал'])
column_anor75 = float(pd.read_excel(excel_file_anor ,sheet_name=s22).at[17,'22-жадвал'])
column_anor76 = float(pd.read_excel(excel_file_anor ,sheet_name=s22).at[20,'22-жадвал'])
column_anor77 = 0
if s23 is not None:
    try:
        column_anor77 = float(pd.read_excel(excel_file_anor ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_anor78 = 0
if s23 is not None:
    try:
        column_anor78 = float(pd.read_excel(excel_file_anor ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_anor79 = 0
if s23 is not None:
    try:
        column_anor79 = float(pd.read_excel(excel_file_anor ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_anor83 = 0
if s23 is not None:
    try:
        column_anor83 = float(pd.read_excel(excel_file_anor ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_anor84 = 0
if s23 is not None:
    try:
        column_anor84 = float(pd.read_excel(excel_file_anor ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_anor85 = column_anor83 - column_anor84
column_anor89 = column_anor9 + column_anor6/2 + column_anor7 + column_anor8
column_anor90 = column_anor25 + column_anor19 + column_anor20/2 + column_anor21 + column_anor22 + column_anor24
column_anor91 = float(pd.read_excel(excel_file_anor ,sheet_name=s1).at[37,'Unnamed: 2'])
column_anor92 = float(pd.read_excel(excel_file_anor ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Uzum)
excel_file_uzum = os.path.join(folder, 'uzum.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_uzum, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_uzum, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_uzum, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_uzum, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_uzum, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_uzum, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_uzum1 = period
column_uzum2 = "UZUM"
column_uzum3 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[45,'Unnamed: 2'])
column_uzum4 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[45,'Unnamed: 3'])
column_uzum5 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[45,'1-жадвал'])
column_uzum6 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[6,'Unnamed: 2'])
column_uzum7 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[9,'Unnamed: 2'])
column_uzum8 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[16,'Unnamed: 2'])
column_uzum9 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[23,'Unnamed: 2'])
column_uzum10 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[23,'Unnamed: 3'])
column_uzum11 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[23,'1-жадвал'])
column_uzum12 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[24,'Unnamed: 2'])
column_uzum14 = float(pd.read_excel(excel_file_uzum ,sheet_name=s17).at[28,'Unnamed: 14'])
column_uzum15 = float(pd.read_excel(excel_file_uzum ,sheet_name=s17).at[28,'Unnamed: 16'])
column_uzum13 = column_uzum9 - column_uzum14 - column_uzum15
column_uzum16 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[61,'Unnamed: 2'])
column_uzum17 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[61,'Unnamed: 3'])
column_uzum18 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[61,'1-жадвал'])
column_uzum19 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[50,'Unnamed: 2'])
column_uzum20 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[51,'Unnamed: 2'])
column_uzum21 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[53,'Unnamed: 2'])
column_uzum22 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[54,'Unnamed: 2'])
column_uzum23 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[55,'Unnamed: 2'])
column_uzum24 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[75,'Unnamed: 2'])
column_uzum25 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[18,'Unnamed: 18'])
column_uzum26 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[5,'Unnamed: 18'])
column_uzum27 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[11,'Unnamed: 18'])
column_uzum28 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[14,'Unnamed: 18'])
column_uzum29 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[9,'Unnamed: 18'])
column_uzum30 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[11,'Unnamed: 18'])
column_uzum31 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[14,'Unnamed: 18'])
column_uzum32 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[18,'Unnamed: 19'])
column_uzum33 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[18,'Unnamed: 20'])
column_uzum35 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[18,'Unnamed: 14'])
column_uzum36 = float(pd.read_excel(excel_file_uzum ,sheet_name=s18).at[18,'Unnamed: 16'])
column_uzum34 = column_uzum25 - column_uzum35 - column_uzum36
column_uzum40 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[13,'Unnamed: 2'])
column_uzum41 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[13,'Unnamed: 3'])
column_uzum42 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[13,'3-жадвал'])
column_uzum43 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[5,'Unnamed: 2'])
column_uzum44 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[10,'Unnamed: 2'])
column_uzum45 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[12,'Unnamed: 2'])
column_uzum46 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[43,'Unnamed: 2'])
column_uzum47 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[43,'Unnamed: 3'])
column_uzum48 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[43,'3-жадвал'])
column_uzum37 = column_uzum40 + column_uzum46
column_uzum38 = column_uzum41 + column_uzum47
column_uzum39 = column_uzum42 + column_uzum48
column_uzum49 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[35,'Unnamed: 2'])
column_uzum50 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[39,'Unnamed: 2'])
column_uzum51 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[41,'Unnamed: 2'])
column_uzum52 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[42,'Unnamed: 2'])
column_uzum56 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[25,'Unnamed: 2'])
column_uzum57 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[25,'Unnamed: 3'])
column_uzum58 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[25,'3-жадвал'])
column_uzum59 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[19,'Unnamed: 2'])
column_uzum60 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[19,'Unnamed: 2'])
column_uzum61 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[24,'Unnamed: 2'])
column_uzum62 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[51,'Unnamed: 2'])
column_uzum63 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[51,'Unnamed: 3'])
column_uzum64 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[51,'3-жадвал'])
column_uzum65 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[67,'Unnamed: 2'])
column_uzum66 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[56,'Unnamed: 2'])
column_uzum67 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[76,'Unnamed: 2'])
column_uzum53 = column_uzum37 - column_uzum67
column_uzum54 = column_uzum57 + column_uzum63 + column_uzum65
column_uzum55 = column_uzum58 + column_uzum64
column_uzum68 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[69,'Unnamed: 2'])
column_uzum70 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[28,'Unnamed: 2'])
column_uzum71 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[31,'Unnamed: 2'])
column_uzum69 = column_uzum70 + column_uzum71
column_uzum72 = float(pd.read_excel(excel_file_uzum ,sheet_name=s22).at[5,'22-жадвал'])
column_uzum73 = float(pd.read_excel(excel_file_uzum ,sheet_name=s22).at[7,'22-жадвал'])
column_uzum74 = float(pd.read_excel(excel_file_uzum ,sheet_name=s22).at[11,'22-жадвал'])
column_uzum75 = float(pd.read_excel(excel_file_uzum ,sheet_name=s22).at[17,'22-жадвал'])
column_uzum76 = float(pd.read_excel(excel_file_uzum ,sheet_name=s22).at[20,'22-жадвал'])
column_uzum77 = 0
if s23 is not None:
    try:
        column_uzum77 = float(pd.read_excel(excel_file_uzum ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_uzum78 = 0
if s23 is not None:
    try:
        column_uzum78 = float(pd.read_excel(excel_file_uzum ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_uzum79 = 0
if s23 is not None:
    try:
        column_uzum79 = float(pd.read_excel(excel_file_uzum ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_uzum83 = 0
if s23 is not None:
    try:
        column_uzum83 = float(pd.read_excel(excel_file_uzum ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_uzum84 = 0
if s23 is not None:
    try:
        column_uzum84 = float(pd.read_excel(excel_file_uzum ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_uzum85 = column_uzum83 - column_uzum84
column_uzum89 = column_uzum9 + column_uzum6/2 + column_uzum7 + column_uzum8
column_uzum90 = column_uzum25 + column_uzum19 + column_uzum20/2 + column_uzum21 + column_uzum22 + column_uzum24
column_uzum91 = float(pd.read_excel(excel_file_uzum ,sheet_name=s1).at[37,'Unnamed: 2'])
column_uzum92 = float(pd.read_excel(excel_file_uzum ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Апекс)
excel_file_apex = os.path.join(folder, 'apexbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_apex, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_apex, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_apex, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_apex, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_apex, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_apex, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
    # Получение значений
    column_apex1 = period
    column_apex2 = "APEX"
    column_apex3 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[45,'Unnamed: 2'])
    column_apex4 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[45,'Unnamed: 3'])
    column_apex5 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[45,'1-жадвал'])
    column_apex6 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[6,'Unnamed: 2'])
    column_apex7 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[9,'Unnamed: 2'])
    column_apex8 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[16,'Unnamed: 2'])
    column_apex9 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[23,'Unnamed: 2'])
    column_apex10 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[23,'Unnamed: 3'])
    column_apex11 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[23,'1-жадвал'])
    column_apex12 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[24,'Unnamed: 2'])
    column_apex14 = float(pd.read_excel(excel_file_apex ,sheet_name=s17).at[28,'Unnamed: 14'])
    column_apex15 = float(pd.read_excel(excel_file_apex ,sheet_name=s17).at[28,'Unnamed: 16'])
    column_apex13 = column_apex9 - column_apex14 - column_apex15
    column_apex16 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[61,'Unnamed: 2'])
    column_apex17 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[61,'Unnamed: 3'])
    column_apex18 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[61,'1-жадвал'])
    column_apex19 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[50,'Unnamed: 2'])
    column_apex20 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[51,'Unnamed: 2'])
    column_apex21 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[53,'Unnamed: 2'])
    column_apex22 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[54,'Unnamed: 2'])
    column_apex23 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[55,'Unnamed: 2'])
    column_apex24 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[75,'Unnamed: 2'])
    column_apex25 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[18,'Unnamed: 18'])
    column_apex26 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[5,'Unnamed: 18'])
    column_apex27 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[11,'Unnamed: 18'])
    column_apex28 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[14,'Unnamed: 18'])
    column_apex29 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[9,'Unnamed: 18'])
    column_apex30 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[11,'Unnamed: 18'])
    column_apex31 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[14,'Unnamed: 18'])
    column_apex32 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[18,'Unnamed: 19'])
    column_apex33 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[18,'Unnamed: 20'])
    column_apex35 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[18,'Unnamed: 14'])
    column_apex36 = float(pd.read_excel(excel_file_apex ,sheet_name=s18).at[18,'Unnamed: 16'])
    column_apex34 = column_apex25 - column_apex35 - column_apex36
    column_apex40 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[13,'Unnamed: 2'])
    column_apex41 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[13,'Unnamed: 3'])
    column_apex42 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[13,'3-жадвал'])
    column_apex43 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[5,'Unnamed: 2'])
    column_apex44 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[10,'Unnamed: 2'])
    column_apex45 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[12,'Unnamed: 2'])
    column_apex46 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[43,'Unnamed: 2'])
    column_apex47 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[43,'Unnamed: 3'])
    column_apex48 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[43,'3-жадвал'])
    column_apex37 = column_apex40 + column_apex46
    column_apex38 = column_apex41 + column_apex47
    column_apex39 = column_apex42 + column_apex48
    column_apex49 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[35,'Unnamed: 2'])
    column_apex50 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[39,'Unnamed: 2'])
    column_apex51 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[41,'Unnamed: 2'])
    column_apex52 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[42,'Unnamed: 2'])
    column_apex56 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[25,'Unnamed: 2'])
    column_apex57 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[25,'Unnamed: 3'])
    column_apex58 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[25,'3-жадвал'])
    column_apex59 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[19,'Unnamed: 2'])
    column_apex60 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[19,'Unnamed: 2'])
    column_apex61 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[24,'Unnamed: 2'])
    column_apex62 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[51,'Unnamed: 2'])
    column_apex63 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[51,'Unnamed: 3'])
    column_apex64 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[51,'3-жадвал'])
    column_apex65 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[67,'Unnamed: 2'])
    column_apex66 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[56,'Unnamed: 2'])
    column_apex67 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[76,'Unnamed: 2'])
    column_apex53 = column_apex37 - column_apex67
    column_apex54 = column_apex57 + column_apex63 + column_apex65
    column_apex55 = column_apex58 + column_apex64
    column_apex68 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[69,'Unnamed: 2'])
    column_apex70 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[28,'Unnamed: 2'])
    column_apex71 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[31,'Unnamed: 2'])
    column_apex69 = column_apex70 + column_apex71
    column_apex72 = float(pd.read_excel(excel_file_apex ,sheet_name=s22).at[5,'22-жадвал'])
    column_apex73 = float(pd.read_excel(excel_file_apex ,sheet_name=s22).at[7,'22-жадвал'])
    column_apex74 = float(pd.read_excel(excel_file_apex ,sheet_name=s22).at[11,'22-жадвал'])
    column_apex75 = float(pd.read_excel(excel_file_apex ,sheet_name=s22).at[17,'22-жадвал'])
    column_apex76 = float(pd.read_excel(excel_file_apex ,sheet_name=s22).at[20,'22-жадвал'])
    column_apex77 = 0
    if s23 is not None:
        try:
            column_apex77 = float(pd.read_excel(excel_file_apex ,sheet_name=s23).at[3,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_apex78 = 0
    if s23 is not None:
        try:
            column_apex78 = float(pd.read_excel(excel_file_apex ,sheet_name=s23).at[6,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_apex79 = 0
    if s23 is not None:
        try:
            column_apex79 = float(pd.read_excel(excel_file_apex ,sheet_name=s23).at[9,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_apex83 = 0
    if s23 is not None:
        try:
            column_apex83 = float(pd.read_excel(excel_file_apex ,sheet_name=s23).at[10,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_apex84 = 0
    if s23 is not None:
        try:
            column_apex84 = float(pd.read_excel(excel_file_apex ,sheet_name=s23).at[12,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_apex85 = column_apex83 - column_apex84
    column_apex89 = column_apex9 + column_apex6/2 + column_apex7 + column_apex8
    column_apex90 = column_apex25 + column_apex19 + column_apex20/2 + column_apex21 + column_apex22 + column_apex24
    column_apex91 = float(pd.read_excel(excel_file_apex ,sheet_name=s1).at[37,'Unnamed: 2'])
    column_apex92 = float(pd.read_excel(excel_file_apex ,sheet_name=s3).at[41,'Unnamed: 2'])
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
    
# Прочитайте данные из Excel-файла для каждого поля (для таблицы Hayot)
excel_file_hayot = os.path.join(folder, 'hayotbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_hayot, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_hayot, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_hayot, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_hayot, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_hayot, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_hayot, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
    # Получение значений
    column_hayot1 = period
    column_hayot2 = "HAYOT"
    column_hayot3 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[45,'Unnamed: 2'])
    column_hayot4 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[45,'Unnamed: 3'])
    column_hayot5 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[45,'1-жадвал'])
    column_hayot6 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[6,'Unnamed: 2'])
    column_hayot7 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[9,'Unnamed: 2'])
    column_hayot8 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[16,'Unnamed: 2'])
    column_hayot9 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[23,'Unnamed: 2'])
    column_hayot10 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[23,'Unnamed: 3'])
    column_hayot11 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[23,'1-жадвал'])
    column_hayot12 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[24,'Unnamed: 2'])
    column_hayot14 = float(pd.read_excel(excel_file_hayot ,sheet_name=s17).at[28,'Unnamed: 14'])
    column_hayot15 = float(pd.read_excel(excel_file_hayot ,sheet_name=s17).at[28,'Unnamed: 16'])
    column_hayot13 = column_hayot9 - column_hayot14 - column_hayot15
    column_hayot16 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[61,'Unnamed: 2'])
    column_hayot17 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[61,'Unnamed: 3'])
    column_hayot18 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[61,'1-жадвал'])
    column_hayot19 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[50,'Unnamed: 2'])
    column_hayot20 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[51,'Unnamed: 2'])
    column_hayot21 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[53,'Unnamed: 2'])
    column_hayot22 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[54,'Unnamed: 2'])
    column_hayot23 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[55,'Unnamed: 2'])
    column_hayot24 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[75,'Unnamed: 2'])
    column_hayot25 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[18,'Unnamed: 18'])
    column_hayot26 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[5,'Unnamed: 18'])
    column_hayot27 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[11,'Unnamed: 18'])
    column_hayot28 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[14,'Unnamed: 18'])
    column_hayot29 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[9,'Unnamed: 18'])
    column_hayot30 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[11,'Unnamed: 18'])
    column_hayot31 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[14,'Unnamed: 18'])
    column_hayot32 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[18,'Unnamed: 19'])
    column_hayot33 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[18,'Unnamed: 20'])
    column_hayot35 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[18,'Unnamed: 14'])
    column_hayot36 = float(pd.read_excel(excel_file_hayot ,sheet_name=s18).at[18,'Unnamed: 16'])
    column_hayot34 = column_hayot25 - column_hayot35 - column_hayot36
    column_hayot40 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[13,'Unnamed: 2'])
    column_hayot41 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[13,'Unnamed: 3'])
    column_hayot42 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[13,'3-жадвал'])
    column_hayot43 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[5,'Unnamed: 2'])
    column_hayot44 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[10,'Unnamed: 2'])
    column_hayot45 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[12,'Unnamed: 2'])
    column_hayot46 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[43,'Unnamed: 2'])
    column_hayot47 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[43,'Unnamed: 3'])
    column_hayot48 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[43,'3-жадвал'])
    column_hayot37 = column_hayot40 + column_hayot46
    column_hayot38 = column_hayot41 + column_hayot47
    column_hayot39 = column_hayot42 + column_hayot48
    column_hayot49 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[35,'Unnamed: 2'])
    column_hayot50 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[39,'Unnamed: 2'])
    column_hayot51 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[41,'Unnamed: 2'])
    column_hayot52 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[42,'Unnamed: 2'])
    column_hayot56 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[25,'Unnamed: 2'])
    column_hayot57 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[25,'Unnamed: 3'])
    column_hayot58 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[25,'3-жадвал'])
    column_hayot59 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[19,'Unnamed: 2'])
    column_hayot60 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[19,'Unnamed: 2'])
    column_hayot61 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[24,'Unnamed: 2'])
    column_hayot62 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[51,'Unnamed: 2'])
    column_hayot63 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[51,'Unnamed: 3'])
    column_hayot64 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[51,'3-жадвал'])
    column_hayot65 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[67,'Unnamed: 2'])
    column_hayot66 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[56,'Unnamed: 2'])
    column_hayot67 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[76,'Unnamed: 2'])
    column_hayot53 = column_hayot37 - column_hayot67
    column_hayot54 = column_hayot57 + column_hayot63 + column_hayot65
    column_hayot55 = column_hayot58 + column_hayot64
    column_hayot68 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[69,'Unnamed: 2'])
    column_hayot70 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[28,'Unnamed: 2'])
    column_hayot71 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[31,'Unnamed: 2'])
    column_hayot69 = column_hayot70 + column_hayot71
    column_hayot72 = float(pd.read_excel(excel_file_hayot ,sheet_name=s22).at[5,'22-жадвал'])
    column_hayot73 = float(pd.read_excel(excel_file_hayot ,sheet_name=s22).at[7,'22-жадвал'])
    column_hayot74 = float(pd.read_excel(excel_file_hayot ,sheet_name=s22).at[11,'22-жадвал'])
    column_hayot75 = float(pd.read_excel(excel_file_hayot ,sheet_name=s22).at[17,'22-жадвал'])
    column_hayot76 = float(pd.read_excel(excel_file_hayot ,sheet_name=s22).at[20,'22-жадвал'])
    column_hayot77 = 0
    if s23 is not None:
        try:
            column_hayot77 = float(pd.read_excel(excel_file_hayot ,sheet_name=s23).at[3,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_hayot78 = 0
    if s23 is not None:
        try:
            column_hayot78 = float(pd.read_excel(excel_file_hayot ,sheet_name=s23).at[6,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_hayot79 = 0
    if s23 is not None:
        try:
            column_hayot79 = float(pd.read_excel(excel_file_hayot ,sheet_name=s23).at[9,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_hayot83 = 0
    if s23 is not None:
        try:
            column_hayot83 = float(pd.read_excel(excel_file_hayot ,sheet_name=s23).at[10,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_hayot84 = 0
    if s23 is not None:
        try:
            column_hayot84 = float(pd.read_excel(excel_file_hayot ,sheet_name=s23).at[12,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_hayot85 = column_hayot83 - column_hayot84
    column_hayot89 = column_hayot9 + column_hayot6/2 + column_hayot7 + column_hayot8
    column_hayot90 = column_hayot25 + column_hayot19 + column_hayot20/2 + column_hayot21 + column_hayot22 + column_hayot24
    column_hayot91 = float(pd.read_excel(excel_file_hayot ,sheet_name=s1).at[37,'Unnamed: 2'])
    column_hayot92 = float(pd.read_excel(excel_file_hayot ,sheet_name=s3).at[41,'Unnamed: 2'])
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Smart)
excel_file_smart = os.path.join(folder, 'smartbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_smart, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_smart, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_smart, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_smart, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_smart, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_smart, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
    # Получение значений
    column_smart1 = period
    column_smart2 = "SMART"
    column_smart3 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[45,'Unnamed: 2'])
    column_smart4 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[45,'Unnamed: 3'])
    column_smart5 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[45,'1-жадвал'])
    column_smart6 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[6,'Unnamed: 2'])
    column_smart7 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[9,'Unnamed: 2'])
    column_smart8 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[16,'Unnamed: 2'])
    column_smart9 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[23,'Unnamed: 2'])
    column_smart10 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[23,'Unnamed: 3'])
    column_smart11 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[23,'1-жадвал'])
    column_smart12 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[24,'Unnamed: 2'])
    column_smart14 = float(pd.read_excel(excel_file_smart ,sheet_name=s17).at[28,'Unnamed: 14'])
    column_smart15 = float(pd.read_excel(excel_file_smart ,sheet_name=s17).at[28,'Unnamed: 16'])
    column_smart13 = column_smart9 - column_smart14 - column_smart15
    column_smart16 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[61,'Unnamed: 2'])
    column_smart17 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[61,'Unnamed: 3'])
    column_smart18 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[61,'1-жадвал'])
    column_smart19 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[50,'Unnamed: 2'])
    column_smart20 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[51,'Unnamed: 2'])
    column_smart21 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[53,'Unnamed: 2'])
    column_smart22 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[54,'Unnamed: 2'])
    column_smart23 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[55,'Unnamed: 2'])
    column_smart24 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[75,'Unnamed: 2'])
    column_smart25 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[18,'Unnamed: 18'])
    column_smart26 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[5,'Unnamed: 18'])
    column_smart27 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[11,'Unnamed: 18'])
    column_smart28 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[14,'Unnamed: 18'])
    column_smart29 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[9,'Unnamed: 18'])
    column_smart30 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[11,'Unnamed: 18'])
    column_smart31 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[14,'Unnamed: 18'])
    column_smart32 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[18,'Unnamed: 19'])
    column_smart33 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[18,'Unnamed: 20'])
    column_smart35 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[18,'Unnamed: 14'])
    column_smart36 = float(pd.read_excel(excel_file_smart ,sheet_name=s18).at[18,'Unnamed: 16'])
    column_smart34 = column_smart25 - column_smart35 - column_smart36
    column_smart40 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[13,'Unnamed: 2'])
    column_smart41 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[13,'Unnamed: 3'])
    column_smart42 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[13,'3-жадвал'])
    column_smart43 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[5,'Unnamed: 2'])
    column_smart44 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[10,'Unnamed: 2'])
    column_smart45 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[12,'Unnamed: 2'])
    column_smart46 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[43,'Unnamed: 2'])
    column_smart47 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[43,'Unnamed: 3'])
    column_smart48 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[43,'3-жадвал'])
    column_smart37 = column_smart40 + column_smart46
    column_smart38 = column_smart41 + column_smart47
    column_smart39 = column_smart42 + column_smart48
    column_smart49 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[35,'Unnamed: 2'])
    column_smart50 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[39,'Unnamed: 2'])
    column_smart51 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[41,'Unnamed: 2'])
    column_smart52 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[42,'Unnamed: 2'])
    column_smart56 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[25,'Unnamed: 2'])
    column_smart57 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[25,'Unnamed: 3'])
    column_smart58 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[25,'3-жадвал'])
    column_smart59 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[19,'Unnamed: 2'])
    column_smart60 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[19,'Unnamed: 2'])
    column_smart61 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[24,'Unnamed: 2'])
    column_smart62 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[51,'Unnamed: 2'])
    column_smart63 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[51,'Unnamed: 3'])
    column_smart64 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[51,'3-жадвал'])
    column_smart65 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[67,'Unnamed: 2'])
    column_smart66 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[56,'Unnamed: 2'])
    column_smart67 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[76,'Unnamed: 2'])
    column_smart53 = column_smart37 - column_smart67
    column_smart54 = column_smart57 + column_smart63 + column_smart65
    column_smart55 = column_smart58 + column_smart64
    column_smart68 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[69,'Unnamed: 2'])
    column_smart70 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[28,'Unnamed: 2'])
    column_smart71 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[31,'Unnamed: 2'])
    column_smart69 = column_smart70 + column_smart71
    column_smart72 = float(pd.read_excel(excel_file_smart ,sheet_name=s22).at[5,'22-жадвал'])
    column_smart73 = float(pd.read_excel(excel_file_smart ,sheet_name=s22).at[7,'22-жадвал'])
    column_smart74 = float(pd.read_excel(excel_file_smart ,sheet_name=s22).at[11,'22-жадвал'])
    column_smart75 = float(pd.read_excel(excel_file_smart ,sheet_name=s22).at[17,'22-жадвал'])
    column_smart76 = float(pd.read_excel(excel_file_smart ,sheet_name=s22).at[20,'22-жадвал'])
    column_smart77 = 0
    if s23 is not None:
        try:
            column_smart77 = float(pd.read_excel(excel_file_smart ,sheet_name=s23).at[3,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_smart78 = 0
    if s23 is not None:
        try:
            column_smart78 = float(pd.read_excel(excel_file_smart ,sheet_name=s23).at[6,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_smart79 = 0
    if s23 is not None:
        try:
            column_smart79 = float(pd.read_excel(excel_file_smart ,sheet_name=s23).at[9,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_smart83 = 0
    if s23 is not None:
        try:
            column_smart83 = float(pd.read_excel(excel_file_smart ,sheet_name=s23).at[10,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_smart84 = 0
    if s23 is not None:
        try:
            column_smart84 = float(pd.read_excel(excel_file_smart ,sheet_name=s23).at[12,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_smart85 = column_smart83 - column_smart84
    column_smart89 = column_smart9 + column_smart6/2 + column_smart7 + column_smart8
    column_smart90 = column_smart25 + column_smart19 + column_smart20/2 + column_smart21 + column_smart22 + column_smart24
    column_smart91 = float(pd.read_excel(excel_file_smart ,sheet_name=s1).at[37,'Unnamed: 2'])
    column_smart92 = float(pd.read_excel(excel_file_smart ,sheet_name=s3).at[41,'Unnamed: 2'])
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Прочитайте данные из Excel-файла для каждого поля (для таблицы Yangi)
excel_file_yangi = os.path.join(folder, 'yangibank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_yangi, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_yangi, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_yangi, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_yangi, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_yangi, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_yangi, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
    # Получение значений
    column_yangi1 = period
    column_yangi2 = "YANGI"
    column_yangi3 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[45,'Unnamed: 2'])
    column_yangi4 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[45,'Unnamed: 3'])
    column_yangi5 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[45,'1-жадвал'])
    column_yangi6 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[6,'Unnamed: 2'])
    column_yangi7 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[9,'Unnamed: 2'])
    column_yangi8 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[16,'Unnamed: 2'])
    column_yangi9 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[23,'Unnamed: 2'])
    column_yangi10 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[23,'Unnamed: 3'])
    column_yangi11 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[23,'1-жадвал'])
    column_yangi12 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[24,'Unnamed: 2'])
    column_yangi14 = float(pd.read_excel(excel_file_yangi ,sheet_name=s17).at[28,'Unnamed: 14'])
    column_yangi15 = float(pd.read_excel(excel_file_yangi ,sheet_name=s17).at[28,'Unnamed: 16'])
    column_yangi13 = column_yangi9 - column_yangi14 - column_yangi15
    column_yangi16 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[61,'Unnamed: 2'])
    column_yangi17 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[61,'Unnamed: 3'])
    column_yangi18 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[61,'1-жадвал'])
    column_yangi19 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[50,'Unnamed: 2'])
    column_yangi20 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[51,'Unnamed: 2'])
    column_yangi21 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[53,'Unnamed: 2'])
    column_yangi22 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[54,'Unnamed: 2'])
    column_yangi23 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[55,'Unnamed: 2'])
    column_yangi24 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[75,'Unnamed: 2'])
    column_yangi25 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[18,'Unnamed: 18'])
    column_yangi26 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[5,'Unnamed: 18'])
    column_yangi27 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[11,'Unnamed: 18'])
    column_yangi28 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[14,'Unnamed: 18'])
    column_yangi29 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[9,'Unnamed: 18'])
    column_yangi30 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[11,'Unnamed: 18'])
    column_yangi31 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[14,'Unnamed: 18'])
    column_yangi32 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[18,'Unnamed: 19'])
    column_yangi33 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[18,'Unnamed: 20'])
    column_yangi35 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[18,'Unnamed: 14'])
    column_yangi36 = float(pd.read_excel(excel_file_yangi ,sheet_name=s18).at[18,'Unnamed: 16'])
    column_yangi34 = column_yangi25 - column_yangi35 - column_yangi36
    column_yangi40 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[13,'Unnamed: 2'])
    column_yangi41 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[13,'Unnamed: 3'])
    column_yangi42 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[13,'3-жадвал'])
    column_yangi43 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[5,'Unnamed: 2'])
    column_yangi44 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[10,'Unnamed: 2'])
    column_yangi45 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[12,'Unnamed: 2'])
    column_yangi46 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[43,'Unnamed: 2'])
    column_yangi47 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[43,'Unnamed: 3'])
    column_yangi48 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[43,'3-жадвал'])
    column_yangi37 = column_yangi40 + column_yangi46
    column_yangi38 = column_yangi41 + column_yangi47
    column_yangi39 = column_yangi42 + column_yangi48
    column_yangi49 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[35,'Unnamed: 2'])
    column_yangi50 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[39,'Unnamed: 2'])
    column_yangi51 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[41,'Unnamed: 2'])
    column_yangi52 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[42,'Unnamed: 2'])
    column_yangi56 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[25,'Unnamed: 2'])
    column_yangi57 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[25,'Unnamed: 3'])
    column_yangi58 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[25,'3-жадвал'])
    column_yangi59 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[19,'Unnamed: 2'])
    column_yangi60 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[19,'Unnamed: 2'])
    column_yangi61 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[24,'Unnamed: 2'])
    column_yangi62 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[51,'Unnamed: 2'])
    column_yangi63 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[51,'Unnamed: 3'])
    column_yangi64 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[51,'3-жадвал'])
    column_yangi65 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[67,'Unnamed: 2'])
    column_yangi66 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[56,'Unnamed: 2'])
    column_yangi67 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[76,'Unnamed: 2'])
    column_yangi53 = column_yangi37 - column_yangi67
    column_yangi54 = column_yangi57 + column_yangi63 + column_yangi65
    column_yangi55 = column_yangi58 + column_yangi64
    column_yangi68 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[69,'Unnamed: 2'])
    column_yangi70 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[28,'Unnamed: 2'])
    column_yangi71 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[31,'Unnamed: 2'])
    column_yangi69 = column_yangi70 + column_yangi71
    column_yangi72 = float(pd.read_excel(excel_file_yangi ,sheet_name=s22).at[5,'22-жадвал'])
    column_yangi73 = float(pd.read_excel(excel_file_yangi ,sheet_name=s22).at[7,'22-жадвал'])
    column_yangi74 = float(pd.read_excel(excel_file_yangi ,sheet_name=s22).at[11,'22-жадвал'])
    column_yangi75 = float(pd.read_excel(excel_file_yangi ,sheet_name=s22).at[17,'22-жадвал'])
    column_yangi76 = float(pd.read_excel(excel_file_yangi ,sheet_name=s22).at[20,'22-жадвал'])
    column_yangi77 = 0
    if s23 is not None:
        try:
            column_yangi77 = float(pd.read_excel(excel_file_yangi ,sheet_name=s23).at[3,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_yangi78 = 0
    if s23 is not None:
        try:
            column_yangi78 = float(pd.read_excel(excel_file_yangi ,sheet_name=s23).at[6,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_yangi79 = 0
    if s23 is not None:
        try:
            column_yangi79 = float(pd.read_excel(excel_file_yangi ,sheet_name=s23).at[9,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_yangi83 = 0
    if s23 is not None:
        try:
            column_yangi83 = float(pd.read_excel(excel_file_yangi ,sheet_name=s23).at[10,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_yangi84 = 0
    if s23 is not None:
        try:
            column_yangi84 = float(pd.read_excel(excel_file_yangi ,sheet_name=s23).at[12,'23-жадвал'])
        except ValueError:
            print("Ошибка: Невозможно преобразовать значение в число.")
    column_yangi85 = column_yangi83 - column_yangi84
    column_yangi89 = column_yangi9 + column_yangi6/2 + column_yangi7 + column_yangi8
    column_yangi90 = column_yangi25 + column_yangi19 + column_yangi20/2 + column_yangi21 + column_yangi22 + column_yangi24
    column_yangi91 = float(pd.read_excel(excel_file_yangi ,sheet_name=s1).at[37,'Unnamed: 2'])
    column_yangi92 = float(pd.read_excel(excel_file_yangi ,sheet_name=s3).at[41,'Unnamed: 2'])
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Хамкор)
excel_file_hamkor = os.path.join(folder, 'hamkorbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_hamkor, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_hamkor, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_hamkor, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_hamkor, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_hamkor, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_hamkor, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_hamkor1 = period
column_hamkor2 = "HAMKOR"
column_hamkor3 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[45,'Unnamed: 2'])
column_hamkor4 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[45,'Unnamed: 3'])
column_hamkor5 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[45,'1-жадвал'])
column_hamkor6 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[6,'Unnamed: 2'])
column_hamkor7 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[9,'Unnamed: 2'])
column_hamkor8 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[16,'Unnamed: 2'])
column_hamkor9 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[23,'Unnamed: 2'])
column_hamkor10 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[23,'Unnamed: 3'])
column_hamkor11 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[23,'1-жадвал'])
column_hamkor12 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[24,'Unnamed: 2'])
column_hamkor14 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s17).at[28,'Unnamed: 14'])
column_hamkor15 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s17).at[28,'Unnamed: 16'])
column_hamkor13 = column_hamkor9 - column_hamkor14 - column_hamkor15
column_hamkor16 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[61,'Unnamed: 2'])
column_hamkor17 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[61,'Unnamed: 3'])
column_hamkor18 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[61,'1-жадвал'])
column_hamkor19 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[50,'Unnamed: 2'])
column_hamkor20 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[51,'Unnamed: 2'])
column_hamkor21 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[53,'Unnamed: 2'])
column_hamkor22 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[54,'Unnamed: 2'])
column_hamkor23 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[55,'Unnamed: 2'])
column_hamkor24 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[75,'Unnamed: 2'])
column_hamkor25 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[18,'Unnamed: 18'])
column_hamkor26 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[5,'Unnamed: 18'])
column_hamkor27 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[11,'Unnamed: 18'])
column_hamkor28 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[14,'Unnamed: 18'])
column_hamkor29 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[9,'Unnamed: 18'])
column_hamkor30 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[11,'Unnamed: 18'])
column_hamkor31 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[14,'Unnamed: 18'])
column_hamkor32 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[18,'Unnamed: 19'])
column_hamkor33 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[18,'Unnamed: 20'])
column_hamkor35 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[18,'Unnamed: 14'])
column_hamkor36 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s18).at[18,'Unnamed: 16'])
column_hamkor34 = column_hamkor25 - column_hamkor35 - column_hamkor36
column_hamkor40 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[13,'Unnamed: 2'])
column_hamkor41 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[13,'Unnamed: 3'])
column_hamkor42 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[13,'3-жадвал'])
column_hamkor43 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[5,'Unnamed: 2'])
column_hamkor44 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[10,'Unnamed: 2'])
column_hamkor45 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[12,'Unnamed: 2'])
column_hamkor46 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[43,'Unnamed: 2'])
column_hamkor47 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[43,'Unnamed: 3'])
column_hamkor48 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[43,'3-жадвал'])
column_hamkor37 = column_hamkor40 + column_hamkor46
column_hamkor38 = column_hamkor41 + column_hamkor47
column_hamkor39 = column_hamkor42 + column_hamkor48
column_hamkor49 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[35,'Unnamed: 2'])
column_hamkor50 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[39,'Unnamed: 2'])
column_hamkor51 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[41,'Unnamed: 2'])
column_hamkor52 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[42,'Unnamed: 2'])
column_hamkor56 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[25,'Unnamed: 2'])
column_hamkor57 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[25,'Unnamed: 3'])
column_hamkor58 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[25,'3-жадвал'])
column_hamkor59 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[19,'Unnamed: 2'])
column_hamkor60 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[19,'Unnamed: 2'])
column_hamkor61 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[24,'Unnamed: 2'])
column_hamkor62 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[51,'Unnamed: 2'])
column_hamkor63 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[51,'Unnamed: 3'])
column_hamkor64 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[51,'3-жадвал'])
column_hamkor65 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[67,'Unnamed: 2'])
column_hamkor66 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[56,'Unnamed: 2'])
column_hamkor67 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[76,'Unnamed: 2'])
column_hamkor53 = column_hamkor37 - column_hamkor67
column_hamkor54 = column_hamkor57 + column_hamkor63 + column_hamkor65
column_hamkor55 = column_hamkor58 + column_hamkor64
column_hamkor68 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[69,'Unnamed: 2'])
column_hamkor70 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[28,'Unnamed: 2'])
column_hamkor71 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[31,'Unnamed: 2'])
column_hamkor69 = column_hamkor70 + column_hamkor71
column_hamkor72 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s22).at[5,'22-жадвал'])
column_hamkor73 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s22).at[7,'22-жадвал'])
column_hamkor74 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s22).at[11,'22-жадвал'])
column_hamkor75 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s22).at[17,'22-жадвал'])
column_hamkor76 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s22).at[20,'22-жадвал'])
column_hamkor77 = 0
if s23 is not None:
    try:
        column_hamkor77 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_hamkor78 = 0
if s23 is not None:
    try:
        column_hamkor78 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_hamkor79 = 0
if s23 is not None:
    try:
        column_hamkor79 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_hamkor83 = 0
if s23 is not None:
    try:
        column_hamkor83 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_hamkor84 = 0
if s23 is not None:
    try:
        column_hamkor84 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_hamkor85 = column_hamkor83 - column_hamkor84
column_hamkor89 = column_hamkor9 + column_hamkor6/2 + column_hamkor7 + column_hamkor8
column_hamkor90 = column_hamkor25 + column_hamkor19 + column_hamkor20/2 + column_hamkor21 + column_hamkor22 + column_hamkor24
column_hamkor91 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s1).at[37,'Unnamed: 2'])
column_hamkor92 = float(pd.read_excel(excel_file_hamkor ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы KDB)
excel_file_kdb = os.path.join(folder, 'kdb.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_kdb, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_kdb, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_kdb, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_kdb, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_kdb, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_kdb, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_kdb1 = period
column_kdb2 = "QDB"
column_kdb3 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[45,'Unnamed: 2'])
column_kdb4 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[45,'Unnamed: 3'])
column_kdb5 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[45,'1-жадвал'])
column_kdb6 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[6,'Unnamed: 2'])
column_kdb7 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[9,'Unnamed: 2'])
column_kdb8 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[16,'Unnamed: 2'])
column_kdb9 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[23,'Unnamed: 2'])
column_kdb10 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[23,'Unnamed: 3'])
column_kdb11 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[23,'1-жадвал'])
column_kdb12 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[24,'Unnamed: 2'])
column_kdb14 = float(pd.read_excel(excel_file_kdb ,sheet_name=s17).at[28,'Unnamed: 14'])
column_kdb15 = float(pd.read_excel(excel_file_kdb ,sheet_name=s17).at[28,'Unnamed: 16'])
column_kdb13 = column_kdb9 - column_kdb14 - column_kdb15
column_kdb16 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[61,'Unnamed: 2'])
column_kdb17 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[61,'Unnamed: 3'])
column_kdb18 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[61,'1-жадвал'])
column_kdb19 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[50,'Unnamed: 2'])
column_kdb20 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[51,'Unnamed: 2'])
column_kdb21 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[53,'Unnamed: 2'])
column_kdb22 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[54,'Unnamed: 2'])
column_kdb23 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[55,'Unnamed: 2'])
column_kdb24 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[75,'Unnamed: 2'])
column_kdb25 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[18,'Unnamed: 18'])
column_kdb26 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[5,'Unnamed: 18'])
column_kdb27 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[11,'Unnamed: 18'])
column_kdb28 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[14,'Unnamed: 18'])
column_kdb29 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[9,'Unnamed: 18'])
column_kdb30 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[11,'Unnamed: 18'])
column_kdb31 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[14,'Unnamed: 18'])
column_kdb32 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[18,'Unnamed: 19'])
column_kdb33 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[18,'Unnamed: 20'])
column_kdb35 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[18,'Unnamed: 14'])
column_kdb36 = float(pd.read_excel(excel_file_kdb ,sheet_name=s18).at[18,'Unnamed: 16'])
column_kdb34 = column_kdb25 - column_kdb35 - column_kdb36
column_kdb40 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[13,'Unnamed: 2'])
column_kdb41 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[13,'Unnamed: 3'])
column_kdb42 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[13,'3-жадвал'])
column_kdb43 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[5,'Unnamed: 2'])
column_kdb44 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[10,'Unnamed: 2'])
column_kdb45 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[12,'Unnamed: 2'])
column_kdb46 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[43,'Unnamed: 2'])
column_kdb47 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[43,'Unnamed: 3'])
column_kdb48 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[43,'3-жадвал'])
column_kdb37 = column_kdb40 + column_kdb46
column_kdb38 = column_kdb41 + column_kdb47
column_kdb39 = column_kdb42 + column_kdb48
column_kdb49 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[35,'Unnamed: 2'])
column_kdb50 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[39,'Unnamed: 2'])
column_kdb51 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[41,'Unnamed: 2'])
column_kdb52 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[42,'Unnamed: 2'])
column_kdb56 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[25,'Unnamed: 2'])
column_kdb57 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[25,'Unnamed: 3'])
column_kdb58 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[25,'3-жадвал'])
column_kdb59 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[19,'Unnamed: 2'])
column_kdb60 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[19,'Unnamed: 2'])
column_kdb61 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[24,'Unnamed: 2'])
column_kdb62 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[51,'Unnamed: 2'])
column_kdb63 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[51,'Unnamed: 3'])
column_kdb64 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[51,'3-жадвал'])
column_kdb65 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[67,'Unnamed: 2'])
column_kdb66 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[56,'Unnamed: 2'])
column_kdb67 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[76,'Unnamed: 2'])
column_kdb53 = column_kdb37 - column_kdb67
column_kdb54 = column_kdb57 + column_kdb63 + column_kdb65
column_kdb55 = column_kdb58 + column_kdb64
column_kdb68 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[69,'Unnamed: 2'])
column_kdb70 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[28,'Unnamed: 2'])
column_kdb71 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[31,'Unnamed: 2'])
column_kdb69 = column_kdb70 + column_kdb71
column_kdb72 = float(pd.read_excel(excel_file_kdb ,sheet_name=s22).at[5,'22-жадвал'])
column_kdb73 = float(pd.read_excel(excel_file_kdb ,sheet_name=s22).at[7,'22-жадвал'])
column_kdb74 = float(pd.read_excel(excel_file_kdb ,sheet_name=s22).at[11,'22-жадвал'])
column_kdb75 = float(pd.read_excel(excel_file_kdb ,sheet_name=s22).at[17,'22-жадвал'])
column_kdb76 = float(pd.read_excel(excel_file_kdb ,sheet_name=s22).at[20,'22-жадвал'])
column_kdb77 = 0
if s23 is not None:
    try:
        column_kdb77 = float(pd.read_excel(excel_file_kdb ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_kdb78 = 0
if s23 is not None:
    try:
        column_kdb78 = float(pd.read_excel(excel_file_kdb ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_kdb79 = 0
if s23 is not None:
    try:
        column_kdb79 = float(pd.read_excel(excel_file_kdb ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_kdb83 = 0
if s23 is not None:
    try:
        column_kdb83 = float(pd.read_excel(excel_file_kdb ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_kdb84 = 0
if s23 is not None:
    try:
        column_kdb84 = float(pd.read_excel(excel_file_kdb ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_kdb85 = column_kdb83 - column_kdb84
column_kdb89 = column_kdb9 + column_kdb6/2 + column_kdb7 + column_kdb8
column_kdb90 = column_kdb25 + column_kdb19 + column_kdb20/2 + column_kdb21 + column_kdb22 + column_kdb24
column_kdb91 = float(pd.read_excel(excel_file_kdb ,sheet_name=s1).at[37,'Unnamed: 2'])
column_kdb92 = float(pd.read_excel(excel_file_kdb ,sheet_name=s3).at[41,'Unnamed: 2'])
# Прочитайте данные из Excel-файла для каждого поля (для таблицы Garant)
excel_file_garant = os.path.join(folder, 'savdogarbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_garant, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_garant, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_garant, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_garant, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_garant, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_garant, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_garant1 = period
column_garant2 = "GARANT"
column_garant3 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[45,'Unnamed: 2'])
column_garant4 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[45,'Unnamed: 3'])
column_garant5 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[45,'1-жадвал'])
column_garant6 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[6,'Unnamed: 2'])
column_garant7 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[9,'Unnamed: 2'])
column_garant8 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[16,'Unnamed: 2'])
column_garant9 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[23,'Unnamed: 2'])
column_garant10 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[23,'Unnamed: 3'])
column_garant11 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[23,'1-жадвал'])
column_garant12 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[24,'Unnamed: 2'])
column_garant14 = float(pd.read_excel(excel_file_garant ,sheet_name=s17).at[28,'Unnamed: 14'])
column_garant15 = float(pd.read_excel(excel_file_garant ,sheet_name=s17).at[28,'Unnamed: 16'])
column_garant13 = column_garant9 - column_garant14 - column_garant15
column_garant16 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[61,'Unnamed: 2'])
column_garant17 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[61,'Unnamed: 3'])
column_garant18 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[61,'1-жадвал'])
column_garant19 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[50,'Unnamed: 2'])
column_garant20 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[51,'Unnamed: 2'])
column_garant21 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[53,'Unnamed: 2'])
column_garant22 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[54,'Unnamed: 2'])
column_garant23 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[55,'Unnamed: 2'])
column_garant24 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[75,'Unnamed: 2'])
column_garant25 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[18,'Unnamed: 18'])
column_garant26 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[5,'Unnamed: 18'])
column_garant27 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[11,'Unnamed: 18'])
column_garant28 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[14,'Unnamed: 18'])
column_garant29 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[9,'Unnamed: 18'])
column_garant30 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[11,'Unnamed: 18'])
column_garant31 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[14,'Unnamed: 18'])
column_garant32 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[18,'Unnamed: 19'])
column_garant33 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[18,'Unnamed: 20'])
column_garant35 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[18,'Unnamed: 14'])
column_garant36 = float(pd.read_excel(excel_file_garant ,sheet_name=s18).at[18,'Unnamed: 16'])
column_garant34 = column_garant25 - column_garant35 - column_garant36
column_garant40 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[13,'Unnamed: 2'])
column_garant41 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[13,'Unnamed: 3'])
column_garant42 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[13,'3-жадвал'])
column_garant43 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[5,'Unnamed: 2'])
column_garant44 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[10,'Unnamed: 2'])
column_garant45 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[12,'Unnamed: 2'])
column_garant46 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[43,'Unnamed: 2'])
column_garant47 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[43,'Unnamed: 3'])
column_garant48 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[43,'3-жадвал'])
column_garant37 = column_garant40 + column_garant46
column_garant38 = column_garant41 + column_garant47
column_garant39 = column_garant42 + column_garant48
column_garant49 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[35,'Unnamed: 2'])
column_garant50 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[39,'Unnamed: 2'])
column_garant51 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[41,'Unnamed: 2'])
column_garant52 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[42,'Unnamed: 2'])
column_garant56 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[25,'Unnamed: 2'])
column_garant57 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[25,'Unnamed: 3'])
column_garant58 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[25,'3-жадвал'])
column_garant59 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[19,'Unnamed: 2'])
column_garant60 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[19,'Unnamed: 2'])
column_garant61 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[24,'Unnamed: 2'])
column_garant62 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[51,'Unnamed: 2'])
column_garant63 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[51,'Unnamed: 3'])
column_garant64 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[51,'3-жадвал'])
column_garant65 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[67,'Unnamed: 2'])
column_garant66 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[56,'Unnamed: 2'])
column_garant67 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[76,'Unnamed: 2'])
column_garant53 = column_garant37 - column_garant67
column_garant54 = column_garant57 + column_garant63 + column_garant65
column_garant55 = column_garant58 + column_garant64
column_garant68 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[69,'Unnamed: 2'])
column_garant70 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[28,'Unnamed: 2'])
column_garant71 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[31,'Unnamed: 2'])
column_garant69 = column_garant70 + column_garant71
column_garant72 = float(pd.read_excel(excel_file_garant ,sheet_name=s22).at[5,'22-жадвал'])
column_garant73 = float(pd.read_excel(excel_file_garant ,sheet_name=s22).at[7,'22-жадвал'])
column_garant74 = float(pd.read_excel(excel_file_garant ,sheet_name=s22).at[11,'22-жадвал'])
column_garant75 = float(pd.read_excel(excel_file_garant ,sheet_name=s22).at[17,'22-жадвал'])
column_garant76 = float(pd.read_excel(excel_file_garant ,sheet_name=s22).at[20,'22-жадвал'])
column_garant77 = 0
if s23 is not None:
    try:
        column_garant77 = float(pd.read_excel(excel_file_garant ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_garant78 = 0
if s23 is not None:
    try:
        column_garant78 = float(pd.read_excel(excel_file_garant ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_garant79 = 0
if s23 is not None:
    try:
        column_garant79 = float(pd.read_excel(excel_file_garant ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_garant83 = 0
if s23 is not None:
    try:
        column_garant83 = float(pd.read_excel(excel_file_garant ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_garant84 = 0
if s23 is not None:
    try:
        column_garant84 = float(pd.read_excel(excel_file_garant ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_garant85 = column_garant83 - column_garant84
column_garant89 = column_garant9 + column_garant6/2 + column_garant7 + column_garant8
column_garant90 = column_garant25 + column_garant19 + column_garant20/2 + column_garant21 + column_garant22 + column_garant24
column_garant91 = float(pd.read_excel(excel_file_garant ,sheet_name=s1).at[37,'Unnamed: 2'])
column_garant92 = float(pd.read_excel(excel_file_garant ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Zirat)
excel_file_zirat = os.path.join(folder, 'ziraatbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_zirat, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_zirat, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_zirat, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_zirat, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_zirat, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_zirat, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_zirat1 = period
column_zirat2 = "ZIRAAT"
column_zirat3 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[45,'Unnamed: 2'])
column_zirat4 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[45,'Unnamed: 3'])
column_zirat5 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[45,'1-жадвал'])
column_zirat6 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[6,'Unnamed: 2'])
column_zirat7 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[9,'Unnamed: 2'])
column_zirat8 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[16,'Unnamed: 2'])
column_zirat9 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[23,'Unnamed: 2'])
column_zirat10 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[23,'Unnamed: 3'])
column_zirat11 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[23,'1-жадвал'])
column_zirat12 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[24,'Unnamed: 2'])
column_zirat14 = float(pd.read_excel(excel_file_zirat ,sheet_name=s17).at[28,'Unnamed: 14'])
column_zirat15 = float(pd.read_excel(excel_file_zirat ,sheet_name=s17).at[28,'Unnamed: 16'])
column_zirat13 = column_zirat9 - column_zirat14 - column_zirat15
column_zirat16 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[61,'Unnamed: 2'])
column_zirat17 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[61,'Unnamed: 3'])
column_zirat18 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[61,'1-жадвал'])
column_zirat19 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[50,'Unnamed: 2'])
column_zirat20 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[51,'Unnamed: 2'])
column_zirat21 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[53,'Unnamed: 2'])
column_zirat22 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[54,'Unnamed: 2'])
column_zirat23 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[55,'Unnamed: 2'])
column_zirat24 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[75,'Unnamed: 2'])
column_zirat25 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[18,'Unnamed: 18'])
column_zirat26 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[5,'Unnamed: 18'])
column_zirat27 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[11,'Unnamed: 18'])
column_zirat28 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[14,'Unnamed: 18'])
column_zirat29 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[9,'Unnamed: 18'])
column_zirat30 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[11,'Unnamed: 18'])
column_zirat31 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[14,'Unnamed: 18'])
column_zirat32 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[18,'Unnamed: 19'])
column_zirat33 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[18,'Unnamed: 20'])
column_zirat35 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[18,'Unnamed: 14'])
column_zirat36 = float(pd.read_excel(excel_file_zirat ,sheet_name=s18).at[18,'Unnamed: 16'])
column_zirat34 = column_zirat25 - column_zirat35 - column_zirat36
column_zirat40 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[13,'Unnamed: 2'])
column_zirat41 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[13,'Unnamed: 3'])
column_zirat42 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[13,'3-жадвал'])
column_zirat43 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[5,'Unnamed: 2'])
column_zirat44 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[10,'Unnamed: 2'])
column_zirat45 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[12,'Unnamed: 2'])
column_zirat46 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[43,'Unnamed: 2'])
column_zirat47 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[43,'Unnamed: 3'])
column_zirat48 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[43,'3-жадвал'])
column_zirat37 = column_zirat40 + column_zirat46
column_zirat38 = column_zirat41 + column_zirat47
column_zirat39 = column_zirat42 + column_zirat48
column_zirat49 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[35,'Unnamed: 2'])
column_zirat50 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[39,'Unnamed: 2'])
column_zirat51 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[41,'Unnamed: 2'])
column_zirat52 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[42,'Unnamed: 2'])
column_zirat56 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[25,'Unnamed: 2'])
column_zirat57 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[25,'Unnamed: 3'])
column_zirat58 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[25,'3-жадвал'])
column_zirat59 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[19,'Unnamed: 2'])
column_zirat60 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[19,'Unnamed: 2'])
column_zirat61 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[24,'Unnamed: 2'])
column_zirat62 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[51,'Unnamed: 2'])
column_zirat63 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[51,'Unnamed: 3'])
column_zirat64 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[51,'3-жадвал'])
column_zirat65 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[67,'Unnamed: 2'])
column_zirat66 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[56,'Unnamed: 2'])
column_zirat67 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[76,'Unnamed: 2'])
column_zirat53 = column_zirat37 - column_zirat67
column_zirat54 = column_zirat57 + column_zirat63 + column_zirat65
column_zirat55 = column_zirat58 + column_zirat64
column_zirat68 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[69,'Unnamed: 2'])
column_zirat70 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[28,'Unnamed: 2'])
column_zirat71 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[31,'Unnamed: 2'])
column_zirat69 = column_zirat70 + column_zirat71
column_zirat72 = float(pd.read_excel(excel_file_zirat ,sheet_name=s22).at[5,'22-жадвал'])
column_zirat73 = float(pd.read_excel(excel_file_zirat ,sheet_name=s22).at[7,'22-жадвал'])
column_zirat74 = float(pd.read_excel(excel_file_zirat ,sheet_name=s22).at[11,'22-жадвал'])
column_zirat75 = float(pd.read_excel(excel_file_zirat ,sheet_name=s22).at[17,'22-жадвал'])
column_zirat76 = float(pd.read_excel(excel_file_zirat ,sheet_name=s22).at[20,'22-жадвал'])
column_zirat77 = 0
if s23 is not None:
    try:
        column_zirat77 = float(pd.read_excel(excel_file_zirat ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_zirat78 = 0
if s23 is not None:
    try:
        column_zirat78 = float(pd.read_excel(excel_file_zirat ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_zirat79 = 0
if s23 is not None:
    try:
        column_zirat79 = float(pd.read_excel(excel_file_zirat ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_zirat83 = 0
if s23 is not None:
    try:
        column_zirat83 = float(pd.read_excel(excel_file_zirat ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_zirat84 = 0
if s23 is not None:
    try:
        column_zirat84 = float(pd.read_excel(excel_file_zirat ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_zirat85 = column_zirat83 - column_zirat84
column_zirat89 = column_zirat9 + column_zirat6/2 + column_zirat7 + column_zirat8
column_zirat90 = column_zirat25 + column_zirat19 + column_zirat20/2 + column_zirat21 + column_zirat22 + column_zirat24
column_zirat91 = float(pd.read_excel(excel_file_zirat ,sheet_name=s1).at[37,'Unnamed: 2'])
column_zirat92 = float(pd.read_excel(excel_file_zirat ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Садерат)
excel_file_saderat = os.path.join(folder, 'saderatbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_saderat, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_saderat, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_saderat, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_saderat, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_saderat, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_saderat, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_saderat1 = period
column_saderat2 = "SADERAT"
column_saderat3 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[45,'Unnamed: 2'])
column_saderat4 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[45,'Unnamed: 3'])
column_saderat5 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[45,'1-жадвал'])
column_saderat6 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[6,'Unnamed: 2'])
column_saderat7 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[9,'Unnamed: 2'])
column_saderat8 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[16,'Unnamed: 2'])
column_saderat9 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[23,'Unnamed: 2'])
column_saderat10 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[23,'Unnamed: 3'])
column_saderat11 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[23,'1-жадвал'])
column_saderat12 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[24,'Unnamed: 2'])
column_saderat14 = float(pd.read_excel(excel_file_saderat ,sheet_name=s17).at[28,'Unnamed: 14'])
column_saderat15 = float(pd.read_excel(excel_file_saderat ,sheet_name=s17).at[28,'Unnamed: 16'])
column_saderat13 = column_saderat9 - column_saderat14 - column_saderat15
column_saderat16 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[61,'Unnamed: 2'])
column_saderat17 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[61,'Unnamed: 3'])
column_saderat18 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[61,'1-жадвал'])
column_saderat19 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[50,'Unnamed: 2'])
column_saderat20 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[51,'Unnamed: 2'])
column_saderat21 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[53,'Unnamed: 2'])
column_saderat22 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[54,'Unnamed: 2'])
column_saderat23 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[55,'Unnamed: 2'])
column_saderat24 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[75,'Unnamed: 2'])
column_saderat25 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[18,'Unnamed: 18'])
column_saderat26 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[5,'Unnamed: 18'])
column_saderat27 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[11,'Unnamed: 18'])
column_saderat28 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[14,'Unnamed: 18'])
column_saderat29 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[9,'Unnamed: 18'])
column_saderat30 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[11,'Unnamed: 18'])
column_saderat31 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[14,'Unnamed: 18'])
column_saderat32 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[18,'Unnamed: 19'])
column_saderat33 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[18,'Unnamed: 20'])
column_saderat35 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[18,'Unnamed: 14'])
column_saderat36 = float(pd.read_excel(excel_file_saderat ,sheet_name=s18).at[18,'Unnamed: 16'])
column_saderat34 = column_saderat25 - column_saderat35 - column_saderat36
column_saderat40 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[13,'Unnamed: 2'])
column_saderat41 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[13,'Unnamed: 3'])
column_saderat42 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[13,'3-жадвал'])
column_saderat43 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[5,'Unnamed: 2'])
column_saderat44 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[10,'Unnamed: 2'])
column_saderat45 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[12,'Unnamed: 2'])
column_saderat46 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[43,'Unnamed: 2'])
column_saderat47 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[43,'Unnamed: 3'])
column_saderat48 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[43,'3-жадвал'])
column_saderat37 = column_saderat40 + column_saderat46
column_saderat38 = column_saderat41 + column_saderat47
column_saderat39 = column_saderat42 + column_saderat48
column_saderat49 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[35,'Unnamed: 2'])
column_saderat50 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[39,'Unnamed: 2'])
column_saderat51 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[41,'Unnamed: 2'])
column_saderat52 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[42,'Unnamed: 2'])
column_saderat56 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[25,'Unnamed: 2'])
column_saderat57 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[25,'Unnamed: 3'])
column_saderat58 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[25,'3-жадвал'])
column_saderat59 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[19,'Unnamed: 2'])
column_saderat60 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[19,'Unnamed: 2'])
column_saderat61 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[24,'Unnamed: 2'])
column_saderat62 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[51,'Unnamed: 2'])
column_saderat63 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[51,'Unnamed: 3'])
column_saderat64 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[51,'3-жадвал'])
column_saderat65 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[67,'Unnamed: 2'])
column_saderat66 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[56,'Unnamed: 2'])
column_saderat67 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[76,'Unnamed: 2'])
column_saderat53 = column_saderat37 - column_saderat67
column_saderat54 = column_saderat57 + column_saderat63 + column_saderat65
column_saderat55 = column_saderat58 + column_saderat64
column_saderat68 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[69,'Unnamed: 2'])
column_saderat70 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[28,'Unnamed: 2'])
column_saderat71 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[31,'Unnamed: 2'])
column_saderat69 = column_saderat70 + column_saderat71
column_saderat72 = float(pd.read_excel(excel_file_saderat ,sheet_name=s22).at[5,'22-жадвал'])
column_saderat73 = float(pd.read_excel(excel_file_saderat ,sheet_name=s22).at[7,'22-жадвал'])
column_saderat74 = float(pd.read_excel(excel_file_saderat ,sheet_name=s22).at[11,'22-жадвал'])
column_saderat75 = float(pd.read_excel(excel_file_saderat ,sheet_name=s22).at[17,'22-жадвал'])
column_saderat76 = float(pd.read_excel(excel_file_saderat ,sheet_name=s22).at[20,'22-жадвал'])
column_saderat77 = 0
if s23 is not None:
    try:
        column_saderat77 = float(pd.read_excel(excel_file_saderat ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_saderat78 = 0
if s23 is not None:
    try:
        column_saderat78 = float(pd.read_excel(excel_file_saderat ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_saderat79 = 0
if s23 is not None:
    try:
        column_saderat79 = float(pd.read_excel(excel_file_saderat ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_saderat83 = 0
if s23 is not None:
    try:
        column_saderat83 = float(pd.read_excel(excel_file_saderat ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_saderat84 = 0
if s23 is not None:
    try:
        column_saderat84 = float(pd.read_excel(excel_file_saderat ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_saderat85 = column_saderat83 - column_saderat84
column_saderat89 = column_saderat9 + column_saderat6/2 + column_saderat7 + column_saderat8
column_saderat90 = column_saderat25 + column_saderat19 + column_saderat20/2 + column_saderat21 + column_saderat22 + column_saderat24
column_saderat91 = float(pd.read_excel(excel_file_saderat ,sheet_name=s1).at[37,'Unnamed: 2'])
column_saderat92 = float(pd.read_excel(excel_file_saderat ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы TBC)
excel_file_tbc = os.path.join(folder, 'tbcbank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_tbc, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_tbc, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_tbc, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_tbc, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_tbc, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_tbc, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_tbc1 = period
column_tbc2 = "TBC"
column_tbc3 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[45,'Unnamed: 2'])
column_tbc4 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[45,'Unnamed: 3'])
column_tbc5 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[45,'1-жадвал'])
column_tbc6 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[6,'Unnamed: 2'])
column_tbc7 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[9,'Unnamed: 2'])
column_tbc8 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[16,'Unnamed: 2'])
column_tbc9 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[23,'Unnamed: 2'])
column_tbc10 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[23,'Unnamed: 3'])
column_tbc11 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[23,'1-жадвал'])
column_tbc12 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[24,'Unnamed: 2'])
column_tbc14 = float(pd.read_excel(excel_file_tbc ,sheet_name=s17).at[28,'Unnamed: 14'])
column_tbc15 = float(pd.read_excel(excel_file_tbc ,sheet_name=s17).at[28,'Unnamed: 16'])
column_tbc13 = column_tbc9 - column_tbc14 - column_tbc15
column_tbc16 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[61,'Unnamed: 2'])
column_tbc17 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[61,'Unnamed: 3'])
column_tbc18 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[61,'1-жадвал'])
column_tbc19 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[50,'Unnamed: 2'])
column_tbc20 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[51,'Unnamed: 2'])
column_tbc21 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[53,'Unnamed: 2'])
column_tbc22 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[54,'Unnamed: 2'])
column_tbc23 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[55,'Unnamed: 2'])
column_tbc24 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[75,'Unnamed: 2'])
column_tbc25 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[18,'Unnamed: 18'])
column_tbc26 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[5,'Unnamed: 18'])
column_tbc27 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[11,'Unnamed: 18'])
column_tbc28 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[14,'Unnamed: 18'])
column_tbc29 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[9,'Unnamed: 18'])
column_tbc30 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[11,'Unnamed: 18'])
column_tbc31 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[14,'Unnamed: 18'])
column_tbc32 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[18,'Unnamed: 19'])
column_tbc33 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[18,'Unnamed: 20'])
column_tbc35 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[18,'Unnamed: 14'])
column_tbc36 = float(pd.read_excel(excel_file_tbc ,sheet_name=s18).at[18,'Unnamed: 16'])
column_tbc34 = column_tbc25 - column_tbc35 - column_tbc36
column_tbc40 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[13,'Unnamed: 2'])
column_tbc41 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[13,'Unnamed: 3'])
column_tbc42 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[13,'3-жадвал'])
column_tbc43 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[5,'Unnamed: 2'])
column_tbc44 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[10,'Unnamed: 2'])
column_tbc45 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[12,'Unnamed: 2'])
column_tbc46 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[43,'Unnamed: 2'])
column_tbc47 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[43,'Unnamed: 3'])
column_tbc48 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[43,'3-жадвал'])
column_tbc37 = column_tbc40 + column_tbc46
column_tbc38 = column_tbc41 + column_tbc47
column_tbc39 = column_tbc42 + column_tbc48
column_tbc49 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[35,'Unnamed: 2'])
column_tbc50 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[39,'Unnamed: 2'])
column_tbc51 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[41,'Unnamed: 2'])
column_tbc52 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[42,'Unnamed: 2'])
column_tbc56 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[25,'Unnamed: 2'])
column_tbc57 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[25,'Unnamed: 3'])
column_tbc58 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[25,'3-жадвал'])
column_tbc59 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[19,'Unnamed: 2'])
column_tbc60 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[19,'Unnamed: 2'])
column_tbc61 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[24,'Unnamed: 2'])
column_tbc62 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[51,'Unnamed: 2'])
column_tbc63 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[51,'Unnamed: 3'])
column_tbc64 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[51,'3-жадвал'])
column_tbc65 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[67,'Unnamed: 2'])
column_tbc66 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[56,'Unnamed: 2'])
column_tbc67 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[76,'Unnamed: 2'])
column_tbc53 = column_tbc37 - column_tbc67
column_tbc54 = column_tbc57 + column_tbc63 + column_tbc65
column_tbc55 = column_tbc58 + column_tbc64
column_tbc68 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[69,'Unnamed: 2'])
column_tbc70 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[28,'Unnamed: 2'])
column_tbc71 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[31,'Unnamed: 2'])
column_tbc69 = column_tbc70 + column_tbc71
column_tbc72 = float(pd.read_excel(excel_file_tbc ,sheet_name=s22).at[5,'22-жадвал'])
column_tbc73 = float(pd.read_excel(excel_file_tbc ,sheet_name=s22).at[7,'22-жадвал'])
column_tbc74 = float(pd.read_excel(excel_file_tbc ,sheet_name=s22).at[11,'22-жадвал'])
column_tbc75 = float(pd.read_excel(excel_file_tbc ,sheet_name=s22).at[17,'22-жадвал'])
column_tbc76 = float(pd.read_excel(excel_file_tbc ,sheet_name=s22).at[20,'22-жадвал'])
column_tbc77 = 0
if s23 is not None:
    try:
        column_tbc77 = float(pd.read_excel(excel_file_tbc ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tbc78 = 0
if s23 is not None:
    try:
        column_tbc78 = float(pd.read_excel(excel_file_tbc ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tbc79 = 0
if s23 is not None:
    try:
        column_tbc79 = float(pd.read_excel(excel_file_tbc ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tbc83 = 0
if s23 is not None:
    try:
        column_tbc83 = float(pd.read_excel(excel_file_tbc ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tbc84 = 0
if s23 is not None:
    try:
        column_tbc84 = float(pd.read_excel(excel_file_tbc ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tbc85 = column_tbc83 - column_tbc84
column_tbc89 = column_tbc9 + column_tbc6/2 + column_tbc7 + column_tbc8
column_tbc90 = column_tbc25 + column_tbc19 + column_tbc20/2 + column_tbc21 + column_tbc22 + column_tbc24
column_tbc91 = float(pd.read_excel(excel_file_tbc ,sheet_name=s1).at[37,'Unnamed: 2'])
column_tbc92 = float(pd.read_excel(excel_file_tbc ,sheet_name=s3).at[41,'Unnamed: 2'])
    # Прочитайте данные из Excel-файла для каждого поля (для таблицы Тенге)
excel_file_tenge = os.path.join(folder, 'tengebank.xlsx')
try:
    s1 = find_sheet_by_keyword(excel_file_tenge, 'БАНК БАЛАНСИ ТАРКИБИ')
    s1 = str(s1)
    s3 = find_sheet_by_keyword(excel_file_tenge, 'МОЛИЯВИЙ НАТИЖАЛАР ТЎҒРИСИДА ҲИСОБОТ')
    s3 = str(s3)
    s17 = find_sheet_by_keyword(excel_file_tenge, 'БАНК АКТИВЛАРИ ТАҲЛИЛИ')
    s17 = str(s17)
    s18 = find_sheet_by_keyword(excel_file_tenge, 'БАНК МАЖБУРИЯТЛАРИ ТАҲЛИЛИ')
    s18 = str(s18)
    s22 = find_sheet_by_keyword(excel_file_tenge, 'ПРУДЕНЦИАЛ МЕЪЁРЛАР ҲИСОБ-КИТОБИ')
    s22 = str(s22)
    s23 = find_sheet_by_keyword(excel_file_tenge, 'БОШҚА МАЪЛУМОТЛАР')
    s23 = str(s23)
except FileNotFoundError:
    print("Файл не найден по указанному пути.")
# Получение значений
column_tenge1 = period
column_tenge2 = "TENGE"
column_tenge3 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[45,'Unnamed: 2'])
column_tenge4 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[45,'Unnamed: 3'])
column_tenge5 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[45,'1-жадвал'])
column_tenge6 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[6,'Unnamed: 2'])
column_tenge7 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[9,'Unnamed: 2'])
column_tenge8 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[16,'Unnamed: 2'])
column_tenge9 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[23,'Unnamed: 2'])
column_tenge10 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[23,'Unnamed: 3'])
column_tenge11 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[23,'1-жадвал'])
column_tenge12 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[24,'Unnamed: 2'])
column_tenge14 = float(pd.read_excel(excel_file_tenge ,sheet_name=s17).at[28,'Unnamed: 14'])
column_tenge15 = float(pd.read_excel(excel_file_tenge ,sheet_name=s17).at[28,'Unnamed: 16'])
column_tenge13 = column_tenge9 - column_tenge14 - column_tenge15
column_tenge16 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[61,'Unnamed: 2'])
column_tenge17 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[61,'Unnamed: 3'])
column_tenge18 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[61,'1-жадвал'])
column_tenge19 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[50,'Unnamed: 2'])
column_tenge20 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[51,'Unnamed: 2'])
column_tenge21 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[53,'Unnamed: 2'])
column_tenge22 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[54,'Unnamed: 2'])
column_tenge23 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[55,'Unnamed: 2'])
column_tenge24 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[75,'Unnamed: 2'])
column_tenge25 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[18,'Unnamed: 18'])
column_tenge26 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[5,'Unnamed: 18'])
column_tenge27 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[11,'Unnamed: 18'])
column_tenge28 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[14,'Unnamed: 18'])
column_tenge29 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[9,'Unnamed: 18'])
column_tenge30 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[10,'Unnamed: 18']) - float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[11,'Unnamed: 18'])
column_tenge31 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[13,'Unnamed: 18']) - float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[14,'Unnamed: 18'])
column_tenge32 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[18,'Unnamed: 19'])
column_tenge33 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[18,'Unnamed: 20'])
column_tenge35 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[18,'Unnamed: 14'])
column_tenge36 = float(pd.read_excel(excel_file_tenge ,sheet_name=s18).at[18,'Unnamed: 16'])
column_tenge34 = column_tenge25 - column_tenge35 - column_tenge36
column_tenge40 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[13,'Unnamed: 2'])
column_tenge41 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[13,'Unnamed: 3'])
column_tenge42 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[13,'3-жадвал'])
column_tenge43 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[5,'Unnamed: 2'])
column_tenge44 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[10,'Unnamed: 2'])
column_tenge45 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[12,'Unnamed: 2'])
column_tenge46 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[43,'Unnamed: 2'])
column_tenge47 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[43,'Unnamed: 3'])
column_tenge48 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[43,'3-жадвал'])
column_tenge37 = column_tenge40 + column_tenge46
column_tenge38 = column_tenge41 + column_tenge47
column_tenge39 = column_tenge42 + column_tenge48
column_tenge49 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[35,'Unnamed: 2'])
column_tenge50 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[39,'Unnamed: 2'])
column_tenge51 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[41,'Unnamed: 2'])
column_tenge52 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[42,'Unnamed: 2'])
column_tenge56 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[25,'Unnamed: 2'])
column_tenge57 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[25,'Unnamed: 3'])
column_tenge58 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[25,'3-жадвал'])
column_tenge59 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[19,'Unnamed: 2'])
column_tenge60 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[19,'Unnamed: 2'])
column_tenge61 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[24,'Unnamed: 2'])
column_tenge62 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[51,'Unnamed: 2'])
column_tenge63 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[51,'Unnamed: 3'])
column_tenge64 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[51,'3-жадвал'])
column_tenge65 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[67,'Unnamed: 2'])
column_tenge66 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[56,'Unnamed: 2'])
column_tenge67 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[76,'Unnamed: 2'])
column_tenge53 = column_tenge37 - column_tenge67
column_tenge54 = column_tenge57 + column_tenge63 + column_tenge65
column_tenge55 = column_tenge58 + column_tenge64
column_tenge68 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[69,'Unnamed: 2'])
column_tenge70 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[28,'Unnamed: 2'])
column_tenge71 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[30,'Unnamed: 2']) + float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[31,'Unnamed: 2'])
column_tenge69 = column_tenge70 + column_tenge71
column_tenge72 = float(pd.read_excel(excel_file_tenge ,sheet_name=s22).at[5,'22-жадвал'])
column_tenge73 = float(pd.read_excel(excel_file_tenge ,sheet_name=s22).at[7,'22-жадвал'])
column_tenge74 = float(pd.read_excel(excel_file_tenge ,sheet_name=s22).at[11,'22-жадвал'])
column_tenge75 = float(pd.read_excel(excel_file_tenge ,sheet_name=s22).at[17,'22-жадвал'])
column_tenge76 = float(pd.read_excel(excel_file_tenge ,sheet_name=s22).at[20,'22-жадвал'])
column_tenge77 = 0
if s23 is not None:
    try:
        column_tenge77 = float(pd.read_excel(excel_file_tenge ,sheet_name=s23).at[3,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tenge78 = 0
if s23 is not None:
    try:
        column_tenge78 = float(pd.read_excel(excel_file_tenge ,sheet_name=s23).at[6,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tenge79 = 0
if s23 is not None:
    try:
        column_tenge79 = float(pd.read_excel(excel_file_tenge ,sheet_name=s23).at[9,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tenge83 = 0
if s23 is not None:
    try:
        column_tenge83 = float(pd.read_excel(excel_file_tenge ,sheet_name=s23).at[10,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tenge84 = 0
if s23 is not None:
    try:
        column_tenge84 = float(pd.read_excel(excel_file_tenge ,sheet_name=s23).at[12,'23-жадвал'])
    except ValueError:
        print("Ошибка: Невозможно преобразовать значение в число.")
column_tenge85 = column_tenge83 - column_tenge84
column_tenge89 = column_tenge9 + column_tenge6/2 + column_tenge7 + column_tenge8
column_tenge90 = column_tenge25 + column_tenge19 + column_tenge20/2 + column_tenge21 + column_tenge22 + column_tenge24
column_tenge91 = float(pd.read_excel(excel_file_tenge ,sheet_name=s1).at[37,'Unnamed: 2'])
column_tenge92 = float(pd.read_excel(excel_file_tenge ,sheet_name=s3).at[41,'Unnamed: 2'])

# Получение значений nbu для Banks
month = period.month

column_banks_nbu1 = column_nbu2
column_banks_nbu2 = period
column_banks_nbu3 = round(np.float64(column_nbu9/column_nbu3 * 100), 1)
column_banks_nbu4 = round(np.float64(column_nbu25/column_nbu16 * 100), 1)
column_banks_nbu5 = round(np.float64(column_nbu9/column_nbu25 * 100), 1)
column_banks_nbu6 = round(np.float64(column_nbu65/column_nbu53 * 100), 1)
column_banks_nbu7 = round(np.float64(column_nbu67/column_nbu37 * 100), 1)
column_banks_nbu8 = round(np.float64(((column_nbu40-column_nbu56)*(days/365))/column_nbu89 * 100), 1)
column_banks_nbu9 = round(np.float64(column_nbu65/(column_nbu40-column_nbu56+column_nbu46-column_nbu62-column_nbu92) * 100), 1)
column_banks_nbu10 = round(np.float64((column_nbu44*(days/365))/column_nbu9 * 100), 1)
column_banks_nbu11 = round(np.float64((column_nbu60*(days/365))/column_nbu25 * 100), 1)
column_banks_nbu12 = round(np.float64((column_nbu67*(days/365))/column_nbu3 * 100), 1)
column_banks_nbu13 = round(np.float64((column_nbu67*(days/365))/column_nbu24 * 100), 1)
if column_nbu77 != 0:
    column_banks_nbu14 = round(np.float64((((column_nbu66/1000)*0.87)*(days/365))/(column_nbu77*month)), 1)
else:
    column_banks_nbu14 = 0
column_banks_nbu15 = round(np.float64(column_nbu9/(column_nbu19+column_nbu20+column_nbu21+column_nbu22+column_nbu23+column_nbu25) * 100), 1)
column_banks_nbu16 = round(np.float64((column_nbu68*(days/365))/column_nbu3 * 100), 1)
if column_nbu77 != 0:
    column_banks_nbu17 = round(np.float64(((column_nbu67/1000)*(days/365))/(column_nbu77*month)), 1)
else:
    column_banks_nbu17 = 0
column_banks_nbu18 = round(np.float64((column_nbu68*(days/365))/column_nbu24 * 100), 1)
column_banks_nbu19 = round(np.float64((column_nbu12*(days/365))/column_nbu9 * 100), 1)
column_banks_nbu20 = round(np.float64(column_nbu67/1000000), 1)
column_banks_nbu21 = round(np.float64((column_nbu67+column_nbu69-column_nbu92)/1000000), 1)
# Получение значений aab для Banks
column_banks_aab1 = column_aab2
column_banks_aab2 = period
column_banks_aab3 = round(np.float64(column_aab9/column_aab3 * 100), 1)
column_banks_aab4 = round(np.float64(column_aab25/column_aab16 * 100), 1)
column_banks_aab5 = round(np.float64(column_aab9/column_aab25 * 100), 1)
column_banks_aab6 = round(np.float64(column_aab65/column_aab53 * 100), 1)
column_banks_aab7 = round(np.float64(column_aab67/column_aab37 * 100), 1)
column_banks_aab8 = round(np.float64(((column_aab40-column_aab56)*(days/365))/column_aab89 * 100), 1)
column_banks_aab9 = round(np.float64(column_aab65/(column_aab40-column_aab56+column_aab46-column_aab62-column_aab92) * 100), 1)
column_banks_aab10 = round(np.float64((column_aab44*(days/365))/column_aab9 * 100), 1)
column_banks_aab11 = round(np.float64((column_aab60*(days/365))/column_aab25 * 100), 1)
column_banks_aab12 = round(np.float64((column_aab67*(days/365))/column_aab3 * 100), 1)
column_banks_aab13 = round(np.float64((column_aab67*(days/365))/column_aab24 * 100), 1)
if column_aab77 != 0:
    column_banks_aab14 = round(np.float64((((column_aab66/1000)*0.87)*(days/365))/(column_aab77*month)), 1)
else:
    column_banks_aab14 = 0
column_banks_aab15 = round(np.float64(column_aab9/(column_aab19+column_aab20+column_aab21+column_aab22+column_aab23+column_aab25) * 100), 1)
column_banks_aab16 = round(np.float64((column_aab68*(days/365))/column_aab3 * 100), 1)
if column_aab77 != 0:
    column_banks_aab17 = round(np.float64(((column_aab67/1000)*(days/365))/(column_aab77*month)), 1)
else:
    column_banks_aab17 = 0
column_banks_aab18 = round(np.float64((column_aab68*(days/365))/column_aab24 * 100), 1)
column_banks_aab19 = round(np.float64((column_aab12*(days/365))/column_aab9 * 100), 1)
column_banks_aab20 = round(np.float64(column_aab67/1000000), 1)
column_banks_aab21 = round(np.float64((column_aab67+column_aab69-column_aab92)/1000000), 1)
# Получение значений asaka для Banks
column_banks_asaka1 = column_asaka2
column_banks_asaka2 = period
column_banks_asaka3 = round(np.float64(column_asaka9/column_asaka3 * 100), 1)
column_banks_asaka4 = round(np.float64(column_asaka25/column_asaka16 * 100), 1)
column_banks_asaka5 = round(np.float64(column_asaka9/column_asaka25 * 100), 1)
column_banks_asaka6 = round(np.float64(column_asaka65/column_asaka53 * 100), 1)
column_banks_asaka7 = round(np.float64(column_asaka67/column_asaka37 * 100), 1)
column_banks_asaka8 = round(np.float64(((column_asaka40-column_asaka56)*(days/365))/column_asaka89 * 100), 1)
column_banks_asaka9 = round(np.float64(column_asaka65/(column_asaka40-column_asaka56+column_asaka46-column_asaka62-column_asaka92) * 100), 1)
column_banks_asaka10 = round(np.float64((column_asaka44*(days/365))/column_asaka9 * 100), 1)
column_banks_asaka11 = round(np.float64((column_asaka60*(days/365))/column_asaka25 * 100), 1)
column_banks_asaka12 = round(np.float64((column_asaka67*(days/365))/column_asaka3 * 100), 1)
column_banks_asaka13 = round(np.float64((column_asaka67*(days/365))/column_asaka24 * 100), 1)
if column_asaka77 != 0:
    column_banks_asaka14 = round(np.float64((((column_asaka66/1000)*0.87)*(days/365))/(column_asaka77*month)), 1)
else:
    column_banks_asaka14 = 0
column_banks_asaka15 = round(np.float64(column_asaka9/(column_asaka19+column_asaka20+column_asaka21+column_asaka22+column_asaka23+column_asaka25) * 100), 1)
column_banks_asaka16 = round(np.float64((column_asaka68*(days/365))/column_asaka3 * 100), 1)
if column_asaka77 != 0:
    column_banks_asaka17 = round(np.float64(((column_asaka67/1000)*(days/365))/(column_asaka77*month)), 1)
else:
    column_banks_asaka17 = 0
column_banks_asaka18 = round(np.float64((column_asaka68*(days/365))/column_asaka24 * 100), 1)
column_banks_asaka19 = round(np.float64((column_asaka12*(days/365))/column_asaka9 * 100), 1)
column_banks_asaka20 = round(np.float64(column_asaka67/1000000), 1)
column_banks_asaka21 = round(np.float64((column_asaka67+column_asaka69-column_asaka92)/1000000), 1)
# Получение значений xb для Banks
column_banks_xb1 = column_xb2
column_banks_xb2 = period
column_banks_xb3 = round(np.float64(column_xb9/column_xb3 * 100), 1)
column_banks_xb4 = round(np.float64(column_xb25/column_xb16 * 100), 1)
column_banks_xb5 = round(np.float64(column_xb9/column_xb25 * 100), 1)
column_banks_xb6 = round(np.float64(column_xb65/column_xb53 * 100), 1)
column_banks_xb7 = round(np.float64(column_xb67/column_xb37 * 100), 1)
column_banks_xb8 = round(np.float64(((column_xb40-column_xb56)*(days/365))/column_xb89 * 100), 1)
column_banks_xb9 = round(np.float64(column_xb65/(column_xb40-column_xb56+column_xb46-column_xb62-column_xb92) * 100), 1)
column_banks_xb10 = round(np.float64((column_xb44*(days/365))/column_xb9 * 100), 1)
column_banks_xb11 = round(np.float64((column_xb60*(days/365))/column_xb25 * 100), 1)
column_banks_xb12 = round(np.float64((column_xb67*(days/365))/column_xb3 * 100), 1)
column_banks_xb13 = round(np.float64((column_xb67*(days/365))/column_xb24 * 100), 1)
if column_xb77 != 0:
    column_banks_xb14 = round(np.float64((((column_xb66/1000)*0.87)*(days/365))/(column_xb77*month)), 1)
else:
    column_banks_xb14 = 0
column_banks_xb15 = round(np.float64(column_xb9/(column_xb19+column_xb20+column_xb21+column_xb22+column_xb23+column_xb25) * 100), 1)
column_banks_xb16 = round(np.float64((column_xb68*(days/365))/column_xb3 * 100), 1)
if column_xb77 != 0:
    column_banks_xb17 = round(np.float64(((column_xb67/1000)*(days/365))/(column_xb77*month)), 1)
else:
    column_banks_xb17 = 0
column_banks_xb18 = round(np.float64((column_xb68*(days/365))/column_xb24 * 100), 1)
column_banks_xb19 = round(np.float64((column_xb12*(days/365))/column_xb9 * 100), 1)
column_banks_xb20 = round(np.float64(column_xb67/1000000), 1)
column_banks_xb21 = round(np.float64((column_xb67+column_xb69-column_xb92)/1000000), 1)
# Получение значений poytaxt для Banks
column_banks_poytaxt1 = column_poytaxt2
column_banks_poytaxt2 = period
column_banks_poytaxt3 = round(np.float64(column_poytaxt9/column_poytaxt3 * 100), 1)
column_banks_poytaxt4 = round(np.float64(column_poytaxt25/column_poytaxt16 * 100), 1)
column_banks_poytaxt5 = round(np.float64(column_poytaxt9/column_poytaxt25 * 100), 1)
column_banks_poytaxt6 = round(np.float64(column_poytaxt65/column_poytaxt53 * 100), 1)
column_banks_poytaxt7 = round(np.float64(column_poytaxt67/column_poytaxt37 * 100), 1)
column_banks_poytaxt8 = round(np.float64(((column_poytaxt40-column_poytaxt56)*(days/365))/column_poytaxt89 * 100), 1)
column_banks_poytaxt9 = round(np.float64(column_poytaxt65/(column_poytaxt40-column_poytaxt56+column_poytaxt46-column_poytaxt62-column_poytaxt92) * 100), 1)
column_banks_poytaxt10 = round(np.float64((column_poytaxt44*(days/365))/column_poytaxt9 * 100), 1)
column_banks_poytaxt11 = round(np.float64((column_poytaxt60*(days/365))/column_poytaxt25 * 100), 1)
column_banks_poytaxt12 = round(np.float64((column_poytaxt67*(days/365))/column_poytaxt3 * 100), 1)
column_banks_poytaxt13 = round(np.float64((column_poytaxt67*(days/365))/column_poytaxt24 * 100), 1)
if column_poytaxt77 != 0:
    column_banks_poytaxt14 = round(np.float64((((column_poytaxt66/1000)*0.87)*(days/365))/(column_poytaxt77*month)), 1)
else:
    column_banks_poytaxt14 = 0
column_banks_poytaxt15 = round(np.float64(column_poytaxt9/(column_poytaxt19+column_poytaxt20+column_poytaxt21+column_poytaxt22+column_poytaxt23+column_poytaxt25) * 100), 1)
column_banks_poytaxt16 = round(np.float64((column_poytaxt68*(days/365))/column_poytaxt3 * 100), 1)
if column_poytaxt77 != 0:
    column_banks_poytaxt17 = round(np.float64(((column_poytaxt67/1000)*(days/365))/(column_poytaxt77*month)), 1)
else:
    column_banks_poytaxt17 = 0
column_banks_poytaxt18 = round(np.float64((column_poytaxt68*(days/365))/column_poytaxt24 * 100), 1)
column_banks_poytaxt19 = round(np.float64((column_poytaxt12*(days/365))/column_poytaxt9 * 100), 1)
column_banks_poytaxt20 = round(np.float64(column_poytaxt67/1000000), 1)
column_banks_poytaxt21 = round(np.float64((column_poytaxt67+column_poytaxt69-column_poytaxt92)/1000000), 1)
# Получение значений sqb для Banks
column_banks_sqb1 = column_sqb2
column_banks_sqb2 = period
column_banks_sqb3 = round(np.float64(column_sqb9/column_sqb3 * 100), 1)
column_banks_sqb4 = round(np.float64(column_sqb25/column_sqb16 * 100), 1)
column_banks_sqb5 = round(np.float64(column_sqb9/column_sqb25 * 100), 1)
column_banks_sqb6 = round(np.float64(column_sqb65/column_sqb53 * 100), 1)
column_banks_sqb7 = round(np.float64(column_sqb67/column_sqb37 * 100), 1)
column_banks_sqb8 = round(np.float64(((column_sqb40-column_sqb56)*(days/365))/column_sqb89 * 100), 1)
column_banks_sqb9 = round(np.float64(column_sqb65/(column_sqb40-column_sqb56+column_sqb46-column_sqb62-column_sqb92) * 100), 1)
column_banks_sqb10 = round(np.float64((column_sqb44*(days/365))/column_sqb9 * 100), 1)
column_banks_sqb11 = round(np.float64((column_sqb60*(days/365))/column_sqb25 * 100), 1)
column_banks_sqb12 = round(np.float64((column_sqb67*(days/365))/column_sqb3 * 100), 1)
column_banks_sqb13 = round(np.float64((column_sqb67*(days/365))/column_sqb24 * 100), 1)
if column_sqb77 != 0:
    column_banks_sqb14 = round(np.float64((((column_sqb66/1000)*0.87)*(days/365))/(column_sqb77*month)), 1)
else:
    column_banks_sqb14 = 0
column_banks_sqb15 = round(np.float64(column_sqb9/(column_sqb19+column_sqb20+column_sqb21+column_sqb22+column_sqb23+column_sqb25) * 100), 1)
column_banks_sqb16 = round(np.float64((column_sqb68*(days/365))/column_sqb3 * 100), 1)
if column_sqb77 != 0:
    column_banks_sqb17 = round(np.float64(((column_sqb67/1000)*(days/365))/(column_sqb77*month)), 1)
else:
    column_banks_sqb17 = 0
column_banks_sqb18 = round(np.float64((column_sqb68*(days/365))/column_sqb24 * 100), 1)
column_banks_sqb19 = round(np.float64((column_sqb12*(days/365))/column_sqb9 * 100), 1)
column_banks_sqb20 = round(np.float64(column_sqb67/1000000), 1)
column_banks_sqb21 = round(np.float64((column_sqb67+column_sqb69-column_sqb92)/1000000), 1)
# Получение значений ipoteka для Banks
column_banks_ipoteka1 = column_ipoteka2
column_banks_ipoteka2 = period
column_banks_ipoteka3 = round(np.float64(column_ipoteka9/column_ipoteka3 * 100), 1)
column_banks_ipoteka4 = round(np.float64(column_ipoteka25/column_ipoteka16 * 100), 1)
column_banks_ipoteka5 = round(np.float64(column_ipoteka9/column_ipoteka25 * 100), 1)
column_banks_ipoteka6 = round(np.float64(column_ipoteka65/column_ipoteka53 * 100), 1)
column_banks_ipoteka7 = round(np.float64(column_ipoteka67/column_ipoteka37 * 100), 1)
column_banks_ipoteka8 = round(np.float64(((column_ipoteka40-column_ipoteka56)*(days/365))/column_ipoteka89 * 100), 1)
column_banks_ipoteka9 = round(np.float64(column_ipoteka65/(column_ipoteka40-column_ipoteka56+column_ipoteka46-column_ipoteka62-column_ipoteka92) * 100), 1)
column_banks_ipoteka10 = round(np.float64((column_ipoteka44*(days/365))/column_ipoteka9 * 100), 1)
column_banks_ipoteka11 = round(np.float64((column_ipoteka60*(days/365))/column_ipoteka25 * 100), 1)
column_banks_ipoteka12 = round(np.float64((column_ipoteka67*(days/365))/column_ipoteka3 * 100), 1)
column_banks_ipoteka13 = round(np.float64((column_ipoteka67*(days/365))/column_ipoteka24 * 100), 1)
if column_ipoteka77 != 0:
    column_banks_ipoteka14 = round(np.float64((((column_ipoteka66/1000)*0.87)*(days/365))/(column_ipoteka77*month)), 1)
else:
    column_banks_ipoteka14 = 0
column_banks_ipoteka15 = round(np.float64(column_ipoteka9/(column_ipoteka19+column_ipoteka20+column_ipoteka21+column_ipoteka22+column_ipoteka23+column_ipoteka25) * 100), 1)
column_banks_ipoteka16 = round(np.float64((column_ipoteka68*(days/365))/column_ipoteka3 * 100), 1)
if column_ipoteka77 != 0:
    column_banks_ipoteka17 = round(np.float64(((column_ipoteka67/1000)*(days/365))/(column_ipoteka77*month)), 1)
else:
    column_banks_ipoteka17 = 0
column_banks_ipoteka18 = round(np.float64((column_ipoteka68*(days/365))/column_ipoteka24 * 100), 1)
column_banks_ipoteka19 = round(np.float64((column_ipoteka12*(days/365))/column_ipoteka9 * 100), 1)
column_banks_ipoteka20 = round(np.float64(column_ipoteka67/1000000), 1)
column_banks_ipoteka21 = round(np.float64((column_ipoteka67+column_ipoteka69-column_ipoteka92)/1000000), 1)
# Получение значений agro для Banks
column_banks_agro1 = column_agro2
column_banks_agro2 = period
column_banks_agro3 = round(np.float64(column_agro9/column_agro3 * 100), 1)
column_banks_agro4 = round(np.float64(column_agro25/column_agro16 * 100), 1)
column_banks_agro5 = round(np.float64(column_agro9/column_agro25 * 100), 1)
column_banks_agro6 = round(np.float64(column_agro65/column_agro53 * 100), 1)
column_banks_agro7 = round(np.float64(column_agro67/column_agro37 * 100), 1)
column_banks_agro8 = round(np.float64(((column_agro40-column_agro56)*(days/365))/column_agro89 * 100), 1)
column_banks_agro9 = round(np.float64(column_agro65/(column_agro40-column_agro56+column_agro46-column_agro62-column_agro92) * 100), 1)
column_banks_agro10 = round(np.float64((column_agro44*(days/365))/column_agro9 * 100), 1)
column_banks_agro11 = round(np.float64((column_agro60*(days/365))/column_agro25 * 100), 1)
column_banks_agro12 = round(np.float64((column_agro67*(days/365))/column_agro3 * 100), 1)
column_banks_agro13 = round(np.float64((column_agro67*(days/365))/column_agro24 * 100), 1)
if column_agro77 != 0:
    column_banks_agro14 = round(np.float64((((column_agro66/1000)*0.87)*(days/365))/(column_agro77*month)), 1)
else:
    column_banks_agro14 = 0
column_banks_agro15 = round(np.float64(column_agro9/(column_agro19+column_agro20+column_agro21+column_agro22+column_agro23+column_agro25) * 100), 1)
column_banks_agro16 = round(np.float64((column_agro68*(days/365))/column_agro3 * 100), 1)
if column_agro77 != 0:
    column_banks_agro17 = round(np.float64(((column_agro67/1000)*(days/365))/(column_agro77*month)), 1)
else:
    column_banks_agro17 = 0
column_banks_agro18 = round(np.float64((column_agro68*(days/365))/column_agro24 * 100), 1)
column_banks_agro19 = round(np.float64((column_agro12*(days/365))/column_agro9 * 100), 1)
column_banks_agro20 = round(np.float64(column_agro67/1000000), 1)
column_banks_agro21 = round(np.float64((column_agro67+column_agro69-column_agro92)/1000000), 1)
# Получение значений qqb для Bank
column_banks_qqb1 = column_qqb2
column_banks_qqb2 = period
column_banks_qqb3 = round(np.float64(column_qqb9/column_qqb3 * 100), 1)
column_banks_qqb4 = round(np.float64(column_qqb25/column_qqb16 * 100), 1)
column_banks_qqb5 = round(np.float64(column_qqb9/column_qqb25 * 100), 1)
column_banks_qqb6 = round(np.float64(column_qqb65/column_qqb53 * 100), 1)
column_banks_qqb7 = round(np.float64(column_qqb67/column_qqb37 * 100), 1)
column_banks_qqb8 = round(np.float64(((column_qqb40-column_qqb56)*(days/365))/column_qqb89 * 100), 1)
column_banks_qqb9 = round(np.float64(column_qqb65/(column_qqb40-column_qqb56+column_qqb46-column_qqb62-column_qqb92) * 100), 1)
column_banks_qqb10 = round(np.float64((column_qqb44*(days/365))/column_qqb9 * 100), 1)
column_banks_qqb11 = round(np.float64((column_qqb60*(days/365))/column_qqb25 * 100), 1)
column_banks_qqb12 = round(np.float64((column_qqb67*(days/365))/column_qqb3 * 100), 1)
column_banks_qqb13 = round(np.float64((column_qqb67*(days/365))/column_qqb24 * 100), 1)
if column_qqb77 != 0:
    column_banks_qqb14 = round(np.float64((((column_qqb66/1000)*0.87)*(days/365))/(column_qqb77*month)), 1)
else:
    column_banks_qqb14 = 0
column_banks_qqb15 = round(np.float64(column_qqb9/(column_qqb19+column_qqb20+column_qqb21+column_qqb22+column_qqb23+column_qqb25) * 100), 1)
column_banks_qqb16 = round(np.float64((column_qqb68*(days/365))/column_qqb3 * 100), 1)
if column_qqb77 != 0:
    column_banks_qqb17 = round(np.float64(((column_qqb67/1000)*(days/365))/(column_qqb77*month)), 1)
else:
    column_banks_qqb17 = 0
column_banks_qqb18 = round(np.float64((column_qqb68*(days/365))/column_qqb24 * 100), 1)
column_banks_qqb19 = round(np.float64((column_qqb12*(days/365))/column_qqb9 * 100), 1)
column_banks_qqb20 = round(np.float64(column_qqb67/1000000), 1)
column_banks_qqb21 = round(np.float64((column_qqb67+column_qqb69-column_qqb92)/1000000), 1)
# Получение значений turon для Banks
column_banks_turon1 = column_turon2
column_banks_turon2 = period
column_banks_turon3 = round(np.float64(column_turon9/column_turon3 * 100), 1)
column_banks_turon4 = round(np.float64(column_turon25/column_turon16 * 100), 1)
column_banks_turon5 = round(np.float64(column_turon9/column_turon25 * 100), 1)
column_banks_turon6 = round(np.float64(column_turon65/column_turon53 * 100), 1)
column_banks_turon7 = round(np.float64(column_turon67/column_turon37 * 100), 1)
column_banks_turon8 = round(np.float64(((column_turon40-column_turon56)*(days/365))/column_turon89 * 100), 1)
column_banks_turon9 = round(np.float64(column_turon65/(column_turon40-column_turon56+column_turon46-column_turon62-column_turon92) * 100), 1)
column_banks_turon10 = round(np.float64((column_turon44*(days/365))/column_turon9 * 100), 1)
column_banks_turon11 = round(np.float64((column_turon60*(days/365))/column_turon25 * 100), 1)
column_banks_turon12 = round(np.float64((column_turon67*(days/365))/column_turon3 * 100), 1)
column_banks_turon13 = round(np.float64((column_turon67*(days/365))/column_turon24 * 100), 1)
if column_turon77 != 0:
    column_banks_turon14 = round(np.float64((((column_turon66/1000)*0.87)*(days/365))/(column_turon77*month)), 1)
else:
    column_banks_turon14 = 0
column_banks_turon15 = round(np.float64(column_turon9/(column_turon19+column_turon20+column_turon21+column_turon22+column_turon23+column_turon25) * 100), 1)
column_banks_turon16 = round(np.float64((column_turon68*(days/365))/column_turon3 * 100), 1)
if column_turon77 != 0:
    column_banks_turon17 = round(np.float64(((column_turon67/1000)*(days/365))/(column_turon77*month)), 1)
else:
    column_banks_turon17 = 0
column_banks_turon18 = round(np.float64((column_turon68*(days/365))/column_turon24 * 100), 1)
column_banks_turon19 = round(np.float64((column_turon12*(days/365))/column_turon9 * 100), 1)
column_banks_turon20 = round(np.float64(column_turon67/1000000), 1)
column_banks_turon21 = round(np.float64((column_turon67+column_turon69-column_turon92)/1000000), 1)
# Получение значений micro для Banks
column_banks_micro1 = column_micro2
column_banks_micro2 = period
column_banks_micro3 = round(np.float64(column_micro9/column_micro3 * 100), 1)
column_banks_micro4 = round(np.float64(column_micro25/column_micro16 * 100), 1)
column_banks_micro5 = round(np.float64(column_micro9/column_micro25 * 100), 1)
column_banks_micro6 = round(np.float64(column_micro65/column_micro53 * 100), 1)
column_banks_micro7 = round(np.float64(column_micro67/column_micro37 * 100), 1)
column_banks_micro8 = round(np.float64(((column_micro40-column_micro56)*(days/365))/column_micro89 * 100), 1)
column_banks_micro9 = round(np.float64(column_micro65/(column_micro40-column_micro56+column_micro46-column_micro62-column_micro92) * 100), 1)
column_banks_micro10 = round(np.float64((column_micro44*(days/365))/column_micro9 * 100), 1)
column_banks_micro11 = round(np.float64((column_micro60*(days/365))/column_micro25 * 100), 1)
column_banks_micro12 = round(np.float64((column_micro67*(days/365))/column_micro3 * 100), 1)
column_banks_micro13 = round(np.float64((column_micro67*(days/365))/column_micro24 * 100), 1)
if column_micro77 != 0:
    column_banks_micro14 = round(np.float64((((column_micro66/1000)*0.87)*(days/365))/(column_micro77*month)), 1)
else:
    column_banks_micro14 = 0
column_banks_micro15 = round(np.float64(column_micro9/(column_micro19+column_micro20+column_micro21+column_micro22+column_micro23+column_micro25) * 100), 1)
column_banks_micro16 = round(np.float64((column_micro68*(days/365))/column_micro3 * 100), 1)
if column_micro77 != 0:
    column_banks_micro17 = round(np.float64(((column_micro67/1000)*(days/365))/(column_micro77*month)), 1)
else:
    column_banks_micro17 = 0
column_banks_micro18 = round(np.float64((column_micro68*(days/365))/column_micro24 * 100), 1)
column_banks_micro19 = round(np.float64((column_micro12*(days/365))/column_micro9 * 100), 1)
column_banks_micro20 = round(np.float64(column_micro67/1000000), 1)
column_banks_micro21 = round(np.float64((column_micro67+column_micro69-column_micro92)/1000000), 1)
# Получение значений aloqa для Banks
column_banks_aloqa1 = column_aloqa2
column_banks_aloqa2 = period
column_banks_aloqa3 = round(np.float64(column_aloqa9/column_aloqa3 * 100), 1)
column_banks_aloqa4 = round(np.float64(column_aloqa25/column_aloqa16 * 100), 1)
column_banks_aloqa5 = round(np.float64(column_aloqa9/column_aloqa25 * 100), 1)
column_banks_aloqa6 = round(np.float64(column_aloqa65/column_aloqa53 * 100), 1)
column_banks_aloqa7 = round(np.float64(column_aloqa67/column_aloqa37 * 100), 1)
column_banks_aloqa8 = round(np.float64(((column_aloqa40-column_aloqa56)*(days/365))/column_aloqa89 * 100), 1)
column_banks_aloqa9 = round(np.float64(column_aloqa65/(column_aloqa40-column_aloqa56+column_aloqa46-column_aloqa62-column_aloqa92) * 100), 1)
column_banks_aloqa10 = round(np.float64((column_aloqa44*(days/365))/column_aloqa9 * 100), 1)
column_banks_aloqa11 = round(np.float64((column_aloqa60*(days/365))/column_aloqa25 * 100), 1)
column_banks_aloqa12 = round(np.float64((column_aloqa67*(days/365))/column_aloqa3 * 100), 1)
column_banks_aloqa13 = round(np.float64((column_aloqa67*(days/365))/column_aloqa24 * 100), 1)
if column_aloqa77 != 0:
    column_banks_aloqa14 = round(np.float64((((column_aloqa66/1000)*0.87)*(days/365))/(column_aloqa77*month)), 1)
else:
    column_banks_aloqa14 = 0
column_banks_aloqa15 = round(np.float64(column_aloqa9/(column_aloqa19+column_aloqa20+column_aloqa21+column_aloqa22+column_aloqa23+column_aloqa25) * 100), 1)
column_banks_aloqa16 = round(np.float64((column_aloqa68*(days/365))/column_aloqa3 * 100), 1)
if column_aloqa77 != 0:
    column_banks_aloqa17 = round(np.float64(((column_aloqa67/1000)*(days/365))/(column_aloqa77*month)), 1)
else:
    column_banks_aloqa17 = 0
column_banks_aloqa18 = round(np.float64((column_aloqa68*(days/365))/column_aloqa24 * 100), 1)
column_banks_aloqa19 = round(np.float64((column_aloqa12*(days/365))/column_aloqa9 * 100), 1)
column_banks_aloqa20 = round(np.float64(column_aloqa67/1000000), 1)
column_banks_aloqa21 = round(np.float64((column_aloqa67+column_aloqa69-column_aloqa92)/1000000), 1)
# Получение значений ipak для Banks
column_banks_ipak1 = column_ipak2
column_banks_ipak2 = period
column_banks_ipak3 = round(np.float64(column_ipak9/column_ipak3 * 100), 1)
column_banks_ipak4 = round(np.float64(column_ipak25/column_ipak16 * 100), 1)
column_banks_ipak5 = round(np.float64(column_ipak9/column_ipak25 * 100), 1)
column_banks_ipak6 = round(np.float64(column_ipak65/column_ipak53 * 100), 1)
column_banks_ipak7 = round(np.float64(column_ipak67/column_ipak37 * 100), 1)
column_banks_ipak8 = round(np.float64(((column_ipak40-column_ipak56)*(days/365))/column_ipak89 * 100), 1)
column_banks_ipak9 = round(np.float64(column_ipak65/(column_ipak40-column_ipak56+column_ipak46-column_ipak62-column_ipak92) * 100), 1)
column_banks_ipak10 = round(np.float64((column_ipak44*(days/365))/column_ipak9 * 100), 1)
column_banks_ipak11 = round(np.float64((column_ipak60*(days/365))/column_ipak25 * 100), 1)
column_banks_ipak12 = round(np.float64((column_ipak67*(days/365))/column_ipak3 * 100), 1)
column_banks_ipak13 = round(np.float64((column_ipak67*(days/365))/column_ipak24 * 100), 1)
if column_ipak77 != 0:
    column_banks_ipak14 = round(np.float64((((column_ipak66/1000)*0.87)*(days/365))/(column_ipak77*month)), 1)
else:
    column_banks_ipak14 = 0
column_banks_ipak15 = round(np.float64(column_ipak9/(column_ipak19+column_ipak20+column_ipak21+column_ipak22+column_ipak23+column_ipak25) * 100), 1)
column_banks_ipak16 = round(np.float64((column_ipak68*(days/365))/column_ipak3 * 100), 1)
if column_ipak77 != 0:
    column_banks_ipak17 = round(np.float64(((column_ipak67/1000)*(days/365))/(column_ipak77*month)), 1)
else:
    column_banks_ipak17 = 0
column_banks_ipak18 = round(np.float64((column_ipak68*(days/365))/column_ipak24 * 100), 1)
column_banks_ipak19 = round(np.float64((column_ipak12*(days/365))/column_ipak9 * 100), 1)
column_banks_ipak20 = round(np.float64(column_ipak67/1000000), 1)
column_banks_ipak21 = round(np.float64((column_ipak67+column_ipak69-column_ipak92)/1000000), 1)
# Получение значений kapital для Banks
column_banks_kapital1 = column_kapital2
column_banks_kapital2 = period
column_banks_kapital3 = round(np.float64(column_kapital9/column_kapital3 * 100), 1)
column_banks_kapital4 = round(np.float64(column_kapital25/column_kapital16 * 100), 1)
column_banks_kapital5 = round(np.float64(column_kapital9/column_kapital25 * 100), 1)
column_banks_kapital6 = round(np.float64(column_kapital65/column_kapital53 * 100), 1)
column_banks_kapital7 = round(np.float64(column_kapital67/column_kapital37 * 100), 1)
column_banks_kapital8 = round(np.float64(((column_kapital40-column_kapital56)*(days/365))/column_kapital89 * 100), 1)
column_banks_kapital9 = round(np.float64(column_kapital65/(column_kapital40-column_kapital56+column_kapital46-column_kapital62-column_kapital92) * 100), 1)
column_banks_kapital10 = round(np.float64((column_kapital44*(days/365))/column_kapital9 * 100), 1)
column_banks_kapital11 = round(np.float64((column_kapital60*(days/365))/column_kapital25 * 100), 1)
column_banks_kapital12 = round(np.float64((column_kapital67*(days/365))/column_kapital3 * 100), 1)
column_banks_kapital13 = round(np.float64((column_kapital67*(days/365))/column_kapital24 * 100), 1)
if column_kapital77 != 0:
    column_banks_kapital14 = round(np.float64((((column_kapital66/1000)*0.87)*(days/365))/(column_kapital77*month)), 1)
else:
    column_banks_kapital14 = 0
column_banks_kapital15 = round(np.float64(column_kapital9/(column_kapital19+column_kapital20+column_kapital21+column_kapital22+column_kapital23+column_kapital25) * 100), 1)
column_banks_kapital16 = round(np.float64((column_kapital68*(days/365))/column_kapital3 * 100), 1)
if column_kapital77 != 0:
    column_banks_kapital17 = round(np.float64(((column_kapital67/1000)*(days/365))/(column_kapital77*month)), 1)
else:
    column_banks_kapital17 = 0
column_banks_kapital18 = round(np.float64((column_kapital68*(days/365))/column_kapital24 * 100), 1)
column_banks_kapital19 = round(np.float64((column_kapital12*(days/365))/column_kapital9 * 100), 1)
column_banks_kapital20 = round(np.float64(column_kapital67/1000000), 1)
column_banks_kapital21 = round(np.float64((column_kapital67+column_kapital69-column_kapital92)/1000000), 1)
# Получение значений infin для Banks
column_banks_infin1 = column_infin2
column_banks_infin2 = period
column_banks_infin3 = round(np.float64(column_infin9/column_infin3 * 100), 1)
column_banks_infin4 = round(np.float64(column_infin25/column_infin16 * 100), 1)
column_banks_infin5 = round(np.float64(column_infin9/column_infin25 * 100), 1)
column_banks_infin6 = round(np.float64(column_infin65/column_infin53 * 100), 1)
column_banks_infin7 = round(np.float64(column_infin67/column_infin37 * 100), 1)
column_banks_infin8 = round(np.float64(((column_infin40-column_infin56)*(days/365))/column_infin89 * 100), 1)
column_banks_infin9 = round(np.float64(column_infin65/(column_infin40-column_infin56+column_infin46-column_infin62-column_infin92) * 100), 1)
column_banks_infin10 = round(np.float64((column_infin44*(days/365))/column_infin9 * 100), 1)
column_banks_infin11 = round(np.float64((column_infin60*(days/365))/column_infin25 * 100), 1)
column_banks_infin12 = round(np.float64((column_infin67*(days/365))/column_infin3 * 100), 1)
column_banks_infin13 = round(np.float64((column_infin67*(days/365))/column_infin24 * 100), 1)
if column_infin77 != 0:
    column_banks_infin14 = round(np.float64((((column_infin66/1000)*0.87)*(days/365))/(column_infin77*month)), 1)
else:
    column_banks_infin14 = 0
column_banks_infin15 = round(np.float64(column_infin9/(column_infin19+column_infin20+column_infin21+column_infin22+column_infin23+column_infin25) * 100), 1)
column_banks_infin16 = round(np.float64((column_infin68*(days/365))/column_infin3 * 100), 1)
if column_infin77 != 0:
    column_banks_infin17 = round(np.float64(((column_infin67/1000)*(days/365))/(column_infin77*month)), 1)
else:
    column_banks_infin17 = 0
column_banks_infin18 = round(np.float64((column_infin68*(days/365))/column_infin24 * 100), 1)
column_banks_infin19 = round(np.float64((column_infin12*(days/365))/column_infin9 * 100), 1)
column_banks_infin20 = round(np.float64(column_infin67/1000000), 1)
column_banks_infin21 = round(np.float64((column_infin67+column_infin69-column_infin92)/1000000), 1)
# Получение значений universal для Banks
column_banks_universal1 = column_universal2
column_banks_universal2 = period
column_banks_universal3 = round(np.float64(column_universal9/column_universal3 * 100), 1)
column_banks_universal4 = round(np.float64(column_universal25/column_universal16 * 100), 1)
column_banks_universal5 = round(np.float64(column_universal9/column_universal25 * 100), 1)
column_banks_universal6 = round(np.float64(column_universal65/column_universal53 * 100), 1)
column_banks_universal7 = round(np.float64(column_universal67/column_universal37 * 100), 1)
column_banks_universal8 = round(np.float64(((column_universal40-column_universal56)*(days/365))/column_universal89 * 100), 1)
column_banks_universal9 = round(np.float64(column_universal65/(column_universal40-column_universal56+column_universal46-column_universal62-column_universal92) * 100), 1)
column_banks_universal10 = round(np.float64((column_universal44*(days/365))/column_universal9 * 100), 1)
column_banks_universal11 = round(np.float64((column_universal60*(days/365))/column_universal25 * 100), 1)
column_banks_universal12 = round(np.float64((column_universal67*(days/365))/column_universal3 * 100), 1)
column_banks_universal13 = round(np.float64((column_universal67*(days/365))/column_universal24 * 100), 1)
if column_universal77 != 0:
    column_banks_universal14 = round(np.float64((((column_universal66/1000)*0.87)*(days/365))/(column_universal77*month)), 1)
else:
    column_banks_universal14 = 0
column_banks_universal15 = round(np.float64(column_universal9/(column_universal19+column_universal20+column_universal21+column_universal22+column_universal23+column_universal25) * 100), 1)
column_banks_universal16 = round(np.float64((column_universal68*(days/365))/column_universal3 * 100), 1)
if column_universal77 != 0:
    column_banks_universal17 = round(np.float64(((column_universal67/1000)*(days/365))/(column_universal77*month)), 1)
else:
    column_banks_universal17 = 0
column_banks_universal18 = round(np.float64((column_universal68*(days/365))/column_universal24 * 100), 1)
column_banks_universal19 = round(np.float64((column_universal12*(days/365))/column_universal9 * 100), 1)
column_banks_universal20 = round(np.float64(column_universal67/1000000), 1)
column_banks_universal21 = round(np.float64((column_universal67+column_universal69-column_universal92)/1000000), 1)
# Получение значений avo для Banks
column_banks_avo1 = column_avo2
column_banks_avo2 = period
column_banks_avo3 = round(np.float64(column_avo9/column_avo3 * 100), 1)
column_banks_avo4 = round(np.float64(column_avo25/column_avo16 * 100), 1)
column_banks_avo5 = round(np.float64(column_avo9/column_avo25 * 100), 1)
column_banks_avo6 = round(np.float64(column_avo65/column_avo53 * 100), 1)
column_banks_avo7 = round(np.float64(column_avo67/column_avo37 * 100), 1)
column_banks_avo8 = round(np.float64(((column_avo40-column_avo56)*(days/365))/column_avo89 * 100), 1)
column_banks_avo9 = round(np.float64(column_avo65/(column_avo40-column_avo56+column_avo46-column_avo62-column_avo92) * 100), 1)
column_banks_avo10 = round(np.float64((column_avo44*(days/365))/column_avo9 * 100), 1)
column_banks_avo11 = round(np.float64((column_avo60*(days/365))/column_avo25 * 100), 1)
column_banks_avo12 = round(np.float64((column_avo67*(days/365))/column_avo3 * 100), 1)
column_banks_avo13 = round(np.float64((column_avo67*(days/365))/column_avo24 * 100), 1)
if column_avo77 != 0:
    column_banks_avo14 = round(np.float64((((column_avo66/1000)*0.87)*(days/365))/(column_avo77*month)), 1)
else:
    column_banks_avo14 = 0
column_banks_avo15 = round(np.float64(column_avo9/(column_avo19+column_avo20+column_avo21+column_avo22+column_avo23+column_avo25) * 100), 1)
column_banks_avo16 = round(np.float64((column_avo68*(days/365))/column_avo3 * 100), 1)
if column_avo77 != 0:
    column_banks_avo17 = round(np.float64(((column_avo67/1000)*(days/365))/(column_avo77*month)), 1)
else:
    column_banks_avo17 = 0
column_banks_avo18 = round(np.float64((column_avo68*(days/365))/column_avo24 * 100), 1)
column_banks_avo19 = round(np.float64((column_avo12*(days/365))/column_avo9 * 100), 1)
column_banks_avo20 = round(np.float64(column_avo67/1000000), 1)
column_banks_avo21 = round(np.float64((column_avo67+column_avo69-column_avo92)/1000000), 1)
# Получение значений madad для Banks
column_banks_madad1 = column_madad2
column_banks_madad2 = period
column_banks_madad3 = round(np.float64(column_madad9/column_madad3 * 100), 1)
column_banks_madad4 = round(np.float64(column_madad25/column_madad16 * 100), 1)
column_banks_madad5 = round(np.float64(column_madad9/column_madad25 * 100), 1)
column_banks_madad6 = round(np.float64(column_madad65/column_madad53 * 100), 1)
column_banks_madad7 = round(np.float64(column_madad67/column_madad37 * 100), 1)
column_banks_madad8 = round(np.float64(((column_madad40-column_madad56)*(days/365))/column_madad89 * 100), 1)
column_banks_madad9 = round(np.float64(column_madad65/(column_madad40-column_madad56+column_madad46-column_madad62-column_madad92) * 100), 1)
column_banks_madad10 = round(np.float64((column_madad44*(days/365))/column_madad9 * 100), 1)
column_banks_madad11 = round(np.float64((column_madad60*(days/365))/column_madad25 * 100), 1)
column_banks_madad12 = round(np.float64((column_madad67*(days/365))/column_madad3 * 100), 1)
column_banks_madad13 = round(np.float64((column_madad67*(days/365))/column_madad24 * 100), 1)
if column_madad77 != 0:
    column_banks_madad14 = round(np.float64((((column_madad66/1000)*0.87)*(days/365))/(column_madad77*month)), 1)
else:
    column_banks_madad14 = 0
column_banks_madad15 = round(np.float64(column_madad9/(column_madad19+column_madad20+column_madad21+column_madad22+column_madad23+column_madad25) * 100), 1)
column_banks_madad16 = round(np.float64((column_madad68*(days/365))/column_madad3 * 100), 1)
if column_madad77 != 0:
    column_banks_madad17 = round(np.float64(((column_madad67/1000)*(days/365))/(column_madad77*month)), 1)
else:
    column_banks_madad17 = 0
column_banks_madad18 = round(np.float64((column_madad68*(days/365))/column_madad24 * 100), 1)
column_banks_madad19 = round(np.float64((column_madad12*(days/365))/column_madad9 * 100), 1)
column_banks_madad20 = round(np.float64(column_madad67/1000000), 1)
column_banks_madad21 = round(np.float64((column_madad67+column_madad69-column_madad92)/1000000), 1)
# Получение значений ofb для Banks
column_banks_ofb1 = column_ofb2
column_banks_ofb2 = period
column_banks_ofb3 = round(np.float64(column_ofb9/column_ofb3 * 100), 1)
column_banks_ofb4 = round(np.float64(column_ofb25/column_ofb16 * 100), 1)
column_banks_ofb5 = round(np.float64(column_ofb9/column_ofb25 * 100), 1)
column_banks_ofb6 = round(np.float64(column_ofb65/column_ofb53 * 100), 1)
column_banks_ofb7 = round(np.float64(column_ofb67/column_ofb37 * 100), 1)
column_banks_ofb8 = round(np.float64(((column_ofb40-column_ofb56)*(days/365))/column_ofb89 * 100), 1)
column_banks_ofb9 = round(np.float64(column_ofb65/(column_ofb40-column_ofb56+column_ofb46-column_ofb62-column_ofb92) * 100), 1)
column_banks_ofb10 = round(np.float64((column_ofb44*(days/365))/column_ofb9 * 100), 1)
column_banks_ofb11 = round(np.float64((column_ofb60*(days/365))/column_ofb25 * 100), 1)
column_banks_ofb12 = round(np.float64((column_ofb67*(days/365))/column_ofb3 * 100), 1)
column_banks_ofb13 = round(np.float64((column_ofb67*(days/365))/column_ofb24 * 100), 1)
if column_ofb77 != 0:
    column_banks_ofb14 = round(np.float64((((column_ofb66/1000)*0.87)*(days/365))/(column_ofb77*month)), 1)
else:
    column_banks_ofb14 = 0
column_banks_ofb15 = round(np.float64(column_ofb9/(column_ofb19+column_ofb20+column_ofb21+column_ofb22+column_ofb23+column_ofb25) * 100), 1)
column_banks_ofb16 = round(np.float64((column_ofb68*(days/365))/column_ofb3 * 100), 1)
if column_ofb77 != 0:
    column_banks_ofb17 = round(np.float64(((column_ofb67/1000)*(days/365))/(column_ofb77*month)), 1)
else:
    column_banks_ofb17 = 0
column_banks_ofb18 = round(np.float64((column_ofb68*(days/365))/column_ofb24 * 100), 1)
column_banks_ofb19 = round(np.float64((column_ofb12*(days/365))/column_ofb9 * 100), 1)
column_banks_ofb20 = round(np.float64(column_ofb67/1000000), 1)
column_banks_ofb21 = round(np.float64((column_ofb67+column_ofb69-column_ofb92)/1000000), 1)
# Получение значений trust для Banks
column_banks_trust1 = column_trust2
column_banks_trust2 = period
column_banks_trust3 = round(np.float64(column_trust9/column_trust3 * 100), 1)
column_banks_trust4 = round(np.float64(column_trust25/column_trust16 * 100), 1)
column_banks_trust5 = round(np.float64(column_trust9/column_trust25 * 100), 1)
column_banks_trust6 = round(np.float64(column_trust65/column_trust53 * 100), 1)
column_banks_trust7 = round(np.float64(column_trust67/column_trust37 * 100), 1)
column_banks_trust8 = round(np.float64(((column_trust40-column_trust56)*(days/365))/column_trust89 * 100), 1)
column_banks_trust9 = round(np.float64(column_trust65/(column_trust40-column_trust56+column_trust46-column_trust62-column_trust92) * 100), 1)
column_banks_trust10 = round(np.float64((column_trust44*(days/365))/column_trust9 * 100), 1)
column_banks_trust11 = round(np.float64((column_trust60*(days/365))/column_trust25 * 100), 1)
column_banks_trust12 = round(np.float64((column_trust67*(days/365))/column_trust3 * 100), 1)
column_banks_trust13 = round(np.float64((column_trust67*(days/365))/column_trust24 * 100), 1)
if column_trust77 != 0:
    column_banks_trust14 = round(np.float64((((column_trust66/1000)*0.87)*(days/365))/(column_trust77*month)), 1)
else:
    column_banks_trust14 = 0
column_banks_trust15 = round(np.float64(column_trust9/(column_trust19+column_trust20+column_trust21+column_trust22+column_trust23+column_trust25) * 100), 1)
column_banks_trust16 = round(np.float64((column_trust68*(days/365))/column_trust3 * 100), 1)
if column_trust77 != 0:
    column_banks_trust17 = round(np.float64(((column_trust67/1000)*(days/365))/(column_trust77*month)), 1)
else:
    column_banks_trust17 = 0
column_banks_trust18 = round(np.float64((column_trust68*(days/365))/column_trust24 * 100), 1)
column_banks_trust19 = round(np.float64((column_trust12*(days/365))/column_trust9 * 100), 1)
column_banks_trust20 = round(np.float64(column_trust67/1000000), 1)
column_banks_trust21 = round(np.float64((column_trust67+column_trust69-column_trust92)/1000000), 1)
# Получение значений davr для Banks
column_banks_davr1 = column_davr2
column_banks_davr2 = period
column_banks_davr3 = round(np.float64(column_davr9/column_davr3 * 100), 1)
column_banks_davr4 = round(np.float64(column_davr25/column_davr16 * 100), 1)
column_banks_davr5 = round(np.float64(column_davr9/column_davr25 * 100), 1)
column_banks_davr6 = round(np.float64(column_davr65/column_davr53 * 100), 1)
column_banks_davr7 = round(np.float64(column_davr67/column_davr37 * 100), 1)
column_banks_davr8 = round(np.float64(((column_davr40-column_davr56)*(days/365))/column_davr89 * 100), 1)
column_banks_davr9 = round(np.float64(column_davr65/(column_davr40-column_davr56+column_davr46-column_davr62-column_davr92) * 100), 1)
column_banks_davr10 = round(np.float64((column_davr44*(days/365))/column_davr9 * 100), 1)
column_banks_davr11 = round(np.float64((column_davr60*(days/365))/column_davr25 * 100), 1)
column_banks_davr12 = round(np.float64((column_davr67*(days/365))/column_davr3 * 100), 1)
column_banks_davr13 = round(np.float64((column_davr67*(days/365))/column_davr24 * 100), 1)
if column_davr77 != 0:
    column_banks_davr14 = round(np.float64((((column_davr66/1000)*0.87)*(days/365))/(column_davr77*month)), 1)
else:
    column_banks_davr14 = 0
column_banks_davr15 = round(np.float64(column_davr9/(column_davr19+column_davr20+column_davr21+column_davr22+column_davr23+column_davr25) * 100), 1)
column_banks_davr16 = round(np.float64((column_davr68*(days/365))/column_davr3 * 100), 1)
if column_davr77 != 0:
    column_banks_davr17 = round(np.float64(((column_davr67/1000)*(days/365))/(column_davr77*month)), 1)
else:
    column_banks_davr17 = 0
column_banks_davr18 = round(np.float64((column_davr68*(days/365))/column_davr24 * 100), 1)
column_banks_davr19 = round(np.float64((column_davr12*(days/365))/column_davr9 * 100), 1)
column_banks_davr20 = round(np.float64(column_davr67/1000000), 1)
column_banks_davr21 = round(np.float64((column_davr67+column_davr69-column_davr92)/1000000), 1)
# Получение значений ravnaq для Banks
column_banks_ravnaq1 = column_ravnaq2
column_banks_ravnaq2 = period
column_banks_ravnaq3 = round(np.float64(column_ravnaq9/column_ravnaq3 * 100), 1)
column_banks_ravnaq4 = round(np.float64(column_ravnaq25/column_ravnaq16 * 100), 1)
column_banks_ravnaq5 = round(np.float64(column_ravnaq9/column_ravnaq25 * 100), 1)
column_banks_ravnaq6 = round(np.float64(column_ravnaq65/column_ravnaq53 * 100), 1)
column_banks_ravnaq7 = round(np.float64(column_ravnaq67/column_ravnaq37 * 100), 1)
column_banks_ravnaq8 = round(np.float64(((column_ravnaq40-column_ravnaq56)*(days/365))/column_ravnaq89 * 100), 1)
column_banks_ravnaq9 = round(np.float64(column_ravnaq65/(column_ravnaq40-column_ravnaq56+column_ravnaq46-column_ravnaq62-column_ravnaq92) * 100), 1)
column_banks_ravnaq10 = round(np.float64((column_ravnaq44*(days/365))/column_ravnaq9 * 100), 1)
column_banks_ravnaq11 = round(np.float64((column_ravnaq60*(days/365))/column_ravnaq25 * 100), 1)
column_banks_ravnaq12 = round(np.float64((column_ravnaq67*(days/365))/column_ravnaq3 * 100), 1)
column_banks_ravnaq13 = round(np.float64((column_ravnaq67*(days/365))/column_ravnaq24 * 100), 1)
if column_ravnaq77 != 0:
    column_banks_ravnaq14 = round(np.float64((((column_ravnaq66/1000)*0.87)*(days/365))/(column_ravnaq77*month)), 1)
else:
    column_banks_ravnaq14 = 0
column_banks_ravnaq15 = round(np.float64(column_ravnaq9/(column_ravnaq19+column_ravnaq20+column_ravnaq21+column_ravnaq22+column_ravnaq23+column_ravnaq25) * 100), 1)
column_banks_ravnaq16 = round(np.float64((column_ravnaq68*(days/365))/column_ravnaq3 * 100), 1)
if column_ravnaq77 != 0:
    column_banks_ravnaq17 = round(np.float64(((column_ravnaq67/1000)*(days/365))/(column_ravnaq77*month)), 1)
else:
    column_banks_ravnaq17 = 0
column_banks_ravnaq18 = round(np.float64((column_ravnaq68*(days/365))/column_ravnaq24 * 100), 1)
column_banks_ravnaq19 = round(np.float64((column_ravnaq12*(days/365))/column_ravnaq9 * 100), 1)
column_banks_ravnaq20 = round(np.float64(column_ravnaq67/1000000), 1)
column_banks_ravnaq21 = round(np.float64((column_ravnaq67+column_ravnaq69-column_ravnaq92)/1000000), 1)
# Получение значений anor для Banks
column_banks_anor1 = column_anor2
column_banks_anor2 = period
column_banks_anor3 = round(np.float64(column_anor9/column_anor3 * 100), 1)
column_banks_anor4 = round(np.float64(column_anor25/column_anor16 * 100), 1)
column_banks_anor5 = round(np.float64(column_anor9/column_anor25 * 100), 1)
column_banks_anor6 = round(np.float64(column_anor65/column_anor53 * 100), 1)
column_banks_anor7 = round(np.float64(column_anor67/column_anor37 * 100), 1)
column_banks_anor8 = round(np.float64(((column_anor40-column_anor56)*(days/365))/column_anor89 * 100), 1)
column_banks_anor9 = round(np.float64(column_anor65/(column_anor40-column_anor56+column_anor46-column_anor62-column_anor92) * 100), 1)
column_banks_anor10 = round(np.float64((column_anor44*(days/365))/column_anor9 * 100), 1)
column_banks_anor11 = round(np.float64((column_anor60*(days/365))/column_anor25 * 100), 1)
column_banks_anor12 = round(np.float64((column_anor67*(days/365))/column_anor3 * 100), 1)
column_banks_anor13 = round(np.float64((column_anor67*(days/365))/column_anor24 * 100), 1)
if column_anor77 != 0:
    column_banks_anor14 = round(np.float64((((column_anor66/1000)*0.87)*(days/365))/(column_anor77*month)), 1)
else:
    column_banks_anor14 = 0
column_banks_anor15 = round(np.float64(column_anor9/(column_anor19+column_anor20+column_anor21+column_anor22+column_anor23+column_anor25) * 100), 1)
column_banks_anor16 = round(np.float64((column_anor68*(days/365))/column_anor3 * 100), 1)
if column_anor77 != 0:
    column_banks_anor17 = round(np.float64(((column_anor67/1000)*(days/365))/(column_anor77*month)), 1)
else:
    column_banks_anor17 = 0
column_banks_anor18 = round(np.float64((column_anor68*(days/365))/column_anor24 * 100), 1)
column_banks_anor19 = round(np.float64((column_anor12*(days/365))/column_anor9 * 100), 1)
column_banks_anor20 = round(np.float64(column_anor67/1000000), 1)
column_banks_anor21 = round(np.float64((column_anor67+column_anor69-column_anor92)/1000000), 1)
# Получение значений uzum для Banks
column_banks_uzum1 = column_uzum2
column_banks_uzum2 = period
if column_uzum9 and column_uzum3 != 0:
    column_banks_uzum3 = round(np.float64(column_uzum9/column_uzum3 * 100), 1)
else:
    column_banks_uzum3 = 0
if column_uzum25 and column_uzum16 != 0:
    column_banks_uzum4 = round(np.float64(column_uzum25/column_uzum16 * 100), 1)
else:
    column_banks_uzum4 = 0
if column_uzum9 and column_uzum25 != 0:
    column_banks_uzum5 = round(np.float64(column_uzum9/column_uzum25 * 100), 1)
else: 
    column_banks_uzum5 = 0
if column_uzum65 and column_uzum53 != 0:
    column_banks_uzum6 = round(np.float64(column_uzum65/column_uzum53 * 100), 1)
else:
    column_banks_uzum6 = 0
if column_uzum67 and column_uzum37 != 0:
    column_banks_uzum7 = round(np.float64(column_uzum67/column_uzum37 * 100), 1)
else:
    column_banks_uzum7 = 0
if column_uzum40 and column_uzum56 and column_uzum89 != 0: 
    column_banks_uzum8 = round(np.float64(((column_uzum40-column_uzum56)*(days/365))/column_uzum89 * 100), 1)
else:
    column_banks_uzum8 = 0
if column_uzum65 != 0:
    column_banks_uzum9 = round(np.float64(column_uzum65/(column_uzum40-column_uzum56+column_uzum46-column_uzum62-column_uzum92) * 100), 1)
else:
    column_banks_uzum9 = 0
if column_uzum44 and column_uzum9 != 0 :
    column_banks_uzum10 = round(np.float64((column_uzum44*(days/365))/column_uzum9 * 100), 1)
else:
    column_banks_uzum10 = 0
if column_uzum60 and column_uzum25 != 0 :
    column_banks_uzum11 = round(np.float64((column_uzum60*(days/365))/column_uzum25 * 100), 1)
else:
    column_banks_uzum11 = 0
if column_uzum67 and column_uzum3 != 0 :    
    column_banks_uzum12 = round(np.float64((column_uzum67*(days/365))/column_uzum3 * 100), 1)
else:
    column_banks_uzum12 = 0
if column_uzum67 and column_uzum24 != 0 :
    column_banks_uzum13 = round(np.float64((column_uzum67*(days/365))/column_uzum24 * 100), 1)
else:
    column_banks_uzum12 = 0
if column_uzum77 != 0:
    column_banks_uzum14 = round(np.float64((((column_uzum66/1000)*0.87)*(days/365))/(column_uzum77*month)), 1)
else:
    column_banks_uzum14 = 0
if column_uzum9 != 0 :
    column_banks_uzum15 = round(np.float64(column_uzum9/(column_uzum19+column_uzum20+column_uzum21+column_uzum22+column_uzum23+column_uzum25) * 100), 1)
else:
    column_banks_uzum15 = 0
if column_uzum68 and column_uzum3 != 0 :
    column_banks_uzum16 = round(np.float64((column_uzum68*(days/365))/column_uzum3 * 100), 1)
else:
    column_banks_uzum16 = 0
if column_uzum77 != 0:
    column_banks_uzum17 = round(np.float64(((column_uzum67/1000)*(days/365))/(column_uzum77*month)), 1)
else:
    column_banks_uzum17 = 0
if column_uzum24 != 0:
    column_banks_uzum18 = round(np.float64((column_uzum68*(days/365))/column_uzum24 * 100), 1)
else:
    column_banks_uzum18 = 0
if column_uzum9 != 0:
    column_banks_uzum19 = round(np.float64((column_uzum12*(days/365))/column_uzum9 * 100), 1)
else:
    column_banks_uzum19 = 0
if column_uzum67 != 0:    
    column_banks_uzum20 = round(np.float64(column_uzum67/1000000), 1)
else:
    column_banks_uzum20 = 0
if column_uzum67 != 0:
    column_banks_uzum21 = round(np.float64((column_uzum67+column_uzum69-column_uzum92)/1000000), 1)
else:
    column_banks_uzum21 = 0
# Получение значений apex для Banks
column_banks_apex1 = "APEX"
column_banks_apex2 = period
if column_apex9 and column_apex3 != 0:
    column_banks_apex3 = round(np.float64(column_apex9/column_apex3 * 100), 1)
else:
    column_banks_apex3 = 0
if column_apex25 and column_apex16 != 0:
    column_banks_apex4 = round(np.float64(column_apex25/column_apex16 * 100), 1)
else:
    column_banks_apex4 = 0
if column_apex9 and column_apex25 != 0:
    column_banks_apex5 = round(np.float64(column_apex9/column_apex25 * 100), 1)
else: 
    column_banks_apex5 = 0
if column_apex65 and column_apex53 != 0:
    column_banks_apex6 = round(np.float64(column_apex65/column_apex53 * 100), 1)
else:
    column_banks_apex6 = 0
if column_apex67 and column_apex37 != 0:
    column_banks_apex7 = round(np.float64(column_apex67/column_apex37 * 100), 1)
else:
    column_banks_apex7 = 0
if column_apex40 and column_apex56 and column_apex89 != 0: 
    column_banks_apex8 = round(np.float64(((column_apex40-column_apex56)*(days/365))/column_apex89 * 100), 1)
else:
    column_banks_apex8 = 0
if column_apex65 != 0:
    column_banks_apex9 = round(np.float64(column_apex65/(column_apex40-column_apex56+column_apex46-column_apex62-column_apex92) * 100), 1)
else:
    column_banks_apex9 = 0
if column_apex44 and column_apex9 != 0 :
    column_banks_apex10 = round(np.float64((column_apex44*(days/365))/column_apex9 * 100), 1)
else:
    column_banks_apex10 = 0
if column_apex60 and column_apex25 != 0 :
    column_banks_apex11 = round(np.float64((column_apex60*(days/365))/column_apex25 * 100), 1)
else:
    column_banks_apex11 = 0
if column_apex67 and column_apex3 != 0 :    
    column_banks_apex12 = round(np.float64((column_apex67*(days/365))/column_apex3 * 100), 1)
else:
    column_banks_apex12 = 0
if column_apex67 and column_apex24 != 0 :
    column_banks_apex13 = round(np.float64((column_apex67*(days/365))/column_apex24 * 100), 1)
else:
    column_banks_apex12 = 0
if column_apex77 != 0:
    column_banks_apex14 = round(np.float64((((column_apex66/1000)*0.87)*(days/365))/(column_apex77*month)), 1)
else:
    column_banks_apex14 = 0
if column_apex9 != 0 :
    column_banks_apex15 = round(np.float64(column_apex9/(column_apex19+column_apex20+column_apex21+column_apex22+column_apex23+column_apex25) * 100), 1)
else:
    column_banks_apex15 = 0
if column_apex68 and column_apex3 != 0 :
    column_banks_apex16 = round(np.float64((column_apex68*(days/365))/column_apex3 * 100), 1)
else:
    column_banks_apex16 = 0
if column_apex77 != 0:
    column_banks_apex17 = round(np.float64(((column_apex67/1000)*(days/365))/(column_apex77*month)), 1)
else:
    column_banks_apex17 = 0
if column_apex24 != 0:
    column_banks_apex18 = round(np.float64((column_apex68*(days/365))/column_apex24 * 100), 1)
else:
    column_banks_apex18 = 0
if column_apex9 != 0:
    column_banks_apex19 = round(np.float64((column_apex12*(days/365))/column_apex9 * 100), 1)
else:
    column_banks_apex19 = 0
if column_apex67 != 0:    
    column_banks_apex20 = round(np.float64(column_apex67/1000000), 1)
else:
    column_banks_apex20 = 0
if column_apex67 != 0:
    column_banks_apex21 = round(np.float64((column_apex67+column_apex69-column_apex92)/1000000), 1)
else:
    column_banks_apex21 = 0

# Получение значений hayot для Banks
column_banks_hayot1 = "HAYOT"
column_banks_hayot2 = period
if column_hayot9 and column_hayot3 != 0:
    column_banks_hayot3 = round(np.float64(column_hayot9/column_hayot3 * 100), 1)
else:
    column_banks_hayot3 = 0
if column_hayot25 and column_hayot16 != 0:
    column_banks_hayot4 = round(np.float64(column_hayot25/column_hayot16 * 100), 1)
else:
    column_banks_hayot4 = 0
if column_hayot9 and column_hayot25 != 0:
    column_banks_hayot5 = round(np.float64(column_hayot9/column_hayot25 * 100), 1)
else: 
    column_banks_hayot5 = 0
if column_hayot65 and column_hayot53 != 0:
    column_banks_hayot6 = round(np.float64(column_hayot65/column_hayot53 * 100), 1)
else:
    column_banks_hayot6 = 0
if column_hayot67 and column_hayot37 != 0:
    column_banks_hayot7 = round(np.float64(column_hayot67/column_hayot37 * 100), 1)
else:
    column_banks_hayot7 = 0
if column_hayot40 and column_hayot56 and column_hayot89 != 0: 
    column_banks_hayot8 = round(np.float64(((column_hayot40-column_hayot56)*(days/365))/column_hayot89 * 100), 1)
else:
    column_banks_hayot8 = 0
if column_hayot65 != 0:
    column_banks_hayot9 = round(np.float64(column_hayot65/(column_hayot40-column_hayot56+column_hayot46-column_hayot62-column_hayot92) * 100), 1)
else:
    column_banks_hayot9 = 0
if column_hayot44 and column_hayot9 != 0 :
    column_banks_hayot10 = round(np.float64((column_hayot44*(days/365))/column_hayot9 * 100), 1)
else:
    column_banks_hayot10 = 0
if column_hayot60 and column_hayot25 != 0 :
    column_banks_hayot11 = round(np.float64((column_hayot60*(days/365))/column_hayot25 * 100), 1)
else:
    column_banks_hayot11 = 0
if column_hayot67 and column_hayot3 != 0 :    
    column_banks_hayot12 = round(np.float64((column_hayot67*(days/365))/column_hayot3 * 100), 1)
else:
    column_banks_hayot12 = 0
if column_hayot67 and column_hayot24 != 0 :
    column_banks_hayot13 = round(np.float64((column_hayot67*(days/365))/column_hayot24 * 100), 1)
else:
    column_banks_hayot12 = 0
if column_hayot77 != 0:
    column_banks_hayot14 = round(np.float64((((column_hayot66/1000)*0.87)*(days/365))/(column_hayot77*month)), 1)
else:
    column_banks_hayot14 = 0
if column_hayot9 != 0 :
    column_banks_hayot15 = round(np.float64(column_hayot9/(column_hayot19+column_hayot20+column_hayot21+column_hayot22+column_hayot23+column_hayot25) * 100), 1)
else:
    column_banks_hayot15 = 0
if column_hayot68 and column_hayot3 != 0 :
    column_banks_hayot16 = round(np.float64((column_hayot68*(days/365))/column_hayot3 * 100), 1)
else:
    column_banks_hayot16 = 0
if column_hayot77 != 0:
    column_banks_hayot17 = round(np.float64(((column_hayot67/1000)*(days/365))/(column_hayot77*month)), 1)
else:
    column_banks_hayot17 = 0
if column_hayot24 != 0:
    column_banks_hayot18 = round(np.float64((column_hayot68*(days/365))/column_hayot24 * 100), 1)
else:
    column_banks_hayot18 = 0
if column_hayot9 != 0:
    column_banks_hayot19 = round(np.float64((column_hayot12*(days/365))/column_hayot9 * 100), 1)
else:
    column_banks_hayot19 = 0
if column_hayot67 != 0:    
    column_banks_hayot20 = round(np.float64(column_hayot67/1000000), 1)
else:
    column_banks_hayot20 = 0
if column_hayot67 != 0:
    column_banks_hayot21 = round(np.float64((column_hayot67+column_hayot69-column_hayot92)/1000000), 1)
else:
    column_banks_hayot21 = 0

# Получение значений smart для Banks
column_banks_smart1 = "SMART"
column_banks_smart2 = period
if column_smart9 and column_smart3 != 0:
    column_banks_smart3 = round(np.float64(column_smart9/column_smart3 * 100), 1)
else:
    column_banks_smart3 = 0
if column_smart25 and column_smart16 != 0:
    column_banks_smart4 = round(np.float64(column_smart25/column_smart16 * 100), 1)
else:
    column_banks_smart4 = 0
if column_smart9 and column_smart25 != 0:
    column_banks_smart5 = round(np.float64(column_smart9/column_smart25 * 100), 1)
else: 
    column_banks_smart5 = 0
if column_smart65 and column_smart53 != 0:
    column_banks_smart6 = round(np.float64(column_smart65/column_smart53 * 100), 1)
else:
    column_banks_smart6 = 0
if column_smart67 and column_smart37 != 0:
    column_banks_smart7 = round(np.float64(column_smart67/column_smart37 * 100), 1)
else:
    column_banks_smart7 = 0
if column_smart40 and column_smart56 and column_smart89 != 0: 
    column_banks_smart8 = round(np.float64(((column_smart40-column_smart56)*(days/365))/column_smart89 * 100), 1)
else:
    column_banks_smart8 = 0
if column_smart65 != 0:
    column_banks_smart9 = round(np.float64(column_smart65/(column_smart40-column_smart56+column_smart46-column_smart62-column_smart92) * 100), 1)
else:
    column_banks_smart9 = 0
if column_smart44 and column_smart9 != 0 :
    column_banks_smart10 = round(np.float64((column_smart44*(days/365))/column_smart9 * 100), 1)
else:
    column_banks_smart10 = 0
if column_smart60 and column_smart25 != 0 :
    column_banks_smart11 = round(np.float64((column_smart60*(days/365))/column_smart25 * 100), 1)
else:
    column_banks_smart11 = 0
if column_smart67 and column_smart3 != 0 :    
    column_banks_smart12 = round(np.float64((column_smart67*(days/365))/column_smart3 * 100), 1)
else:
    column_banks_smart12 = 0
if column_smart67 and column_smart24 != 0 :
    column_banks_smart13 = round(np.float64((column_smart67*(days/365))/column_smart24 * 100), 1)
else:
    column_banks_smart12 = 0
if column_smart77 != 0:
    column_banks_smart14 = round(np.float64((((column_smart66/1000)*0.87)*(days/365))/(column_smart77*month)), 1)
else:
    column_banks_smart14 = 0
if column_smart9 != 0 :
    column_banks_smart15 = round(np.float64(column_smart9/(column_smart19+column_smart20+column_smart21+column_smart22+column_smart23+column_smart25) * 100), 1)
else:
    column_banks_smart15 = 0
if column_smart68 and column_smart3 != 0 :
    column_banks_smart16 = round(np.float64((column_smart68*(days/365))/column_smart3 * 100), 1)
else:
    column_banks_smart16 = 0
if column_smart77 != 0:
    column_banks_smart17 = round(np.float64(((column_smart67/1000)*(days/365))/(column_smart77*month)), 1)
else:
    column_banks_smart17 = 0
if column_smart24 != 0:
    column_banks_smart18 = round(np.float64((column_smart68*(days/365))/column_smart24 * 100), 1)
else:
    column_banks_smart18 = 0
if column_smart9 != 0:
    column_banks_smart19 = round(np.float64((column_smart12*(days/365))/column_smart9 * 100), 1)
else:
    column_banks_smart19 = 0
if column_smart67 != 0:    
    column_banks_smart20 = round(np.float64(column_smart67/1000000), 1)
else:
    column_banks_smart20 = 0
if column_smart67 != 0:
    column_banks_smart21 = round(np.float64((column_smart67+column_smart69-column_smart92)/1000000), 1)
else:
    column_banks_smart21 = 0

# Получение значений yangi для Banks
column_banks_yangi1 = "YANGI"
column_banks_yangi2 = period
if column_yangi9 and column_yangi3 != 0:
    column_banks_yangi3 = round(np.float64(column_yangi9/column_yangi3 * 100), 1)
else:
    column_banks_yangi3 = 0
if column_yangi25 and column_yangi16 != 0:
    column_banks_yangi4 = round(np.float64(column_yangi25/column_yangi16 * 100), 1)
else:
    column_banks_yangi4 = 0
if column_yangi9 and column_yangi25 != 0:
    column_banks_yangi5 = round(np.float64(column_yangi9/column_yangi25 * 100), 1)
else: 
    column_banks_yangi5 = 0
if column_yangi65 and column_yangi53 != 0:
    column_banks_yangi6 = round(np.float64(column_yangi65/column_yangi53 * 100), 1)
else:
    column_banks_yangi6 = 0
if column_yangi67 and column_yangi37 != 0:
    column_banks_yangi7 = round(np.float64(column_yangi67/column_yangi37 * 100), 1)
else:
    column_banks_yangi7 = 0
if column_yangi40 and column_yangi56 and column_yangi89 != 0: 
    column_banks_yangi8 = round(np.float64(((column_yangi40-column_yangi56)*(days/365))/column_yangi89 * 100), 1)
else:
    column_banks_yangi8 = 0
if column_yangi65 != 0:
    column_banks_yangi9 = round(np.float64(column_yangi65/(column_yangi40-column_yangi56+column_yangi46-column_yangi62-column_yangi92) * 100), 1)
else:
    column_banks_yangi9 = 0
if column_yangi44 and column_yangi9 != 0 :
    column_banks_yangi10 = round(np.float64((column_yangi44*(days/365))/column_yangi9 * 100), 1)
else:
    column_banks_yangi10 = 0
if column_yangi60 and column_yangi25 != 0 :
    column_banks_yangi11 = round(np.float64((column_yangi60*(days/365))/column_yangi25 * 100), 1)
else:
    column_banks_yangi11 = 0
if column_yangi67 and column_yangi3 != 0 :    
    column_banks_yangi12 = round(np.float64((column_yangi67*(days/365))/column_yangi3 * 100), 1)
else:
    column_banks_yangi12 = 0
if column_yangi67 and column_yangi24 != 0 :
    column_banks_yangi13 = round(np.float64((column_yangi67*(days/365))/column_yangi24 * 100), 1)
else:
    column_banks_yangi12 = 0
if column_yangi77 != 0:
    column_banks_yangi14 = round(np.float64((((column_yangi66/1000)*0.87)*(days/365))/(column_yangi77*month)), 1)
else:
    column_banks_yangi14 = 0
if column_yangi9 != 0 :
    column_banks_yangi15 = round(np.float64(column_yangi9/(column_yangi19+column_yangi20+column_yangi21+column_yangi22+column_yangi23+column_yangi25) * 100), 1)
else:
    column_banks_yangi15 = 0
if column_yangi68 and column_yangi3 != 0 :
    column_banks_yangi16 = round(np.float64((column_yangi68*(days/365))/column_yangi3 * 100), 1)
else:
    column_banks_yangi16 = 0
if column_yangi77 != 0:
    column_banks_yangi17 = round(np.float64(((column_yangi67/1000)*(days/365))/(column_yangi77*month)), 1)
else:
    column_banks_yangi17 = 0
if column_yangi24 != 0:
    column_banks_yangi18 = round(np.float64((column_yangi68*(days/365))/column_yangi24 * 100), 1)
else:
    column_banks_yangi18 = 0
if column_yangi9 != 0:
    column_banks_yangi19 = round(np.float64((column_yangi12*(days/365))/column_yangi9 * 100), 1)
else:
    column_banks_yangi19 = 0
if column_yangi67 != 0:    
    column_banks_yangi20 = round(np.float64(column_yangi67/1000000), 1)
else:
    column_banks_yangi20 = 0
if column_yangi67 != 0:
    column_banks_yangi21 = round(np.float64((column_yangi67+column_yangi69-column_yangi92)/1000000), 1)
else:
    column_banks_yangi21 = 0

# Получение значений hamkor для Banks
column_banks_hamkor1 = column_hamkor2
column_banks_hamkor2 = period
column_banks_hamkor3 = round(np.float64(column_hamkor9/column_hamkor3 * 100), 1)
column_banks_hamkor4 = round(np.float64(column_hamkor25/column_hamkor16 * 100), 1)
column_banks_hamkor5 = round(np.float64(column_hamkor9/column_hamkor25 * 100), 1)
column_banks_hamkor6 = round(np.float64(column_hamkor65/column_hamkor53 * 100), 1)
column_banks_hamkor7 = round(np.float64(column_hamkor67/column_hamkor37 * 100), 1)
column_banks_hamkor8 = round(np.float64(((column_hamkor40-column_hamkor56)*(days/365))/column_hamkor89 * 100), 1)
column_banks_hamkor9 = round(np.float64(column_hamkor65/(column_hamkor40-column_hamkor56+column_hamkor46-column_hamkor62-column_hamkor92) * 100), 1)
column_banks_hamkor10 = round(np.float64((column_hamkor44*(days/365))/column_hamkor9 * 100), 1)
column_banks_hamkor11 = round(np.float64((column_hamkor60*(days/365))/column_hamkor25 * 100), 1)
column_banks_hamkor12 = round(np.float64((column_hamkor67*(days/365))/column_hamkor3 * 100), 1)
column_banks_hamkor13 = round(np.float64((column_hamkor67*(days/365))/column_hamkor24 * 100), 1)
if column_hamkor77 != 0:
    column_banks_hamkor14 = round(np.float64((((column_hamkor66/1000)*0.87)*(days/365))/(column_hamkor77*month)), 1)
else:
    column_banks_hamkor14 = 0
column_banks_hamkor15 = round(np.float64(column_hamkor9/(column_hamkor19+column_hamkor20+column_hamkor21+column_hamkor22+column_hamkor23+column_hamkor25) * 100), 1)
column_banks_hamkor16 = round(np.float64((column_hamkor68*(days/365))/column_hamkor3 * 100), 1)
if column_hamkor77 != 0:
    column_banks_hamkor17 = round(np.float64(((column_hamkor67/1000)*(days/365))/(column_hamkor77*month)), 1)
else:
    column_banks_hamkor17 = 0
column_banks_hamkor18 = round(np.float64((column_hamkor68*(days/365))/column_hamkor24 * 100), 1)
column_banks_hamkor19 = round(np.float64((column_hamkor12*(days/365))/column_hamkor9 * 100), 1)
column_banks_hamkor20 = round(np.float64(column_hamkor67/1000000), 1)
column_banks_hamkor21 = round(np.float64((column_hamkor67+column_hamkor69-column_hamkor92)/1000000), 1)
# Получение значений kdb для Banks
column_banks_kdb1 = column_kdb2
column_banks_kdb2 = period
column_banks_kdb3 = round(np.float64(column_kdb9/column_kdb3 * 100), 1)
column_banks_kdb4 = round(np.float64(column_kdb25/column_kdb16 * 100), 1)
column_banks_kdb5 = round(np.float64(column_kdb9/column_kdb25 * 100), 1)
column_banks_kdb6 = round(np.float64(column_kdb65/column_kdb53 * 100), 1)
column_banks_kdb7 = round(np.float64(column_kdb67/column_kdb37 * 100), 1)
column_banks_kdb8 = round(np.float64(((column_kdb40-column_kdb56)*(days/365))/column_kdb89 * 100), 1)
column_banks_kdb9 = round(np.float64(column_kdb65/(column_kdb40-column_kdb56+column_kdb46-column_kdb62-column_kdb92) * 100), 1)
column_banks_kdb10 = round(np.float64((column_kdb44*(days/365))/column_kdb9 * 100), 1)
column_banks_kdb11 = round(np.float64((column_kdb60*(days/365))/column_kdb25 * 100), 1)
column_banks_kdb12 = round(np.float64((column_kdb67*(days/365))/column_kdb3 * 100), 1)
column_banks_kdb13 = round(np.float64((column_kdb67*(days/365))/column_kdb24 * 100), 1)
if column_kdb77 != 0:
    column_banks_kdb14 = round(np.float64((((column_kdb66/1000)*0.87)*(days/365))/(column_kdb77*month)), 1)
else:
    column_banks_kdb14 = 0
column_banks_kdb15 = round(np.float64(column_kdb9/(column_kdb19+column_kdb20+column_kdb21+column_kdb22+column_kdb23+column_kdb25) * 100), 1)
column_banks_kdb16 = round(np.float64((column_kdb68*(days/365))/column_kdb3 * 100), 1)
if column_kdb77 != 0:
    column_banks_kdb17 = round(np.float64(((column_kdb67/1000)*(days/365))/(column_kdb77*month)), 1)
else:
    column_banks_kdb17 = 0
column_banks_kdb18 = round(np.float64((column_kdb68*(days/365))/column_kdb24 * 100), 1)
column_banks_kdb19 = round(np.float64((column_kdb12*(days/365))/column_kdb9 * 100), 1)
column_banks_kdb20 = round(np.float64(column_kdb67/1000000), 1)
column_banks_kdb21 = round(np.float64((column_kdb67+column_kdb69-column_kdb92)/1000000), 1)
# Получение значений garant для Banks
column_banks_garant1 = column_garant2
column_banks_garant2 = period
column_banks_garant3 = round(np.float64(column_garant9/column_garant3 * 100), 1)
column_banks_garant4 = round(np.float64(column_garant25/column_garant16 * 100), 1)
column_banks_garant5 = round(np.float64(column_garant9/column_garant25 * 100), 1)
column_banks_garant6 = round(np.float64(column_garant65/column_garant53 * 100), 1)
column_banks_garant7 = round(np.float64(column_garant67/column_garant37 * 100), 1)
column_banks_garant8 = round(np.float64(((column_garant40-column_garant56)*(days/365))/column_garant89 * 100), 1)
column_banks_garant9 = round(np.float64(column_garant65/(column_garant40-column_garant56+column_garant46-column_garant62-column_garant92) * 100), 1)
column_banks_garant10 = round(np.float64((column_garant44*(days/365))/column_garant9 * 100), 1)
column_banks_garant11 = round(np.float64((column_garant60*(days/365))/column_garant25 * 100), 1)
column_banks_garant12 = round(np.float64((column_garant67*(days/365))/column_garant3 * 100), 1)
column_banks_garant13 = round(np.float64((column_garant67*(days/365))/column_garant24 * 100), 1)
if column_garant77 != 0:
    column_banks_garant14 = round(np.float64((((column_garant66/1000)*0.87)*(days/365))/(column_garant77*month)), 1)
else:
    column_banks_garant14 = 0
column_banks_garant15 = round(np.float64(column_garant9/(column_garant19+column_garant20+column_garant21+column_garant22+column_garant23+column_garant25) * 100), 1)
column_banks_garant16 = round(np.float64((column_garant68*(days/365))/column_garant3 * 100), 1)
if column_garant77 != 0:
    column_banks_garant17 = round(np.float64(((column_garant67/1000)*(days/365))/(column_garant77*month)), 1)
else:
    column_banks_garant17 = 0
column_banks_garant18 = round(np.float64((column_garant68*(days/365))/column_garant24 * 100), 1)
column_banks_garant19 = round(np.float64((column_garant12*(days/365))/column_garant9 * 100), 1)
column_banks_garant20 = round(np.float64(column_garant67/1000000), 1)
column_banks_garant21 = round(np.float64((column_garant67+column_garant69-column_garant92)/1000000), 1)
# Получение значений zirat для Banks
column_banks_zirat1 = column_zirat2
column_banks_zirat2 = period
column_banks_zirat3 = round(np.float64(column_zirat9/column_zirat3 * 100), 1)
column_banks_zirat4 = round(np.float64(column_zirat25/column_zirat16 * 100), 1)
column_banks_zirat5 = round(np.float64(column_zirat9/column_zirat25 * 100), 1)
column_banks_zirat6 = round(np.float64(column_zirat65/column_zirat53 * 100), 1)
column_banks_zirat7 = round(np.float64(column_zirat67/column_zirat37 * 100), 1)
column_banks_zirat8 = round(np.float64(((column_zirat40-column_zirat56)*(days/365))/column_zirat89 * 100), 1)
column_banks_zirat9 = round(np.float64(column_zirat65/(column_zirat40-column_zirat56+column_zirat46-column_zirat62-column_zirat92) * 100), 1)
column_banks_zirat10 = round(np.float64((column_zirat44*(days/365))/column_zirat9 * 100), 1)
column_banks_zirat11 = round(np.float64((column_zirat60*(days/365))/column_zirat25 * 100), 1)
column_banks_zirat12 = round(np.float64((column_zirat67*(days/365))/column_zirat3 * 100), 1)
column_banks_zirat13 = round(np.float64((column_zirat67*(days/365))/column_zirat24 * 100), 1)
if column_zirat77 != 0:
    column_banks_zirat14 = round(np.float64((((column_zirat66/1000)*0.87)*(days/365))/(column_zirat77*month)), 1)
else:
    column_banks_zirat14 = 0
column_banks_zirat15 = round(np.float64(column_zirat9/(column_zirat19+column_zirat20+column_zirat21+column_zirat22+column_zirat23+column_zirat25) * 100), 1)
column_banks_zirat16 = round(np.float64((column_zirat68*(days/365))/column_zirat3 * 100), 1)
if column_zirat77 != 0:
    column_banks_zirat17 = round(np.float64(((column_zirat67/1000)*(days/365))/(column_zirat77*month)), 1)
else:
    column_banks_zirat17 = 0
column_banks_zirat18 = round(np.float64((column_zirat68*(days/365))/column_zirat24 * 100), 1)
column_banks_zirat19 = round(np.float64((column_zirat12*(days/365))/column_zirat9 * 100), 1)
column_banks_zirat20 = round(np.float64(column_zirat67/1000000), 1)
column_banks_zirat21 = round(np.float64((column_zirat67+column_zirat69-column_zirat92)/1000000), 1)
# Получение значений saderat для Banks
column_banks_saderat1 = column_saderat2
column_banks_saderat2 = period
column_banks_saderat3 = round(np.float64(column_saderat9/column_saderat3 * 100), 1)
column_banks_saderat4 = round(np.float64(column_saderat25/column_saderat16 * 100), 1)
column_banks_saderat5 = round(np.float64(column_saderat9/column_saderat25 * 100), 1)
column_banks_saderat6 = round(np.float64(column_saderat65/column_saderat53 * 100), 1)
column_banks_saderat7 = round(np.float64(column_saderat67/column_saderat37 * 100), 1)
column_banks_saderat8 = round(np.float64(((column_saderat40-column_saderat56)*(days/365))/column_saderat89 * 100), 1)
column_banks_saderat9 = round(np.float64(column_saderat65/(column_saderat40-column_saderat56+column_saderat46-column_saderat62-column_saderat92) * 100), 1)
column_banks_saderat10 = round(np.float64((column_saderat44*(days/365))/column_saderat9 * 100), 1)
column_banks_saderat11 = round(np.float64((column_saderat60*(days/365))/column_saderat25 * 100), 1)
column_banks_saderat12 = round(np.float64((column_saderat67*(days/365))/column_saderat3 * 100), 1)
column_banks_saderat13 = round(np.float64((column_saderat67*(days/365))/column_saderat24 * 100), 1)
if column_saderat77 != 0:
    column_banks_saderat14 = round(np.float64((((column_saderat66/1000)*0.87)*(days/365))/(column_saderat77*month)), 1)
else:
    column_banks_saderat14 = 0
column_banks_saderat15 = round(np.float64(column_saderat9/(column_saderat19+column_saderat20+column_saderat21+column_saderat22+column_saderat23+column_saderat25) * 100), 1)
column_banks_saderat16 = round(np.float64((column_saderat68*(days/365))/column_saderat3 * 100), 1)
if column_saderat77 != 0:
    column_banks_saderat17 = round(np.float64(((column_saderat67/1000)*(days/365))/(column_saderat77*month)), 1)
else:
    column_banks_saderat17 = 0
column_banks_saderat18 = round(np.float64((column_saderat68*(days/365))/column_saderat24 * 100), 1)
column_banks_saderat19 = round(np.float64((column_saderat12*(days/365))/column_saderat9 * 100), 1)
column_banks_saderat20 = round(np.float64(column_saderat67/1000000), 1)
column_banks_saderat21 = round(np.float64((column_saderat67+column_saderat69-column_saderat92)/1000000), 1)
# Получение значений tbc для Banks
column_banks_tbc1 = column_tbc2
column_banks_tbc2 = period
column_banks_tbc3 = round(np.float64(column_tbc9/column_tbc3 * 100), 1)
column_banks_tbc4 = round(np.float64(column_tbc25/column_tbc16 * 100), 1)
column_banks_tbc5 = round(np.float64(column_tbc9/column_tbc25 * 100), 1)
column_banks_tbc6 = round(np.float64(column_tbc65/column_tbc53 * 100), 1)
column_banks_tbc7 = round(np.float64(column_tbc67/column_tbc37 * 100), 1)
column_banks_tbc8 = round(np.float64(((column_tbc40-column_tbc56)*(days/365))/column_tbc89 * 100), 1)
column_banks_tbc9 = round(np.float64(column_tbc65/(column_tbc40-column_tbc56+column_tbc46-column_tbc62-column_tbc92) * 100), 1)
column_banks_tbc10 = round(np.float64((column_tbc44*(days/365))/column_tbc9 * 100), 1)
column_banks_tbc11 = round(np.float64((column_tbc60*(days/365))/column_tbc25 * 100), 1)
column_banks_tbc12 = round(np.float64((column_tbc67*(days/365))/column_tbc3 * 100), 1)
column_banks_tbc13 = round(np.float64((column_tbc67*(days/365))/column_tbc24 * 100), 1)
if column_tbc77 != 0:
    column_banks_tbc14 = round(np.float64((((column_tbc66/1000)*0.87)*(days/365))/(column_tbc77*month)), 1)
else:
    column_banks_tbc14 = 0
column_banks_tbc15 = round(np.float64(column_tbc9/(column_tbc19+column_tbc20+column_tbc21+column_tbc22+column_tbc23+column_tbc25) * 100), 1)
column_banks_tbc16 = round(np.float64((column_tbc68*(days/365))/column_tbc3 * 100), 1)
if column_tbc77 != 0:
    column_banks_tbc17 = round(np.float64(((column_tbc67/1000)*(days/365))/(column_tbc77*month)), 1)
else:
    column_banks_tbc17 = 0
column_banks_tbc18 = round(np.float64((column_tbc68*(days/365))/column_tbc24 * 100), 1)
column_banks_tbc19 = round(np.float64((column_tbc12*(days/365))/column_tbc9 * 100), 1)
column_banks_tbc20 = round(np.float64(column_tbc67/1000000), 1)
column_banks_tbc21 = round(np.float64((column_tbc67+column_tbc69-column_tbc92)/1000000), 1)
# Получение значений tenge для Banks
column_banks_tenge1 = column_tenge2
column_banks_tenge2 = period
column_banks_tenge3 = round(np.float64(column_tenge9/column_tenge3 * 100), 1)
column_banks_tenge4 = round(np.float64(column_tenge25/column_tenge16 * 100), 1)
column_banks_tenge5 = round(np.float64(column_tenge9/column_tenge25 * 100), 1)
column_banks_tenge6 = round(np.float64(column_tenge65/column_tenge53 * 100), 1)
column_banks_tenge7 = round(np.float64(column_tenge67/column_tenge37 * 100), 1)
column_banks_tenge8 = round(np.float64(((column_tenge40-column_tenge56)*(days/365))/column_tenge89 * 100), 1)
column_banks_tenge9 = round(np.float64(column_tenge65/(column_tenge40-column_tenge56+column_tenge46-column_tenge62-column_tenge92) * 100), 1)
column_banks_tenge10 = round(np.float64((column_tenge44*(days/365))/column_tenge9 * 100), 1)
column_banks_tenge11 = round(np.float64((column_tenge60*(days/365))/column_tenge25 * 100), 1)
column_banks_tenge12 = round(np.float64((column_tenge67*(days/365))/column_tenge3 * 100), 1)
column_banks_tenge13 = round(np.float64((column_tenge67*(days/365))/column_tenge24 * 100), 1)
if column_tenge77 != 0:
    column_banks_tenge14 = round(np.float64((((column_tenge66/1000)*0.87)*(days/365))/(column_tenge77*month)), 1)
else:
    column_banks_tenge14 = 0
column_banks_tenge15 = round(np.float64(column_tenge9/(column_tenge19+column_tenge20+column_tenge21+column_tenge22+column_tenge23+column_tenge25) * 100), 1)
column_banks_tenge16 = round(np.float64((column_tenge68*(days/365))/column_tenge3 * 100), 1)
if column_tenge77 != 0:
    column_banks_tenge17 = round(np.float64(((column_tenge67/1000)*(days/365))/(column_tenge77*month)), 1)
else:
    column_banks_tenge17 = 0
column_banks_tenge18 = round(np.float64((column_tenge68*(days/365))/column_tenge24 * 100), 1)
column_banks_tenge19 = round(np.float64((column_tenge12*(days/365))/column_tenge9 * 100), 1)
column_banks_tenge20 = round(np.float64(column_tenge67/1000000), 1)
column_banks_tenge21 = round(np.float64((column_tenge67+column_tenge69-column_tenge92)/1000000), 1)

# Подключение к базе данных
conn = sqlite3.connect('bank_analytics.db')
cursor = conn.cursor()
# Вызов функции для вставки данных в таблицу Banks - nbu
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_nbu1, column_banks_nbu2, column_banks_nbu3, column_banks_nbu4, column_banks_nbu5, column_banks_nbu6, column_banks_nbu7, column_banks_nbu8, column_banks_nbu9, column_banks_nbu10, column_banks_nbu11, column_banks_nbu12, column_banks_nbu13, column_banks_nbu14, column_banks_nbu15, column_banks_nbu16, column_banks_nbu17, column_banks_nbu18, column_banks_nbu19, column_banks_nbu20,
column_banks_nbu21))
# Вызов функции для вставки данных в таблицу Banks - aab
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_aab1, column_banks_aab2, column_banks_aab3, column_banks_aab4, column_banks_aab5, column_banks_aab6, column_banks_aab7, column_banks_aab8, column_banks_aab9, column_banks_aab10, column_banks_aab11, column_banks_aab12, column_banks_aab13, column_banks_aab14, column_banks_aab15, column_banks_aab16, column_banks_aab17, column_banks_aab18, column_banks_aab19, column_banks_aab20,
column_banks_aab21))    
# Вызов функции для вставки данных в таблицу Banks - asaka
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_asaka1, column_banks_asaka2, column_banks_asaka3, column_banks_asaka4, column_banks_asaka5, column_banks_asaka6, column_banks_asaka7, column_banks_asaka8, column_banks_asaka9, column_banks_asaka10, column_banks_asaka11, column_banks_asaka12, column_banks_asaka13, column_banks_asaka14, column_banks_asaka15, column_banks_asaka16, column_banks_asaka17, column_banks_asaka18, column_banks_asaka19, column_banks_asaka20,
column_banks_asaka21))
# Вызов функции для вставки данных в таблицу Banks - xb
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_xb1, column_banks_xb2, column_banks_xb3, column_banks_xb4, column_banks_xb5, column_banks_xb6, column_banks_xb7, column_banks_xb8, column_banks_xb9, column_banks_xb10, column_banks_xb11, column_banks_xb12, column_banks_xb13, column_banks_xb14, column_banks_xb15, column_banks_xb16, column_banks_xb17, column_banks_xb18, column_banks_xb19, column_banks_xb20,
column_banks_xb21))
# Вызов функции для вставки данных в таблицу Banks - poytaxt
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_poytaxt1, column_banks_poytaxt2, column_banks_poytaxt3, column_banks_poytaxt4, column_banks_poytaxt5, column_banks_poytaxt6, column_banks_poytaxt7, column_banks_poytaxt8, column_banks_poytaxt9, column_banks_poytaxt10, column_banks_poytaxt11, column_banks_poytaxt12, column_banks_poytaxt13, column_banks_poytaxt14, column_banks_poytaxt15, column_banks_poytaxt16, column_banks_poytaxt17, column_banks_poytaxt18, column_banks_poytaxt19, column_banks_poytaxt20,
column_banks_poytaxt21))
# Вызов функции для вставки данных в таблицу Banks - sqb
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_sqb1, column_banks_sqb2, column_banks_sqb3, column_banks_sqb4, column_banks_sqb5, column_banks_sqb6, column_banks_sqb7, column_banks_sqb8, column_banks_sqb9, column_banks_sqb10, column_banks_sqb11, column_banks_sqb12, column_banks_sqb13, column_banks_sqb14, column_banks_sqb15, column_banks_sqb16, column_banks_sqb17, column_banks_sqb18, column_banks_sqb19, column_banks_sqb20,
column_banks_sqb21))
# Вызов функции для вставки данных в таблицу Banks - ipoteka
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_ipoteka1, column_banks_ipoteka2, column_banks_ipoteka3, column_banks_ipoteka4, column_banks_ipoteka5, column_banks_ipoteka6, column_banks_ipoteka7, column_banks_ipoteka8, column_banks_ipoteka9, column_banks_ipoteka10, column_banks_ipoteka11, column_banks_ipoteka12, column_banks_ipoteka13, column_banks_ipoteka14, column_banks_ipoteka15, column_banks_ipoteka16, column_banks_ipoteka17, column_banks_ipoteka18, column_banks_ipoteka19, column_banks_ipoteka20,
column_banks_ipoteka21))
# Вызов функции для вставки данных в таблицу Banks - agro
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_agro1, column_banks_agro2, column_banks_agro3, column_banks_agro4, column_banks_agro5, column_banks_agro6, column_banks_agro7, column_banks_agro8, column_banks_agro9, column_banks_agro10, column_banks_agro11, column_banks_agro12, column_banks_agro13, column_banks_agro14, column_banks_agro15, column_banks_agro16, column_banks_agro17, column_banks_agro18, column_banks_agro19, column_banks_agro20,
column_banks_agro21))
# Вызов функции для вставки данных в таблицу Banks - qqb
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_qqb1, column_banks_qqb2, column_banks_qqb3, column_banks_qqb4, column_banks_qqb5, column_banks_qqb6, column_banks_qqb7, column_banks_qqb8, column_banks_qqb9, column_banks_qqb10, column_banks_qqb11, column_banks_qqb12, column_banks_qqb13, column_banks_qqb14, column_banks_qqb15, column_banks_qqb16, column_banks_qqb17, column_banks_qqb18, column_banks_qqb19, column_banks_qqb20,
column_banks_qqb21))
# Вызов функции для вставки данных в таблицу Banks - turon
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_turon1, column_banks_turon2, column_banks_turon3, column_banks_turon4, column_banks_turon5, column_banks_turon6, column_banks_turon7, column_banks_turon8, column_banks_turon9, column_banks_turon10, column_banks_turon11, column_banks_turon12, column_banks_turon13, column_banks_turon14, column_banks_turon15, column_banks_turon16, column_banks_turon17, column_banks_turon18, column_banks_turon19, column_banks_turon20,
column_banks_turon21))
# Вызов функции для вставки данных в таблицу Banks - micro
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_micro1, column_banks_micro2, column_banks_micro3, column_banks_micro4, column_banks_micro5, column_banks_micro6, column_banks_micro7, column_banks_micro8, column_banks_micro9, column_banks_micro10, column_banks_micro11, column_banks_micro12, column_banks_micro13, column_banks_micro14, column_banks_micro15, column_banks_micro16, column_banks_micro17, column_banks_micro18, column_banks_micro19, column_banks_micro20,
column_banks_micro21))
# Вызов функции для вставки данных в таблицу Banks - aloqa
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_aloqa1, column_banks_aloqa2, column_banks_aloqa3, column_banks_aloqa4, column_banks_aloqa5, column_banks_aloqa6, column_banks_aloqa7, column_banks_aloqa8, column_banks_aloqa9, column_banks_aloqa10, column_banks_aloqa11, column_banks_aloqa12, column_banks_aloqa13, column_banks_aloqa14, column_banks_aloqa15, column_banks_aloqa16, column_banks_aloqa17, column_banks_aloqa18, column_banks_aloqa19, column_banks_aloqa20,
column_banks_aloqa21))
# Вызов функции для вставки данных в таблицу Banks - ipak
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_ipak1, column_banks_ipak2, column_banks_ipak3, column_banks_ipak4, column_banks_ipak5, column_banks_ipak6, column_banks_ipak7, column_banks_ipak8, column_banks_ipak9, column_banks_ipak10, column_banks_ipak11, column_banks_ipak12, column_banks_ipak13, column_banks_ipak14, column_banks_ipak15, column_banks_ipak16, column_banks_ipak17, column_banks_ipak18, column_banks_ipak19, column_banks_ipak20,
column_banks_ipak21))
# Вызов функции для вставки данных в таблицу Banks - kapital
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_kapital1, column_banks_kapital2, column_banks_kapital3, column_banks_kapital4, column_banks_kapital5, column_banks_kapital6, column_banks_kapital7, column_banks_kapital8, column_banks_kapital9, column_banks_kapital10, column_banks_kapital11, column_banks_kapital12, column_banks_kapital13, column_banks_kapital14, column_banks_kapital15, column_banks_kapital16, column_banks_kapital17, column_banks_kapital18, column_banks_kapital19, column_banks_kapital20,
column_banks_kapital21))
# Вызов функции для вставки данных в таблицу Banks - infin
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_infin1, column_banks_infin2, column_banks_infin3, column_banks_infin4, column_banks_infin5, column_banks_infin6, column_banks_infin7, column_banks_infin8, column_banks_infin9, column_banks_infin10, column_banks_infin11, column_banks_infin12, column_banks_infin13, column_banks_infin14, column_banks_infin15, column_banks_infin16, column_banks_infin17, column_banks_infin18, column_banks_infin19, column_banks_infin20,
column_banks_infin21))
# Вызов функции для вставки данных в таблицу Banks - universal
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_universal1, column_banks_universal2, column_banks_universal3, column_banks_universal4, column_banks_universal5, column_banks_universal6, column_banks_universal7, column_banks_universal8, column_banks_universal9, column_banks_universal10, column_banks_universal11, column_banks_universal12, column_banks_universal13, column_banks_universal14, column_banks_universal15, column_banks_universal16, column_banks_universal17, column_banks_universal18, column_banks_universal19, column_banks_universal20,
column_banks_universal21))
# Вызов функции для вставки данных в таблицу Banks - avo
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_avo1, column_banks_avo2, column_banks_avo3, column_banks_avo4, column_banks_avo5, column_banks_avo6, column_banks_avo7, column_banks_avo8, column_banks_avo9, column_banks_avo10, column_banks_avo11, column_banks_avo12, column_banks_avo13, column_banks_avo14, column_banks_avo15, column_banks_avo16, column_banks_avo17, column_banks_avo18, column_banks_avo19, column_banks_avo20,
column_banks_avo21))
# Вызов функции для вставки данных в таблицу Banks - madad
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_madad1, column_banks_madad2, column_banks_madad3, column_banks_madad4, column_banks_madad5, column_banks_madad6, column_banks_madad7, column_banks_madad8, column_banks_madad9, column_banks_madad10, column_banks_madad11, column_banks_madad12, column_banks_madad13, column_banks_madad14, column_banks_madad15, column_banks_madad16, column_banks_madad17, column_banks_madad18, column_banks_madad19, column_banks_madad20,
column_banks_madad21))
# Вызов функции для вставки данных в таблицу Banks - ofb
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_ofb1, column_banks_ofb2, column_banks_ofb3, column_banks_ofb4, column_banks_ofb5, column_banks_ofb6, column_banks_ofb7, column_banks_ofb8, column_banks_ofb9, column_banks_ofb10, column_banks_ofb11, column_banks_ofb12, column_banks_ofb13, column_banks_ofb14, column_banks_ofb15, column_banks_ofb16, column_banks_ofb17, column_banks_ofb18, column_banks_ofb19, column_banks_ofb20,
column_banks_ofb21))
# Вызов функции для вставки данных в таблицу Banks - trust
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_trust1, column_banks_trust2, column_banks_trust3, column_banks_trust4, column_banks_trust5, column_banks_trust6, column_banks_trust7, column_banks_trust8, column_banks_trust9, column_banks_trust10, column_banks_trust11, column_banks_trust12, column_banks_trust13, column_banks_trust14, column_banks_trust15, column_banks_trust16, column_banks_trust17, column_banks_trust18, column_banks_trust19, column_banks_trust20,
column_banks_trust21))
# Вызов функции для вставки данных в таблицу Banks - davr
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_davr1, column_banks_davr2, column_banks_davr3, column_banks_davr4, column_banks_davr5, column_banks_davr6, column_banks_davr7, column_banks_davr8, column_banks_davr9, column_banks_davr10, column_banks_davr11, column_banks_davr12, column_banks_davr13, column_banks_davr14, column_banks_davr15, column_banks_davr16, column_banks_davr17, column_banks_davr18, column_banks_davr19, column_banks_davr20,
column_banks_davr21))
# Вызов функции для вставки данных в таблицу Banks - ravnaq
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_ravnaq1, column_banks_ravnaq2, column_banks_ravnaq3, column_banks_ravnaq4, column_banks_ravnaq5, column_banks_ravnaq6, column_banks_ravnaq7, column_banks_ravnaq8, column_banks_ravnaq9, column_banks_ravnaq10, column_banks_ravnaq11, column_banks_ravnaq12, column_banks_ravnaq13, column_banks_ravnaq14, column_banks_ravnaq15, column_banks_ravnaq16, column_banks_ravnaq17, column_banks_ravnaq18, column_banks_ravnaq19, column_banks_ravnaq20,
column_banks_ravnaq21))
    # Вызов функции для вставки данных в таблицу Banks - anor
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_anor1, column_banks_anor2, column_banks_anor3, column_banks_anor4, column_banks_anor5, column_banks_anor6, column_banks_anor7, column_banks_anor8, column_banks_anor9, column_banks_anor10, column_banks_anor11, column_banks_anor12, column_banks_anor13, column_banks_anor14, column_banks_anor15, column_banks_anor16, column_banks_anor17, column_banks_anor18, column_banks_anor19, column_banks_anor20,
column_banks_anor21))
# Вызов функции для вставки данных в таблицу Banks - uzum
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_uzum1, column_banks_uzum2, column_banks_uzum3, column_banks_uzum4, column_banks_uzum5, column_banks_uzum6, column_banks_uzum7, column_banks_uzum8, column_banks_uzum9, column_banks_uzum10, column_banks_uzum11, column_banks_uzum12, column_banks_uzum13, column_banks_uzum14, column_banks_uzum15, column_banks_uzum16, column_banks_uzum17, column_banks_uzum18, column_banks_uzum19, column_banks_uzum20,
column_banks_uzum21))
# Вызов функции для вставки данных в таблицу Banks - apex
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_apex1, column_banks_apex2, column_banks_apex3, column_banks_apex4, column_banks_apex5, column_banks_apex6, column_banks_apex7, column_banks_apex8, column_banks_apex9, column_banks_apex10, column_banks_apex11, column_banks_apex12, column_banks_apex13, column_banks_apex14, column_banks_apex15, column_banks_apex16, column_banks_apex17, column_banks_apex18, column_banks_apex19, column_banks_apex20,
column_banks_apex21))
# Вызов функции для вставки данных в таблицу Banks - hayot
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_hayot1, column_banks_hayot2, column_banks_hayot3, column_banks_hayot4, column_banks_hayot5, column_banks_hayot6, column_banks_hayot7, column_banks_hayot8, column_banks_hayot9, column_banks_hayot10, column_banks_hayot11, column_banks_hayot12, column_banks_hayot13, column_banks_hayot14, column_banks_hayot15, column_banks_hayot16, column_banks_hayot17, column_banks_hayot18, column_banks_hayot19, column_banks_hayot20,
column_banks_hayot21))
# Вызов функции для вставки данных в таблицу Banks - smart
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_smart1, column_banks_smart2, column_banks_smart3, column_banks_smart4, column_banks_smart5, column_banks_smart6, column_banks_smart7, column_banks_smart8, column_banks_smart9, column_banks_smart10, column_banks_smart11, column_banks_smart12, column_banks_smart13, column_banks_smart14, column_banks_smart15, column_banks_smart16, column_banks_smart17, column_banks_smart18, column_banks_smart19, column_banks_smart20,
column_banks_smart21))
# Вызов функции для вставки данных в таблицу Banks - yangi
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_yangi1, column_banks_yangi2, column_banks_yangi3, column_banks_yangi4, column_banks_yangi5, column_banks_yangi6, column_banks_yangi7, column_banks_yangi8, column_banks_yangi9, column_banks_yangi10, column_banks_yangi11, column_banks_yangi12, column_banks_yangi13, column_banks_yangi14, column_banks_yangi15, column_banks_yangi16, column_banks_yangi17, column_banks_yangi18, column_banks_yangi19, column_banks_yangi20,
column_banks_yangi21))
# Вызов функции для вставки данных в таблицу Banks - hamkor
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_hamkor1, column_banks_hamkor2, column_banks_hamkor3, column_banks_hamkor4, column_banks_hamkor5, column_banks_hamkor6, column_banks_hamkor7, column_banks_hamkor8, column_banks_hamkor9, column_banks_hamkor10, column_banks_hamkor11, column_banks_hamkor12, column_banks_hamkor13, column_banks_hamkor14, column_banks_hamkor15, column_banks_hamkor16, column_banks_hamkor17, column_banks_hamkor18, column_banks_hamkor19, column_banks_hamkor20,
column_banks_hamkor21))
# Вызов функции для вставки данных в таблицу Banks - kdb
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_kdb1, column_banks_kdb2, column_banks_kdb3, column_banks_kdb4, column_banks_kdb5, column_banks_kdb6, column_banks_kdb7, column_banks_kdb8, column_banks_kdb9, column_banks_kdb10, column_banks_kdb11, column_banks_kdb12, column_banks_kdb13, column_banks_kdb14, column_banks_kdb15, column_banks_kdb16, column_banks_kdb17, column_banks_kdb18, column_banks_kdb19, column_banks_kdb20,
column_banks_kdb21))
# Вызов функции для вставки данных в таблицу Banks - garant
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_garant1, column_banks_garant2, column_banks_garant3, column_banks_garant4, column_banks_garant5, column_banks_garant6, column_banks_garant7, column_banks_garant8, column_banks_garant9, column_banks_garant10, column_banks_garant11, column_banks_garant12, column_banks_garant13, column_banks_garant14, column_banks_garant15, column_banks_garant16, column_banks_garant17, column_banks_garant18, column_banks_garant19, column_banks_garant20,
column_banks_garant21))
# Вызов функции для вставки данных в таблицу Banks - zirat
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_zirat1, column_banks_zirat2, column_banks_zirat3, column_banks_zirat4, column_banks_zirat5, column_banks_zirat6, column_banks_zirat7, column_banks_zirat8, column_banks_zirat9, column_banks_zirat10, column_banks_zirat11, column_banks_zirat12, column_banks_zirat13, column_banks_zirat14, column_banks_zirat15, column_banks_zirat16, column_banks_zirat17, column_banks_zirat18, column_banks_zirat19, column_banks_zirat20,
column_banks_zirat21))
# Вызов функции для вставки данных в таблицу Banks - saderat
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_saderat1, column_banks_saderat2, column_banks_saderat3, column_banks_saderat4, column_banks_saderat5, column_banks_saderat6, column_banks_saderat7, column_banks_saderat8, column_banks_saderat9, column_banks_saderat10, column_banks_saderat11, column_banks_saderat12, column_banks_saderat13, column_banks_saderat14, column_banks_saderat15, column_banks_saderat16, column_banks_saderat17, column_banks_saderat18, column_banks_saderat19, column_banks_saderat20,
column_banks_saderat21))
# Вызов функции для вставки данных в таблицу Banks - tbc
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_tbc1, column_banks_tbc2, column_banks_tbc3, column_banks_tbc4, column_banks_tbc5, column_banks_tbc6, column_banks_tbc7, column_banks_tbc8, column_banks_tbc9, column_banks_tbc10, column_banks_tbc11, column_banks_tbc12, column_banks_tbc13, column_banks_tbc14, column_banks_tbc15, column_banks_tbc16, column_banks_tbc17, column_banks_tbc18, column_banks_tbc19, column_banks_tbc20,
column_banks_tbc21))
# Вызов функции для вставки данных в таблицу Banks - tenge
insert_data_into_banks(conn, cursor, 'Banks', (column_banks_tenge1, column_banks_tenge2, column_banks_tenge3, column_banks_tenge4, column_banks_tenge5, column_banks_tenge6, column_banks_tenge7, column_banks_tenge8, column_banks_tenge9, column_banks_tenge10, column_banks_tenge11, column_banks_tenge12, column_banks_tenge13, column_banks_tenge14, column_banks_tenge15, column_banks_tenge16, column_banks_tenge17, column_banks_tenge18, column_banks_tenge19, column_banks_tenge20,
column_banks_tenge21))
# Вызов функции для вставки данных в таблицу NBU
insert_data_into_table(conn, cursor, 'NBU', (
column_nbu2, 
column_nbu1, 
column_nbu3, 
column_nbu4, 
column_nbu5, 
column_nbu6, 
column_nbu7, 
column_nbu8, 
column_nbu9, 
column_nbu10, 
column_nbu11, 
column_nbu12,     
column_nbu13, 
column_nbu14, 
column_nbu15, 
column_nbu16, 
column_nbu17, 
column_nbu18, 
column_nbu19,
column_nbu20, 
column_nbu21,
column_nbu22,
column_nbu23,
column_nbu24,
column_nbu25,
column_nbu26,
column_nbu27,
column_nbu28,
column_nbu29,
column_nbu30,
column_nbu31,
column_nbu32,
column_nbu33,
column_nbu34,
column_nbu35,
column_nbu36,
column_nbu37,
column_nbu38,
column_nbu39,
column_nbu40,
column_nbu41,
column_nbu42,
column_nbu43,
column_nbu44,
column_nbu45,
column_nbu46,
column_nbu47,
column_nbu48,
column_nbu49,
column_nbu50,
column_nbu51,
column_nbu52,
column_nbu53,
column_nbu54,
column_nbu55,
column_nbu56,
column_nbu57,
column_nbu58,
column_nbu59,
column_nbu60,
column_nbu61,
column_nbu62,
column_nbu63,
column_nbu64,
column_nbu65,
column_nbu66,
column_nbu67,
column_nbu68,
column_nbu69,
column_nbu70,
column_nbu71,
column_nbu72,
column_nbu73,
column_nbu74,
column_nbu75,
column_nbu76,
column_nbu77,
column_nbu78,
column_nbu79,
column_nbu83,
column_nbu84,
column_nbu85,
column_nbu89,
column_nbu90,
column_nbu91,
column_nbu92))
# Вызов функции для вставки данных в таблицу Asia  
insert_data_into_table(conn, cursor, 'ASIAALIANCE', (column_aab2, column_aab1, column_aab3, column_aab4, column_aab5, column_aab6, column_aab7, column_aab8, column_aab9, column_aab10, column_aab11, column_aab12, column_aab13, column_aab14, column_aab15, column_aab16, column_aab17, column_aab18, column_aab19, 
column_aab20,
column_aab21,
column_aab22,
column_aab23,
column_aab24,
column_aab25,
column_aab26,
column_aab27,
column_aab28,
column_aab29,
column_aab30,
column_aab31,
column_aab32,
column_aab33,
column_aab34,
column_aab35,
column_aab36,
column_aab37,
column_aab38,
column_aab39,
column_aab40,
column_aab41,
column_aab42,
column_aab43,
column_aab44,
column_aab45,
column_aab46,
column_aab47,
column_aab48,
column_aab49,
column_aab50,
column_aab51,
column_aab52,
column_aab53,
column_aab54,
column_aab55,
column_aab56,
column_aab57,
column_aab58,
column_aab59,
column_aab60,
column_aab61,
column_aab62,
column_aab63,
column_aab64,
column_aab65,
column_aab66,
column_aab67,
column_aab68,
column_aab69,
column_aab70,
column_aab71,
column_aab72,
column_aab73,
column_aab74,
column_aab75,
column_aab76,
column_aab77,
column_aab78,
column_aab79,
column_aab83,
column_aab84,
column_aab85,
column_aab89,
column_aab90,
column_aab91,
column_aab92))
# Вызов функции для вставки данных в таблицу Asaka  
insert_data_into_table(conn, cursor, 'ASAKA', (column_asaka2, column_asaka1, column_asaka3, column_asaka4, column_asaka5, column_asaka6, column_asaka7, column_asaka8, column_asaka9, column_asaka10, column_asaka11, column_asaka12, column_asaka13, column_asaka14, column_asaka15, column_asaka16, column_asaka17, column_asaka18, column_asaka19, 
column_asaka20,
column_asaka21,
column_asaka22,
column_asaka23,
column_asaka24,
column_asaka25,
column_asaka26,
column_asaka27,
column_asaka28,
column_asaka29,
column_asaka30,
column_asaka31,
column_asaka32,
column_asaka33,
column_asaka34,
column_asaka35,
column_asaka36,
column_asaka37,
column_asaka38,
column_asaka39,
column_asaka40,
column_asaka41,
column_asaka42,
column_asaka43,
column_asaka44,
column_asaka45,
column_asaka46,
column_asaka47,
column_asaka48,
column_asaka49,
column_asaka50,
column_asaka51,
column_asaka52,
column_asaka53,
column_asaka54,
column_asaka55,
column_asaka56,
column_asaka57,
column_asaka58,
column_asaka59,
column_asaka60,
column_asaka61,
column_asaka62,
column_asaka63,
column_asaka64,
column_asaka65,
column_asaka66,
column_asaka67,
column_asaka68,
column_asaka69,
column_asaka70,
column_asaka71,
column_asaka72,
column_asaka73,
column_asaka74,
column_asaka75,
column_asaka76,
column_asaka77,
column_asaka78,
column_asaka79,
column_asaka83,
column_asaka84,
column_asaka85,
column_asaka89,
column_asaka90,
column_asaka91,
column_asaka92))
# Вызов функции для вставки данных в таблицу XB  
insert_data_into_table(conn, cursor, 'XALQ', (column_xb2, column_xb1, column_xb3, column_xb4, column_xb5, column_xb6, column_xb7, column_xb8, column_xb9, column_xb10, column_xb11, column_xb12, column_xb13, column_xb14, column_xb15, column_xb16, column_xb17, column_xb18, column_xb19, 
column_xb20,
column_xb21,
column_xb22,
column_xb23,
column_xb24,
column_xb25,
column_xb26,
column_xb27,
column_xb28,
column_xb29,
column_xb30,
column_xb31,
column_xb32,
column_xb33,
column_xb34,
column_xb35,
column_xb36,
column_xb37,
column_xb38,
column_xb39,
column_xb40,
column_xb41,
column_xb42,
column_xb43,
column_xb44,
column_xb45,
column_xb46,
column_xb47,
column_xb48,
column_xb49,
column_xb50,
column_xb51,
column_xb52,
column_xb53,
column_xb54,
column_xb55,
column_xb56,
column_xb57,
column_xb58,
column_xb59,
column_xb60,
column_xb61,
column_xb62,
column_xb63,
column_xb64,
column_xb65,
column_xb66,
column_xb67,
column_xb68,
column_xb69,
column_xb70,
column_xb71,
column_xb72,
column_xb73,
column_xb74,
column_xb75,
column_xb76,
column_xb77,
column_xb78,
column_xb79,
column_xb83,
column_xb84,
column_xb85,
column_xb89,
column_xb90,
column_xb91,
column_xb92))
# Вызов функции для вставки данных в таблицу POYAXT  
insert_data_into_table(conn, cursor, 'POYTAXT', (column_poytaxt2, column_poytaxt1, column_poytaxt3, column_poytaxt4, column_poytaxt5, column_poytaxt6, column_poytaxt7, column_poytaxt8, column_poytaxt9, column_poytaxt10, column_poytaxt11, column_poytaxt12, column_poytaxt13, column_poytaxt14, column_poytaxt15, column_poytaxt16, column_poytaxt17, column_poytaxt18, column_poytaxt19, 
column_poytaxt20,
column_poytaxt21,
column_poytaxt22,
column_poytaxt23,
column_poytaxt24,
column_poytaxt25,
column_poytaxt26,
column_poytaxt27,
column_poytaxt28,
column_poytaxt29,
column_poytaxt30,
column_poytaxt31,
column_poytaxt32,
column_poytaxt33,
column_poytaxt34,
column_poytaxt35,
column_poytaxt36,
column_poytaxt37,
column_poytaxt38,
column_poytaxt39,
column_poytaxt40,
column_poytaxt41,
column_poytaxt42,
column_poytaxt43,
column_poytaxt44,
column_poytaxt45,
column_poytaxt46,
column_poytaxt47,
column_poytaxt48,
column_poytaxt49,
column_poytaxt50,
column_poytaxt51,
column_poytaxt52,
column_poytaxt53,
column_poytaxt54,
column_poytaxt55,
column_poytaxt56,
column_poytaxt57,
column_poytaxt58,
column_poytaxt59,
column_poytaxt60,
column_poytaxt61,
column_poytaxt62,
column_poytaxt63,
column_poytaxt64,
column_poytaxt65,
column_poytaxt66,
column_poytaxt67,
column_poytaxt68,
column_poytaxt69,
column_poytaxt70,
column_poytaxt71,
column_poytaxt72,
column_poytaxt73,
column_poytaxt74,
column_poytaxt75,
column_poytaxt76,
column_poytaxt77,
column_poytaxt78,
column_poytaxt79,
column_poytaxt83,
column_poytaxt84,
column_poytaxt85,
column_poytaxt89,
column_poytaxt90,
column_poytaxt91,
column_poytaxt92))
# Вызов функции для вставки данных в таблицу SQB  
insert_data_into_table(conn, cursor, 'SQB', (column_sqb2, column_sqb1, column_sqb3, column_sqb4, column_sqb5, column_sqb6, column_sqb7, column_sqb8, column_sqb9, column_sqb10, column_sqb11, column_sqb12, column_sqb13, column_sqb14, column_sqb15, column_sqb16, column_sqb17, column_sqb18, column_sqb19, 
column_sqb20,
column_sqb21,
column_sqb22,
column_sqb23,
column_sqb24,
column_sqb25,
column_sqb26,
column_sqb27,
column_sqb28,
column_sqb29,
column_sqb30,
column_sqb31,
column_sqb32,
column_sqb33,
column_sqb34,
column_sqb35,
column_sqb36,
column_sqb37,
column_sqb38,
column_sqb39,
column_sqb40,
column_sqb41,
column_sqb42,
column_sqb43,
column_sqb44,
column_sqb45,
column_sqb46,
column_sqb47,
column_sqb48,
column_sqb49,
column_sqb50,
column_sqb51,
column_sqb52,
column_sqb53,
column_sqb54,
column_sqb55,
column_sqb56,
column_sqb57,
column_sqb58,
column_sqb59,
column_sqb60,
column_sqb61,
column_sqb62,
column_sqb63,
column_sqb64,
column_sqb65,
column_sqb66,
column_sqb67,
column_sqb68,
column_sqb69,
column_sqb70,
column_sqb71,
column_sqb72,
column_sqb73,
column_sqb74,
column_sqb75,
column_sqb76,
column_sqb77,
column_sqb78,
column_sqb79,
column_sqb83,
column_sqb84,
column_sqb85,
column_sqb89,
column_sqb90,
column_sqb91,
column_sqb92))
# Вызов функции для вставки данных в таблицу IPOTEKA  
insert_data_into_table(conn, cursor, 'IPOTEKABANK', (column_ipoteka2, column_ipoteka1, column_ipoteka3, column_ipoteka4, column_ipoteka5, column_ipoteka6, column_ipoteka7, column_ipoteka8, column_ipoteka9, column_ipoteka10, column_ipoteka11, column_ipoteka12, column_ipoteka13, column_ipoteka14, column_ipoteka15, column_ipoteka16, column_ipoteka17, column_ipoteka18, column_ipoteka19, 
column_ipoteka20,
column_ipoteka21,
column_ipoteka22,
column_ipoteka23,
column_ipoteka24,
column_ipoteka25,
column_ipoteka26,
column_ipoteka27,
column_ipoteka28,
column_ipoteka29,
column_ipoteka30,
column_ipoteka31,
column_ipoteka32,
column_ipoteka33,
column_ipoteka34,
column_ipoteka35,
column_ipoteka36,
column_ipoteka37,
column_ipoteka38,
column_ipoteka39,
column_ipoteka40,
column_ipoteka41,
column_ipoteka42,
column_ipoteka43,
column_ipoteka44,
column_ipoteka45,
column_ipoteka46,
column_ipoteka47,
column_ipoteka48,
column_ipoteka49,
column_ipoteka50,
column_ipoteka51,
column_ipoteka52,
column_ipoteka53,
column_ipoteka54,
column_ipoteka55,
column_ipoteka56,
column_ipoteka57,
column_ipoteka58,
column_ipoteka59,
column_ipoteka60,
column_ipoteka61,
column_ipoteka62,
column_ipoteka63,
column_ipoteka64,
column_ipoteka65,
column_ipoteka66,
column_ipoteka67,
column_ipoteka68,
column_ipoteka69,
column_ipoteka70,
column_ipoteka71,
column_ipoteka72,
column_ipoteka73,
column_ipoteka74,
column_ipoteka75,
column_ipoteka76,
column_ipoteka77,
column_ipoteka78,
column_ipoteka79,
column_ipoteka83,
column_ipoteka84,
column_ipoteka85,
column_ipoteka89,
column_ipoteka90,
column_ipoteka91,
column_ipoteka92))
# Вызов функции для вставки данных в таблицу AGROBANK  
insert_data_into_table(conn, cursor, 'AGROBANK', (column_agro2, column_agro1, column_agro3, column_agro4, column_agro5, column_agro6, column_agro7, column_agro8, column_agro9, column_agro10, column_agro11, column_agro12, column_agro13, column_agro14, column_agro15, column_agro16, column_agro17, column_agro18, column_agro19, 
column_agro20,
column_agro21,
column_agro22,
column_agro23,
column_agro24,
column_agro25,
column_agro26,
column_agro27,
column_agro28,
column_agro29,
column_agro30,
column_agro31,
column_agro32,
column_agro33,
column_agro34,
column_agro35,
column_agro36,
column_agro37,
column_agro38,
column_agro39,
column_agro40,
column_agro41,
column_agro42,
column_agro43,
column_agro44,
column_agro45,
column_agro46,
column_agro47,
column_agro48,
column_agro49,
column_agro50,
column_agro51,
column_agro52,
column_agro53,
column_agro54,
column_agro55,
column_agro56,
column_agro57,
column_agro58,
column_agro59,
column_agro60,
column_agro61,
column_agro62,
column_agro63,
column_agro64,
column_agro65,
column_agro66,
column_agro67,
column_agro68,
column_agro69,
column_agro70,
column_agro71,
column_agro72,
column_agro73,
column_agro74,
column_agro75,
column_agro76,
column_agro77,
column_agro78,
column_agro79,
column_agro83,
column_agro84,
column_agro85,
column_agro89,
column_agro90,
column_agro91,
column_agro92))
# Вызов функции для вставки данных в таблицу RBB  
insert_data_into_table(conn, cursor, 'RBB', (column_qqb2, column_qqb1, column_qqb3, column_qqb4, column_qqb5, column_qqb6, column_qqb7, column_qqb8, column_qqb9, column_qqb10, column_qqb11, column_qqb12, column_qqb13, column_qqb14, column_qqb15, column_qqb16, column_qqb17, column_qqb18, column_qqb19, 
column_qqb20,
column_qqb21,
column_qqb22,
column_qqb23,
column_qqb24,
column_qqb25,
column_qqb26,
column_qqb27,
column_qqb28,
column_qqb29,
column_qqb30,
column_qqb31,
column_qqb32,
column_qqb33,
column_qqb34,
column_qqb35,
column_qqb36,
column_qqb37,
column_qqb38,
column_qqb39,
column_qqb40,
column_qqb41,
column_qqb42,
column_qqb43,
column_qqb44,
column_qqb45,
column_qqb46,
column_qqb47,
column_qqb48,
column_qqb49,
column_qqb50,
column_qqb51,
column_qqb52,
column_qqb53,
column_qqb54,
column_qqb55,
column_qqb56,
column_qqb57,
column_qqb58,
column_qqb59,
column_qqb60,
column_qqb61,
column_qqb62,
column_qqb63,
column_qqb64,
column_qqb65,
column_qqb66,
column_qqb67,
column_qqb68,
column_qqb69,
column_qqb70,
column_qqb71,
column_qqb72,
column_qqb73,
column_qqb74,
column_qqb75,
column_qqb76,
column_qqb77,
column_qqb78,
column_qqb79,
column_qqb83,
column_qqb84,
column_qqb85,
column_qqb89,
column_qqb90,
column_qqb91,
column_qqb92))
# Вызов функции для вставки данных в таблицу TURON
insert_data_into_table(conn, cursor, 'TURON', (column_turon2, column_turon1, column_turon3, column_turon4, column_turon5, column_turon6, column_turon7, column_turon8, column_turon9, column_turon10, column_turon11, column_turon12, column_turon13, column_turon14, column_turon15, column_turon16, column_turon17, column_turon18, column_turon19, 
column_turon20,
column_turon21,
column_turon22,
column_turon23,
column_turon24,
column_turon25,
column_turon26,
column_turon27,
column_turon28,
column_turon29,
column_turon30,
column_turon31,
column_turon32,
column_turon33,
column_turon34,
column_turon35,
column_turon36,
column_turon37,
column_turon38,
column_turon39,
column_turon40,
column_turon41,
column_turon42,
column_turon43,
column_turon44,
column_turon45,
column_turon46,
column_turon47,
column_turon48,
column_turon49,
column_turon50,
column_turon51,
column_turon52,
column_turon53,
column_turon54,
column_turon55,
column_turon56,
column_turon57,
column_turon58,
column_turon59,
column_turon60,
column_turon61,
column_turon62,
column_turon63,
column_turon64,
column_turon65,
column_turon66,
column_turon67,
column_turon68,
column_turon69,
column_turon70,
column_turon71,
column_turon72,
column_turon73,
column_turon74,
column_turon75,
column_turon76,
column_turon77,
column_turon78,
column_turon79,
column_turon83,
column_turon84,
column_turon85,
column_turon89,
column_turon90,
column_turon91,
column_turon92))
# Вызов функции для вставки данных в таблицу MICRO
insert_data_into_table(conn, cursor, 'MICROCREDIT', (column_micro2, column_micro1, column_micro3, column_micro4, column_micro5, column_micro6, column_micro7, column_micro8, column_micro9, column_micro10, column_micro11, column_micro12, column_micro13, column_micro14, column_micro15, column_micro16, column_micro17, column_micro18, column_micro19, 
column_micro20,
column_micro21,
column_micro22,
column_micro23,
column_micro24,
column_micro25,
column_micro26,
column_micro27,
column_micro28,
column_micro29,
column_micro30,
column_micro31,
column_micro32,
column_micro33,
column_micro34,
column_micro35,
column_micro36,
column_micro37,
column_micro38,
column_micro39,
column_micro40,
column_micro41,
column_micro42,
column_micro43,
column_micro44,
column_micro45,
column_micro46,
column_micro47,
column_micro48,
column_micro49,
column_micro50,
column_micro51,
column_micro52,
column_micro53,
column_micro54,
column_micro55,
column_micro56,
column_micro57,
column_micro58,
column_micro59,
column_micro60,
column_micro61,
column_micro62,
column_micro63,
column_micro64,
column_micro65,
column_micro66,
column_micro67,
column_micro68,
column_micro69,
column_micro70,
column_micro71,
column_micro72,
column_micro73,
column_micro74,
column_micro75,
column_micro76,
column_micro77,
column_micro78,
column_micro79,
column_micro83,
column_micro84,
column_micro85,
column_micro89,
column_micro90,
column_micro91,
column_micro92))
# Вызов функции для вставки данных в таблицу ALOQA
insert_data_into_table(conn, cursor, 'ALOQA', (column_aloqa2, column_aloqa1, column_aloqa3, column_aloqa4, column_aloqa5, column_aloqa6, column_aloqa7, column_aloqa8, column_aloqa9, column_aloqa10, column_aloqa11, column_aloqa12, column_aloqa13, column_aloqa14, column_aloqa15, column_aloqa16, column_aloqa17, column_aloqa18, column_aloqa19, 
column_aloqa20,
column_aloqa21,
column_aloqa22,
column_aloqa23,
column_aloqa24,
column_aloqa25,
column_aloqa26,
column_aloqa27,
column_aloqa28,
column_aloqa29,
column_aloqa30,
column_aloqa31,
column_aloqa32,
column_aloqa33,
column_aloqa34,
column_aloqa35,
column_aloqa36,
column_aloqa37,
column_aloqa38,
column_aloqa39,
column_aloqa40,
column_aloqa41,
column_aloqa42,
column_aloqa43,
column_aloqa44,
column_aloqa45,
column_aloqa46,
column_aloqa47,
column_aloqa48,
column_aloqa49,
column_aloqa50,
column_aloqa51,
column_aloqa52,
column_aloqa53,
column_aloqa54,
column_aloqa55,
column_aloqa56,
column_aloqa57,
column_aloqa58,
column_aloqa59,
column_aloqa60,
column_aloqa61,
column_aloqa62,
column_aloqa63,
column_aloqa64,
column_aloqa65,
column_aloqa66,
column_aloqa67,
column_aloqa68,
column_aloqa69,
column_aloqa70,
column_aloqa71,
column_aloqa72,
column_aloqa73,
column_aloqa74,
column_aloqa75,
column_aloqa76,
column_aloqa77,
column_aloqa78,
column_aloqa79,
column_aloqa83,
column_aloqa84,
column_aloqa85,
column_aloqa89,
column_aloqa90,
column_aloqa91,
column_aloqa92))
# Вызов функции для вставки данных в таблицу IPAK
insert_data_into_table(conn, cursor, 'IPAKYULI', (column_ipak2, column_ipak1, column_ipak3, column_ipak4, column_ipak5, column_ipak6, column_ipak7, column_ipak8, column_ipak9, column_ipak10, column_ipak11, column_ipak12, column_ipak13, column_ipak14, column_ipak15, column_ipak16, column_ipak17, column_ipak18, column_ipak19, 
column_ipak20,
column_ipak21,
column_ipak22,
column_ipak23,
column_ipak24,
column_ipak25,
column_ipak26,
column_ipak27,
column_ipak28,
column_ipak29,
column_ipak30,
column_ipak31,
column_ipak32,
column_ipak33,
column_ipak34,
column_ipak35,
column_ipak36,
column_ipak37,
column_ipak38,
column_ipak39,
column_ipak40,
column_ipak41,
column_ipak42,
column_ipak43,
column_ipak44,
column_ipak45,
column_ipak46,
column_ipak47,
column_ipak48,
column_ipak49,
column_ipak50,
column_ipak51,
column_ipak52,
column_ipak53,
column_ipak54,
column_ipak55,
column_ipak56,
column_ipak57,
column_ipak58,
column_ipak59,
column_ipak60,
column_ipak61,
column_ipak62,
column_ipak63,
column_ipak64,
column_ipak65,
column_ipak66,
column_ipak67,
column_ipak68,
column_ipak69,
column_ipak70,
column_ipak71,
column_ipak72,
column_ipak73,
column_ipak74,
column_ipak75,
column_ipak76,
column_ipak77,
column_ipak78,
column_ipak79,
column_ipak83,
column_ipak84,
column_ipak85,
column_ipak89,
column_ipak90,
column_ipak91,
column_ipak92))
# Вызов функции для вставки данных в таблицу KAPITAL
insert_data_into_table(conn, cursor, 'KAPITAL', (column_kapital2, column_kapital1, column_kapital3, column_kapital4, column_kapital5, column_kapital6, column_kapital7, column_kapital8, column_kapital9, column_kapital10, column_kapital11, column_kapital12, column_kapital13, column_kapital14, column_kapital15, column_kapital16, column_kapital17, column_kapital18, column_kapital19, 
column_kapital20,
column_kapital21,
column_kapital22,
column_kapital23,
column_kapital24,
column_kapital25,
column_kapital26,
column_kapital27,
column_kapital28,
column_kapital29,
column_kapital30,
column_kapital31,
column_kapital32,
column_kapital33,
column_kapital34,
column_kapital35,
column_kapital36,
column_kapital37,
column_kapital38,
column_kapital39,
column_kapital40,
column_kapital41,
column_kapital42,
column_kapital43,
column_kapital44,
column_kapital45,
column_kapital46,
column_kapital47,
column_kapital48,
column_kapital49,
column_kapital50,
column_kapital51,
column_kapital52,
column_kapital53,
column_kapital54,
column_kapital55,
column_kapital56,
column_kapital57,
column_kapital58,
column_kapital59,
column_kapital60,
column_kapital61,
column_kapital62,
column_kapital63,
column_kapital64,
column_kapital65,
column_kapital66,
column_kapital67,
column_kapital68,
column_kapital69,
column_kapital70,
column_kapital71,
column_kapital72,
column_kapital73,
column_kapital74,
column_kapital75,
column_kapital76,
column_kapital77,
column_kapital78,
column_kapital79,
column_kapital83,
column_kapital84,
column_kapital85,
column_kapital89,
column_kapital90,
column_kapital91,
column_kapital92))
# Вызов функции для вставки данных в таблицу INVESTFINANCE
insert_data_into_table(conn, cursor, 'INVESTFINANCE', (column_infin2, column_infin1, column_infin3, column_infin4, column_infin5, column_infin6, column_infin7, column_infin8, column_infin9, column_infin10, column_infin11, column_infin12, column_infin13, column_infin14, column_infin15, column_infin16, column_infin17, column_infin18, column_infin19, 
column_infin20,
column_infin21,
column_infin22,
column_infin23,
column_infin24,
column_infin25,
column_infin26,
column_infin27,
column_infin28,
column_infin29,
column_infin30,
column_infin31,
column_infin32,
column_infin33,
column_infin34,
column_infin35,
column_infin36,
column_infin37,
column_infin38,
column_infin39,
column_infin40,
column_infin41,
column_infin42,
column_infin43,
column_infin44,
column_infin45,
column_infin46,
column_infin47,
column_infin48,
column_infin49,
column_infin50,
column_infin51,
column_infin52,
column_infin53,
column_infin54,
column_infin55,
column_infin56,
column_infin57,
column_infin58,
column_infin59,
column_infin60,
column_infin61,
column_infin62,
column_infin63,
column_infin64,
column_infin65,
column_infin66,
column_infin67,
column_infin68,
column_infin69,
column_infin70,
column_infin71,
column_infin72,
column_infin73,
column_infin74,
column_infin75,
column_infin76,
column_infin77,
column_infin78,
column_infin79,
column_infin83,
column_infin84,
column_infin85,
column_infin89,
column_infin90,
column_infin91,
column_infin92))
# Вызов функции для вставки данных в таблицу KAPITAL
insert_data_into_table(conn, cursor, 'UNIVERSAL', (column_universal2, column_universal1, column_universal3, column_universal4, column_universal5, column_universal6, column_universal7, column_universal8, column_universal9, column_universal10, column_universal11, column_universal12, column_universal13, column_universal14, column_universal15, column_universal16, column_universal17, column_universal18, column_universal19, 
column_universal20,
column_universal21,
column_universal22,
column_universal23,
column_universal24,
column_universal25,
column_universal26,
column_universal27,
column_universal28,
column_universal29,
column_universal30,
column_universal31,
column_universal32,
column_universal33,
column_universal34,
column_universal35,
column_universal36,
column_universal37,
column_universal38,
column_universal39,
column_universal40,
column_universal41,
column_universal42,
column_universal43,
column_universal44,
column_universal45,
column_universal46,
column_universal47,
column_universal48,
column_universal49,
column_universal50,
column_universal51,
column_universal52,
column_universal53,
column_universal54,
column_universal55,
column_universal56,
column_universal57,
column_universal58,
column_universal59,
column_universal60,
column_universal61,
column_universal62,
column_universal63,
column_universal64,
column_universal65,
column_universal66,
column_universal67,
column_universal68,
column_universal69,
column_universal70,
column_universal71,
column_universal72,
column_universal73,
column_universal74,
column_universal75,
column_universal76,
column_universal77,
column_universal78,
column_universal79,
column_universal83,
column_universal84,
column_universal85,
column_universal89,
column_universal90,
column_universal91,
column_universal92))
# Вызов функции для вставки данных в таблицу AVO
insert_data_into_table(conn, cursor, 'AVO', (column_avo2, column_avo1, column_avo3, column_avo4, column_avo5, column_avo6, column_avo7, column_avo8, column_avo9, column_avo10, column_avo11, column_avo12, column_avo13, column_avo14, column_avo15, column_avo16, column_avo17, column_avo18, column_avo19, 
column_avo20,
column_avo21,
column_avo22,
column_avo23,
column_avo24,
column_avo25,
column_avo26,
column_avo27,
column_avo28,
column_avo29,
column_avo30,
column_avo31,
column_avo32,
column_avo33,
column_avo34,
column_avo35,
column_avo36,
column_avo37,
column_avo38,
column_avo39,
column_avo40,
column_avo41,
column_avo42,
column_avo43,
column_avo44,
column_avo45,
column_avo46,
column_avo47,
column_avo48,
column_avo49,
column_avo50,
column_avo51,
column_avo52,
column_avo53,
column_avo54,
column_avo55,
column_avo56,
column_avo57,
column_avo58,
column_avo59,
column_avo60,
column_avo61,
column_avo62,
column_avo63,
column_avo64,
column_avo65,
column_avo66,
column_avo67,
column_avo68,
column_avo69,
column_avo70,
column_avo71,
column_avo72,
column_avo73,
column_avo74,
column_avo75,
column_avo76,
column_avo77,
column_avo78,
column_avo79,
column_avo83,
column_avo84,
column_avo85,
column_avo89,
column_avo90,
column_avo91,
column_avo92))
# Вызов функции для вставки данных в таблицу MADAD
insert_data_into_table(conn, cursor, 'MADADINVEST', (column_madad2, column_madad1, column_madad3, column_madad4, column_madad5, column_madad6, column_madad7, column_madad8, column_madad9, column_madad10, column_madad11, column_madad12, column_madad13, column_madad14, column_madad15, column_madad16, column_madad17, column_madad18, column_madad19, 
column_madad20,
column_madad21,
column_madad22,
column_madad23,
column_madad24,
column_madad25,
column_madad26,
column_madad27,
column_madad28,
column_madad29,
column_madad30,
column_madad31,
column_madad32,
column_madad33,
column_madad34,
column_madad35,
column_madad36,
column_madad37,
column_madad38,
column_madad39,
column_madad40,
column_madad41,
column_madad42,
column_madad43,
column_madad44,
column_madad45,
column_madad46,
column_madad47,
column_madad48,
column_madad49,
column_madad50,
column_madad51,
column_madad52,
column_madad53,
column_madad54,
column_madad55,
column_madad56,
column_madad57,
column_madad58,
column_madad59,
column_madad60,
column_madad61,
column_madad62,
column_madad63,
column_madad64,
column_madad65,
column_madad66,
column_madad67,
column_madad68,
column_madad69,
column_madad70,
column_madad71,
column_madad72,
column_madad73,
column_madad74,
column_madad75,
column_madad76,
column_madad77,
column_madad78,
column_madad79,
column_madad83,
column_madad84,
column_madad85,
column_madad89,
column_madad90,
column_madad91,
column_madad92))
# Вызов функции для вставки данных в таблицу OFB
insert_data_into_table(conn, cursor, 'OFB', (column_ofb2, column_ofb1, column_ofb3, column_ofb4, column_ofb5, column_ofb6, column_ofb7, column_ofb8, column_ofb9, column_ofb10, column_ofb11, column_ofb12, column_ofb13, column_ofb14, column_ofb15, column_ofb16, column_ofb17, column_ofb18, column_ofb19, 
column_ofb20,
column_ofb21,
column_ofb22,
column_ofb23,
column_ofb24,
column_ofb25,
column_ofb26,
column_ofb27,
column_ofb28,
column_ofb29,
column_ofb30,
column_ofb31,
column_ofb32,
column_ofb33,
column_ofb34,
column_ofb35,
column_ofb36,
column_ofb37,
column_ofb38,
column_ofb39,
column_ofb40,
column_ofb41,
column_ofb42,
column_ofb43,
column_ofb44,
column_ofb45,
column_ofb46,
column_ofb47,
column_ofb48,
column_ofb49,
column_ofb50,
column_ofb51,
column_ofb52,
column_ofb53,
column_ofb54,
column_ofb55,
column_ofb56,
column_ofb57,
column_ofb58,
column_ofb59,
column_ofb60,
column_ofb61,
column_ofb62,
column_ofb63,
column_ofb64,
column_ofb65,
column_ofb66,
column_ofb67,
column_ofb68,
column_ofb69,
column_ofb70,
column_ofb71,
column_ofb72,
column_ofb73,
column_ofb74,
column_ofb75,
column_ofb76,
column_ofb77,
column_ofb78,
column_ofb79,
column_ofb83,
column_ofb84,
column_ofb85,
column_ofb89,
column_ofb90,
column_ofb91,
column_ofb92))
# Вызов функции для вставки данных в таблицу TRAST
insert_data_into_table(conn, cursor, 'TRAST', (column_trust2, column_trust1, column_trust3, column_trust4, column_trust5, column_trust6, column_trust7, column_trust8, column_trust9, column_trust10, column_trust11, column_trust12, column_trust13, column_trust14, column_trust15, column_trust16, column_trust17, column_trust18, column_trust19, 
column_trust20,
column_trust21,
column_trust22,
column_trust23,
column_trust24,
column_trust25,
column_trust26,
column_trust27,
column_trust28,
column_trust29,
column_trust30,
column_trust31,
column_trust32,
column_trust33,
column_trust34,
column_trust35,
column_trust36,
column_trust37,
column_trust38,
column_trust39,
column_trust40,
column_trust41,
column_trust42,
column_trust43,
column_trust44,
column_trust45,
column_trust46,
column_trust47,
column_trust48,
column_trust49,
column_trust50,
column_trust51,
column_trust52,
column_trust53,
column_trust54,
column_trust55,
column_trust56,
column_trust57,
column_trust58,
column_trust59,
column_trust60,
column_trust61,
column_trust62,
column_trust63,
column_trust64,
column_trust65,
column_trust66,
column_trust67,
column_trust68,
column_trust69,
column_trust70,
column_trust71,
column_trust72,
column_trust73,
column_trust74,
column_trust75,
column_trust76,
column_trust77,
column_trust78,
column_trust79,
column_trust83,
column_trust84,
column_trust85,
column_trust89,
column_trust90,
column_trust91,
column_trust92))
# Вызов функции для вставки данных в таблицу DAVR
insert_data_into_table(conn, cursor, 'DAVR', (column_davr2, column_davr1, column_davr3, column_davr4, column_davr5, column_davr6, column_davr7, column_davr8, column_davr9, column_davr10, column_davr11, column_davr12, column_davr13, column_davr14, column_davr15, column_davr16, column_davr17, column_davr18, column_davr19, 
column_davr20,
column_davr21,
column_davr22,
column_davr23,
column_davr24,
column_davr25,
column_davr26,
column_davr27,
column_davr28,
column_davr29,
column_davr30,
column_davr31,
column_davr32,
column_davr33,
column_davr34,
column_davr35,
column_davr36,
column_davr37,
column_davr38,
column_davr39,
column_davr40,
column_davr41,
column_davr42,
column_davr43,
column_davr44,
column_davr45,
column_davr46,
column_davr47,
column_davr48,
column_davr49,
column_davr50,
column_davr51,
column_davr52,
column_davr53,
column_davr54,
column_davr55,
column_davr56,
column_davr57,
column_davr58,
column_davr59,
column_davr60,
column_davr61,
column_davr62,
column_davr63,
column_davr64,
column_davr65,
column_davr66,
column_davr67,
column_davr68,
column_davr69,
column_davr70,
column_davr71,
column_davr72,
column_davr73,
column_davr74,
column_davr75,
column_davr76,
column_davr77,
column_davr78,
column_davr79,
column_davr83,
column_davr84,
column_davr85,
column_davr89,
column_davr90,
column_davr91,
column_davr92))
# Вызов функции для вставки данных в таблицу RAVNAQ
insert_data_into_table(conn, cursor, 'OCTOBANK', (column_ravnaq2, column_ravnaq1, column_ravnaq3, column_ravnaq4, column_ravnaq5, column_ravnaq6, column_ravnaq7, column_ravnaq8, column_ravnaq9, column_ravnaq10, column_ravnaq11, column_ravnaq12, column_ravnaq13, column_ravnaq14, column_ravnaq15, column_ravnaq16, column_ravnaq17, column_ravnaq18, column_ravnaq19, 
column_ravnaq20,
column_ravnaq21,
column_ravnaq22,
column_ravnaq23,
column_ravnaq24,
column_ravnaq25,
column_ravnaq26,
column_ravnaq27,
column_ravnaq28,
column_ravnaq29,
column_ravnaq30,
column_ravnaq31,
column_ravnaq32,
column_ravnaq33,
column_ravnaq34,
column_ravnaq35,
column_ravnaq36,
column_ravnaq37,
column_ravnaq38,
column_ravnaq39,
column_ravnaq40,
column_ravnaq41,
column_ravnaq42,
column_ravnaq43,
column_ravnaq44,
column_ravnaq45,
column_ravnaq46,
column_ravnaq47,
column_ravnaq48,
column_ravnaq49,
column_ravnaq50,
column_ravnaq51,
column_ravnaq52,
column_ravnaq53,
column_ravnaq54,
column_ravnaq55,
column_ravnaq56,
column_ravnaq57,
column_ravnaq58,
column_ravnaq59,
column_ravnaq60,
column_ravnaq61,
column_ravnaq62,
column_ravnaq63,
column_ravnaq64,
column_ravnaq65,
column_ravnaq66,
column_ravnaq67,
column_ravnaq68,
column_ravnaq69,
column_ravnaq70,
column_ravnaq71,
column_ravnaq72,
column_ravnaq73,
column_ravnaq74,
column_ravnaq75,
column_ravnaq76,
column_ravnaq77,
column_ravnaq78,
column_ravnaq79,
column_ravnaq83,
column_ravnaq84,
column_ravnaq85,
column_ravnaq89,
column_ravnaq90,
column_ravnaq91,
column_ravnaq92))
# Вызов функции для вставки данных в таблицу ANOR
insert_data_into_table(conn, cursor, 'ANOR', (column_anor2, column_anor1, column_anor3, column_anor4, column_anor5, column_anor6, column_anor7, column_anor8, column_anor9, column_anor10, column_anor11, column_anor12, column_anor13, column_anor14, column_anor15, column_anor16, column_anor17, column_anor18, column_anor19, 
column_anor20,
column_anor21,
column_anor22,
column_anor23,
column_anor24,
column_anor25,
column_anor26,
column_anor27,
column_anor28,
column_anor29,
column_anor30,
column_anor31,
column_anor32,
column_anor33,
column_anor34,
column_anor35,
column_anor36,
column_anor37,
column_anor38,
column_anor39,
column_anor40,
column_anor41,
column_anor42,
column_anor43,
column_anor44,
column_anor45,
column_anor46,
column_anor47,
column_anor48,
column_anor49,
column_anor50,
column_anor51,
column_anor52,
column_anor53,
column_anor54,
column_anor55,
column_anor56,
column_anor57,
column_anor58,
column_anor59,
column_anor60,
column_anor61,
column_anor62,
column_anor63,
column_anor64,
column_anor65,
column_anor66,
column_anor67,
column_anor68,
column_anor69,
column_anor70,
column_anor71,
column_anor72,
column_anor73,
column_anor74,
column_anor75,
column_anor76,
column_anor77,
column_anor78,
column_anor79,
column_anor83,
column_anor84,
column_anor85,
column_anor89,
column_anor90,
column_anor91,
column_anor92))
# Вызов функции для вставки данных в таблицу UZUM
insert_data_into_table(conn, cursor, 'UZUM', (column_uzum2, column_uzum1, column_uzum3, column_uzum4, column_uzum5, column_uzum6, column_uzum7, column_uzum8, column_uzum9, column_uzum10, column_uzum11, column_uzum12, column_uzum13, column_uzum14, column_uzum15, column_uzum16, column_uzum17, column_uzum18, column_uzum19, 
column_uzum20,
column_uzum21,
column_uzum22,
column_uzum23,
column_uzum24,
column_uzum25,
column_uzum26,
column_uzum27,
column_uzum28,
column_uzum29,
column_uzum30,
column_uzum31,
column_uzum32,
column_uzum33,
column_uzum34,
column_uzum35,
column_uzum36,
column_uzum37,
column_uzum38,
column_uzum39,
column_uzum40,
column_uzum41,
column_uzum42,
column_uzum43,
column_uzum44,
column_uzum45,
column_uzum46,
column_uzum47,
column_uzum48,
column_uzum49,
column_uzum50,
column_uzum51,
column_uzum52,
column_uzum53,
column_uzum54,
column_uzum55,
column_uzum56,
column_uzum57,
column_uzum58,
column_uzum59,
column_uzum60,
column_uzum61,
column_uzum62,
column_uzum63,
column_uzum64,
column_uzum65,
column_uzum66,
column_uzum67,
column_uzum68,
column_uzum69,
column_uzum70,
column_uzum71,
column_uzum72,
column_uzum73,
column_uzum74,
column_uzum75,
column_uzum76,
column_uzum77,
column_uzum78,
column_uzum79,
column_uzum83,
column_uzum84,
column_uzum85,
column_uzum89,
column_uzum90,
column_uzum91,
column_uzum92))
# Вызов функции для вставки данных в таблицу APEX
insert_data_into_table(conn, cursor, 'APEX', (column_apex2, column_apex1, column_apex3, column_apex4, column_apex5, column_apex6, column_apex7, column_apex8, column_apex9, column_apex10, column_apex11, column_apex12, column_apex13, column_apex14, column_apex15, column_apex16, column_apex17, column_apex18, column_apex19, 
column_apex20,
column_apex21,
column_apex22,
column_apex23,
column_apex24,
column_apex25,
column_apex26,
column_apex27,
column_apex28,
column_apex29,
column_apex30,
column_apex31,
column_apex32,
column_apex33,
column_apex34,
column_apex35,
column_apex36,
column_apex37,
column_apex38,
column_apex39,
column_apex40,
column_apex41,
column_apex42,
column_apex43,
column_apex44,
column_apex45,
column_apex46,
column_apex47,
column_apex48,
column_apex49,
column_apex50,
column_apex51,
column_apex52,
column_apex53,
column_apex54,
column_apex55,
column_apex56,
column_apex57,
column_apex58,
column_apex59,
column_apex60,
column_apex61,
column_apex62,
column_apex63,
column_apex64,
column_apex65,
column_apex66,
column_apex67,
column_apex68,
column_apex69,
column_apex70,
column_apex71,
column_apex72,
column_apex73,
column_apex74,
column_apex75,
column_apex76,
column_apex77,
column_apex78,
column_apex79,
column_apex83,
column_apex84,
column_apex85,
column_apex89,
column_apex90,
column_apex91,
column_apex92))
# Вызов функции для вставки данных в таблицу HAYOT
insert_data_into_table(conn, cursor, 'HAYOT', (column_hayot2, column_hayot1, column_hayot3, column_hayot4, column_hayot5, column_hayot6, column_hayot7, column_hayot8, column_hayot9, column_hayot10, column_hayot11, column_hayot12, column_hayot13, column_hayot14, column_hayot15, column_hayot16, column_hayot17, column_hayot18, column_hayot19, 
column_hayot20,
column_hayot21,
column_hayot22,
column_hayot23,
column_hayot24,
column_hayot25,
column_hayot26,
column_hayot27,
column_hayot28,
column_hayot29,
column_hayot30,
column_hayot31,
column_hayot32,
column_hayot33,
column_hayot34,
column_hayot35,
column_hayot36,
column_hayot37,
column_hayot38,
column_hayot39,
column_hayot40,
column_hayot41,
column_hayot42,
column_hayot43,
column_hayot44,
column_hayot45,
column_hayot46,
column_hayot47,
column_hayot48,
column_hayot49,
column_hayot50,
column_hayot51,
column_hayot52,
column_hayot53,
column_hayot54,
column_hayot55,
column_hayot56,
column_hayot57,
column_hayot58,
column_hayot59,
column_hayot60,
column_hayot61,
column_hayot62,
column_hayot63,
column_hayot64,
column_hayot65,
column_hayot66,
column_hayot67,
column_hayot68,
column_hayot69,
column_hayot70,
column_hayot71,
column_hayot72,
column_hayot73,
column_hayot74,
column_hayot75,
column_hayot76,
column_hayot77,
column_hayot78,
column_hayot79,
column_hayot83,
column_hayot84,
column_hayot85,
column_hayot89,
column_hayot90,
column_hayot91,
column_hayot92))
# Вызов функции для вставки данных в таблицу SMART
insert_data_into_table(conn, cursor, 'SMART', (column_smart2, column_smart1, column_smart3, column_smart4, column_smart5, column_smart6, column_smart7, column_smart8, column_smart9, column_smart10, column_smart11, column_smart12, column_smart13, column_smart14, column_smart15, column_smart16, column_smart17, column_smart18, column_smart19, 
column_smart20,
column_smart21,
column_smart22,
column_smart23,
column_smart24,
column_smart25,
column_smart26,
column_smart27,
column_smart28,
column_smart29,
column_smart30,
column_smart31,
column_smart32,
column_smart33,
column_smart34,
column_smart35,
column_smart36,
column_smart37,
column_smart38,
column_smart39,
column_smart40,
column_smart41,
column_smart42,
column_smart43,
column_smart44,
column_smart45,
column_smart46,
column_smart47,
column_smart48,
column_smart49,
column_smart50,
column_smart51,
column_smart52,
column_smart53,
column_smart54,
column_smart55,
column_smart56,
column_smart57,
column_smart58,
column_smart59,
column_smart60,
column_smart61,
column_smart62,
column_smart63,
column_smart64,
column_smart65,
column_smart66,
column_smart67,
column_smart68,
column_smart69,
column_smart70,
column_smart71,
column_smart72,
column_smart73,
column_smart74,
column_smart75,
column_smart76,
column_smart77,
column_smart78,
column_smart79,
column_smart83,
column_smart84,
column_smart85,
column_smart89,
column_smart90,
column_smart91,
column_smart92))
# Вызов функции для вставки данных в таблицу YANGI
insert_data_into_table(conn, cursor, 'YANGI', (column_yangi2, column_yangi1, column_yangi3, column_yangi4, column_yangi5, column_yangi6, column_yangi7, column_yangi8, column_yangi9, column_yangi10, column_yangi11, column_yangi12, column_yangi13, column_yangi14, column_yangi15, column_yangi16, column_yangi17, column_yangi18, column_yangi19, 
column_yangi20,
column_yangi21,
column_yangi22,
column_yangi23,
column_yangi24,
column_yangi25,
column_yangi26,
column_yangi27,
column_yangi28,
column_yangi29,
column_yangi30,
column_yangi31,
column_yangi32,
column_yangi33,
column_yangi34,
column_yangi35,
column_yangi36,
column_yangi37,
column_yangi38,
column_yangi39,
column_yangi40,
column_yangi41,
column_yangi42,
column_yangi43,
column_yangi44,
column_yangi45,
column_yangi46,
column_yangi47,
column_yangi48,
column_yangi49,
column_yangi50,
column_yangi51,
column_yangi52,
column_yangi53,
column_yangi54,
column_yangi55,
column_yangi56,
column_yangi57,
column_yangi58,
column_yangi59,
column_yangi60,
column_yangi61,
column_yangi62,
column_yangi63,
column_yangi64,
column_yangi65,
column_yangi66,
column_yangi67,
column_yangi68,
column_yangi69,
column_yangi70,
column_yangi71,
column_yangi72,
column_yangi73,
column_yangi74,
column_yangi75,
column_yangi76,
column_yangi77,
column_yangi78,
column_yangi79,
column_yangi83,
column_yangi84,
column_yangi85,
column_yangi89,
column_yangi90,
column_yangi91,
column_yangi92))
# Вызов функции для вставки данных в таблицу HAMKOR
insert_data_into_table(conn, cursor, 'HAMKOR', (column_hamkor2, column_hamkor1, column_hamkor3, column_hamkor4, column_hamkor5, column_hamkor6, column_hamkor7, column_hamkor8, column_hamkor9, column_hamkor10, column_hamkor11, column_hamkor12, column_hamkor13, column_hamkor14, column_hamkor15, column_hamkor16, column_hamkor17, column_hamkor18, column_hamkor19, 
column_hamkor20,
column_hamkor21,
column_hamkor22,
column_hamkor23,
column_hamkor24,
column_hamkor25,
column_hamkor26,
column_hamkor27,
column_hamkor28,
column_hamkor29,
column_hamkor30,
column_hamkor31,
column_hamkor32,
column_hamkor33,
column_hamkor34,
column_hamkor35,
column_hamkor36,
column_hamkor37,
column_hamkor38,
column_hamkor39,
column_hamkor40,
column_hamkor41,
column_hamkor42,
column_hamkor43,
column_hamkor44,
column_hamkor45,
column_hamkor46,
column_hamkor47,
column_hamkor48,
column_hamkor49,
column_hamkor50,
column_hamkor51,
column_hamkor52,
column_hamkor53,
column_hamkor54,
column_hamkor55,
column_hamkor56,
column_hamkor57,
column_hamkor58,
column_hamkor59,
column_hamkor60,
column_hamkor61,
column_hamkor62,
column_hamkor63,
column_hamkor64,
column_hamkor65,
column_hamkor66,
column_hamkor67,
column_hamkor68,
column_hamkor69,
column_hamkor70,
column_hamkor71,
column_hamkor72,
column_hamkor73,
column_hamkor74,
column_hamkor75,
column_hamkor76,
column_hamkor77,
column_hamkor78,
column_hamkor79,
column_hamkor83,
column_hamkor84,
column_hamkor85,
column_hamkor89,
column_hamkor90,
column_hamkor91,
column_hamkor92))
# Вызов функции для вставки данных в таблицу QDB
insert_data_into_table(conn, cursor, 'QDB', (column_kdb2, column_kdb1, column_kdb3, column_kdb4, column_kdb5, column_kdb6, column_kdb7, column_kdb8, column_kdb9, column_kdb10, column_kdb11, column_kdb12, column_kdb13, column_kdb14, column_kdb15, column_kdb16, column_kdb17, column_kdb18, column_kdb19, 
column_kdb20,
column_kdb21,
column_kdb22,
column_kdb23,
column_kdb24,
column_kdb25,
column_kdb26,
column_kdb27,
column_kdb28,
column_kdb29,
column_kdb30,
column_kdb31,
column_kdb32,
column_kdb33,
column_kdb34,
column_kdb35,
column_kdb36,
column_kdb37,
column_kdb38,
column_kdb39,
column_kdb40,
column_kdb41,
column_kdb42,
column_kdb43,
column_kdb44,
column_kdb45,
column_kdb46,
column_kdb47,
column_kdb48,
column_kdb49,
column_kdb50,
column_kdb51,
column_kdb52,
column_kdb53,
column_kdb54,
column_kdb55,
column_kdb56,
column_kdb57,
column_kdb58,
column_kdb59,
column_kdb60,
column_kdb61,
column_kdb62,
column_kdb63,
column_kdb64,
column_kdb65,
column_kdb66,
column_kdb67,
column_kdb68,
column_kdb69,
column_kdb70,
column_kdb71,
column_kdb72,
column_kdb73,
column_kdb74,
column_kdb75,
column_kdb76,
column_kdb77,
column_kdb78,
column_kdb79,
column_kdb83,
column_kdb84,
column_kdb85,
column_kdb89,
column_kdb90,
column_kdb91,
column_kdb92))
# Вызов функции для вставки данных в таблицу QDB
insert_data_into_table(conn, cursor, 'GARANT', (column_garant2, column_garant1, column_garant3, column_garant4, column_garant5, column_garant6, column_garant7, column_garant8, column_garant9, column_garant10, column_garant11, column_garant12, column_garant13, column_garant14, column_garant15, column_garant16, column_garant17, column_garant18, column_garant19, 
column_garant20,
column_garant21,
column_garant22,
column_garant23,
column_garant24,
column_garant25,
column_garant26,
column_garant27,
column_garant28,
column_garant29,
column_garant30,
column_garant31,
column_garant32,
column_garant33,
column_garant34,
column_garant35,
column_garant36,
column_garant37,
column_garant38,
column_garant39,
column_garant40,
column_garant41,
column_garant42,
column_garant43,
column_garant44,
column_garant45,
column_garant46,
column_garant47,
column_garant48,
column_garant49,
column_garant50,
column_garant51,
column_garant52,
column_garant53,
column_garant54,
column_garant55,
column_garant56,
column_garant57,
column_garant58,
column_garant59,
column_garant60,
column_garant61,
column_garant62,
column_garant63,
column_garant64,
column_garant65,
column_garant66,
column_garant67,
column_garant68,
column_garant69,
column_garant70,
column_garant71,
column_garant72,
column_garant73,
column_garant74,
column_garant75,
column_garant76,
column_garant77,
column_garant78,
column_garant79,
column_garant83,
column_garant84,
column_garant85,
column_garant89,
column_garant90,
column_garant91,
column_garant92))
# Вызов функции для вставки данных в таблицу ZIRAT
insert_data_into_table(conn, cursor, 'ZIRAT', (column_zirat2, column_zirat1, column_zirat3, column_zirat4, column_zirat5, column_zirat6, column_zirat7, column_zirat8, column_zirat9, column_zirat10, column_zirat11, column_zirat12, column_zirat13, column_zirat14, column_zirat15, column_zirat16, column_zirat17, column_zirat18, column_zirat19, 
column_zirat20,
column_zirat21,
column_zirat22,
column_zirat23,
column_zirat24,
column_zirat25,
column_zirat26,
column_zirat27,
column_zirat28,
column_zirat29,
column_zirat30,
column_zirat31,
column_zirat32,
column_zirat33,
column_zirat34,
column_zirat35,
column_zirat36,
column_zirat37,
column_zirat38,
column_zirat39,
column_zirat40,
column_zirat41,
column_zirat42,
column_zirat43,
column_zirat44,
column_zirat45,
column_zirat46,
column_zirat47,
column_zirat48,
column_zirat49,
column_zirat50,
column_zirat51,
column_zirat52,
column_zirat53,
column_zirat54,
column_zirat55,
column_zirat56,
column_zirat57,
column_zirat58,
column_zirat59,
column_zirat60,
column_zirat61,
column_zirat62,
column_zirat63,
column_zirat64,
column_zirat65,
column_zirat66,
column_zirat67,
column_zirat68,
column_zirat69,
column_zirat70,
column_zirat71,
column_zirat72,
column_zirat73,
column_zirat74,
column_zirat75,
column_zirat76,
column_zirat77,
column_zirat78,
column_zirat79,
column_zirat83,
column_zirat84,
column_zirat85,
column_zirat89,
column_zirat90,
column_zirat91,
column_zirat92))
# Вызов функции для вставки данных в таблицу SADERAT
insert_data_into_table(conn, cursor, 'SADERAT', (column_saderat2, column_saderat1, column_saderat3, column_saderat4, column_saderat5, column_saderat6, column_saderat7, column_saderat8, column_saderat9, column_saderat10, column_saderat11, column_saderat12, column_saderat13, column_saderat14, column_saderat15, column_saderat16, column_saderat17, column_saderat18, column_saderat19, 
column_saderat20,
column_saderat21,
column_saderat22,
column_saderat23,
column_saderat24,
column_saderat25,
column_saderat26,
column_saderat27,
column_saderat28,
column_saderat29,
column_saderat30,
column_saderat31,
column_saderat32,
column_saderat33,
column_saderat34,
column_saderat35,
column_saderat36,
column_saderat37,
column_saderat38,
column_saderat39,
column_saderat40,
column_saderat41,
column_saderat42,
column_saderat43,
column_saderat44,
column_saderat45,
column_saderat46,
column_saderat47,
column_saderat48,
column_saderat49,
column_saderat50,
column_saderat51,
column_saderat52,
column_saderat53,
column_saderat54,
column_saderat55,
column_saderat56,
column_saderat57,
column_saderat58,
column_saderat59,
column_saderat60,
column_saderat61,
column_saderat62,
column_saderat63,
column_saderat64,
column_saderat65,
column_saderat66,
column_saderat67,
column_saderat68,
column_saderat69,
column_saderat70,
column_saderat71,
column_saderat72,
column_saderat73,
column_saderat74,
column_saderat75,
column_saderat76,
column_saderat77,
column_saderat78,
column_saderat79,
column_saderat83,
column_saderat84,
column_saderat85,
column_saderat89,
column_saderat90,
column_saderat91,
column_saderat92))
# Вызов функции для вставки данных в таблицу SADERAT
insert_data_into_table(conn, cursor, 'TBC', (column_tbc2, column_tbc1, column_tbc3, column_tbc4, column_tbc5, column_tbc6, column_tbc7, column_tbc8, column_tbc9, column_tbc10, column_tbc11, column_tbc12, column_tbc13, column_tbc14, column_tbc15, column_tbc16, column_tbc17, column_tbc18, column_tbc19, 
column_tbc20,
column_tbc21,
column_tbc22,
column_tbc23,
column_tbc24,
column_tbc25,
column_tbc26,
column_tbc27,
column_tbc28,
column_tbc29,
column_tbc30,
column_tbc31,
column_tbc32,
column_tbc33,
column_tbc34,
column_tbc35,
column_tbc36,
column_tbc37,
column_tbc38,
column_tbc39,
column_tbc40,
column_tbc41,
column_tbc42,
column_tbc43,
column_tbc44,
column_tbc45,
column_tbc46,
column_tbc47,
column_tbc48,
column_tbc49,
column_tbc50,
column_tbc51,
column_tbc52,
column_tbc53,
column_tbc54,
column_tbc55,
column_tbc56,
column_tbc57,
column_tbc58,
column_tbc59,
column_tbc60,
column_tbc61,
column_tbc62,
column_tbc63,
column_tbc64,
column_tbc65,
column_tbc66,
column_tbc67,
column_tbc68,
column_tbc69,
column_tbc70,
column_tbc71,
column_tbc72,
column_tbc73,
column_tbc74,
column_tbc75,
column_tbc76,
column_tbc77,
column_tbc78,
column_tbc79,
column_tbc83,
column_tbc84,
column_tbc85,
column_tbc89,
column_tbc90,
column_tbc91,
column_tbc92))
# Вызов функции для вставки данных в таблицу TENGE
insert_data_into_table(conn, cursor, 'TENGE', (column_tenge2, column_tenge1, column_tenge3, column_tenge4, column_tenge5, column_tenge6, column_tenge7, column_tenge8, column_tenge9, column_tenge10, column_tenge11, column_tenge12, column_tenge13, column_tenge14, column_tenge15, column_tenge16, column_tenge17, column_tenge18, column_tenge19, 
column_tenge20,
column_tenge21,
column_tenge22,
column_tenge23,
column_tenge24,
column_tenge25,
column_tenge26,
column_tenge27,
column_tenge28,
column_tenge29,
column_tenge30,
column_tenge31,
column_tenge32,
column_tenge33,
column_tenge34,
column_tenge35,
column_tenge36,
column_tenge37,
column_tenge38,
column_tenge39,
column_tenge40,
column_tenge41,
column_tenge42,
column_tenge43,
column_tenge44,
column_tenge45,
column_tenge46,
column_tenge47,
column_tenge48,
column_tenge49,
column_tenge50,
column_tenge51,
column_tenge52,
column_tenge53,
column_tenge54,
column_tenge55,
column_tenge56,
column_tenge57,
column_tenge58,
column_tenge59,
column_tenge60,
column_tenge61,
column_tenge62,
column_tenge63,
column_tenge64,
column_tenge65,
column_tenge66,
column_tenge67,
column_tenge68,
column_tenge69,
column_tenge70,
column_tenge71,
column_tenge72,
column_tenge73,
column_tenge74,
column_tenge75,
column_tenge76,
column_tenge77,
column_tenge78,
column_tenge79,
column_tenge83,
column_tenge84,
column_tenge85,
column_tenge89,
column_tenge90,
column_tenge91,
column_tenge92))


Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: '22'!$A:$K.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: '23'!$A:$C.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: '3'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: '22'!$A:$K.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\User\AppData\Local\Programs\Python\P

Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.
Ошибка: Невозможно преобразовать значение в число.


C:\Users\User\AppData\Local\Temp\ipykernel_14684\565118028.py:27: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute(insert_script, data)
C:\Users\User\AppData\Local\Temp\ipykernel_14684\8930645.py:92: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute(insert_script, data)


In [10]:
conn.close()